The first code cell defines the Trace, Operator, Basis and CorrelatorCalculator classes. The second cell defines various functions that use these. There are then several cells of code that load different bases of operators. Some of these can take up to half an hour to run. After that, there's lots of rough working. There will be useful code in there, but it's not organised in any way.

To anyone reading this, sorry for how messy it all is, and good luck working it out. If you're trying to understand the code and are struggling, feel free to open an issue on the repository so I get notified, and I'll be happy to help.

In [1]:
import numpy as np
#declare N as a variable
var('N')

#Class to model a single multi-trace
class Trace:
    def __init__(self, fieldContent, permutation, fieldPositions=None):
        
        self.fieldContent = fieldContent
        m = fieldContent[0]
        n = fieldContent[1]
        
        if type(permutation) == sage.groups.perm_gps.permgroup_element.PermutationGroupElement:
            self.permutation = permutation
        else:
            self.permutation = PermutationGroupElement(permutation, parent=SymmetricGroup(m+n))
            
        if fieldPositions != None:
            if fieldPositions.count(1) != m or fieldPositions.count(2) != n:
                raise Exception('fieldPositions must match the given field content')
            perm = permFromFieldPositions(fieldPositions)
            self.permutation = perm * self.permutation * perm
            
    def __repr__(self):
        matrixWords = []
        for cycle in self.permutation.cycle_tuples(singletons=True):
            word = ''
            for i in cycle:
                if i <= self.fieldContent[0]:
                    word += 'X'
                else:
                    word += 'Y'
            
            wordRotations = [word]
            for i in range(len(word)-1):
                word = word[-1] + word[:-1]
                wordRotations.append(word)
            matrixWords.append(min(wordRotations))
        matrixWords.sort()
        return ''.join(['(Tr' + word + ')' for word in matrixWords])
        
            
    def __mul__(self, other):
        if isinstance(other, Trace):
            m1 = self.fieldContent[0]
            m2 = other.fieldContent[0]
            n1 = self.fieldContent[1]
            n2 = other.fieldContent[1]
            subperm1 = self.permutation([i+1 for i in range(m1)] + [i+1 for i in range(m1+m2,m1+m2+n1)])
            subperm2 = other.permutation([i+1 for i in range(m1,m1+m2)] + [i+1 for i in range(m1+m2+n1,m1+m2+n1+n2)])
            return Trace([m1+m2, n1+n2], subperm1[:m1] + subperm2[:m2] + subperm1[m1:] + subperm2[m2:])
        elif isscalar(other):
            return Operator(self.fieldContent, [other], traces=[self])
        else:
            return NotImplemented
        
    def __rmul__(self,other):
        return self * other
    
    def __div__(self, other):
        if isscalar(other):
            return self * (1/other)
        else:
            return NotImplemented
            
    def __eq__(self, other):
        if isinstance(other, Trace):
            return str(self) == str(other)
        else:
            return NotImplemented
            
    #Creates an identical copy of the trace
    def copy(self):
        return Trace(self.fieldContent, self.permutation)

            
#Class to model a linear combination of multi-trace operators    
class Operator:
    def __init__(self, fieldContent, coefficients, traces=None, permutations=None, fieldPositions=None):
        self.fieldContent = fieldContent
        m = fieldContent[0]
        n = fieldContent[1]
        
        if permutations == None and traces == None:
            raise Exception('You must provide either a list of permutations or a list of traces')
        elif permutations != None and traces !=None:
            raise Exception('You cannot provide both a list of permutations and a list of traces')
        elif permutations != None:
            if len(permutations) != len(coefficients):
                raise Exception('Lists of permutations and coefficients must have the same length')
            else:
                traces = [ Trace(fieldContent, permutations[i]) 
                          for i in range(len(permutations)) if coefficients[i] != 0 ]
                
        elif traces != None:
            if len(traces) != len(coefficients):
                raise Exception('Lists of traces and coefficients must have the same length')
            if not all([ tr.fieldContent == fieldContent for tr in traces ]):
                raise Exception('Traces must match the given field content')
            traces = [ traces[i] for i in range(len(coefficients)) if coefficients[i] != 0]
            
        self.traces = traces
        self.coefficients = [ coefficients[i] for i in range(len(coefficients)) if coefficients[i] != 0 ]
        
        if fieldPositions != None:
            perm = permFromFieldPositions(fieldPositions)
            for tr in self.traces:
                tr.permutation = perm * tr.permutation * perm
            
    
    
    def __add__(self, other):
        if isinstance(other, Operator):
            if self.fieldContent != other.fieldContent:
                raise Exception('Cannot add operators with different field contents')
            newoperator = self.copy()
            for cf, tr in other:
                if tr in newoperator.traces:
                    index = newoperator.traces.index(tr)
                    newoperator.coefficients[index] += cf
                    if newoperator.coefficients[index] == 0:
                        newoperator.coefficients.pop(index)
                        newoperator.traces.pop(index)
                else:
                    newoperator.traces.append(tr.copy())
                    newoperator.coefficients.append(cf)
            return newoperator
        else:
            return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        if isinstance(other, Operator):
            return self + (-1 * other)
        else:
            return NotImplemented
        
    def __mul__(self, other):
        if isscalar(other):
            coefficients = [ other * cf for cf in self.coefficients]
            return Operator(self.fieldContent, coefficients, self.traces)
        elif isinstance(other, Trace):
            traces = [ other * tr for tr in self.traces ]
            fieldContent = [ self.fieldContent[i] + other.fieldContent[i] for i in range(2)]
            return Operator(fieldContent, self.coefficients, traces)
        elif isinstance(other, Operator):
            newFieldContent = [self.fieldContent[i] + other.fieldContent[i] for i in range(2)]
            newoperator = Operator(newFieldContent, [], [])
            for cf1, tr1 in self:
                for cf2, tr2 in other:
                    newoperator += cf1 * cf2 * tr1 * tr2
            return newoperator
        else:
            return NotImplemented
        
    def __rmul__(self, other):
        return self * other
    
    def __div__(self, other):
        if isscalar(other):
            return self * (1/other)
        else:
            return NotImplemented
    
    
    def __len__(self):
        return len(self.traces)
        
    def __repr__(self):
        string = ''
        for cf, tr in self:
            if type(cf) == sage.symbolic.expression.Expression:
                string += ' + (' + str(cf) + ')' + str(tr)
            elif cf == 1:
                string += ' + ' + str(tr)
            elif cf == -1:
                string += ' - ' + str(tr)
            elif cf >= 0:
                string += ' + ' + str(cf) + str(tr)
            else:
                string += ' - ' + str(-cf) + str(tr)
        if string[0:2] == ' -':
            string = string[1] + string[3:]
        else:
            string = string[3:]
        return string
        
    def __iter__(self):
        return iter([ (self.coefficients[i], self.traces[i]) for i in range(len(self.traces))])
        
    def __contains__(self, trace):
        return trace in self.traces
        
    #Creates an identical copy of the operator
    def copy(self):
        traces = [ tr.copy() for tr in self.traces]
        return Operator(self.fieldContent, coefficients=self.coefficients, traces=traces)
    
    #For an operator with symbolic coefficients, factorises the coefficients
    def factorCoefficients(self):
        for i in range(len(self.coefficients)):
            if type(self.coefficients[i]) == sage.symbolic.expression.Expression:
                self.coefficients[i] = self.coefficients[i].factor()
    
    #For an operator with symbolic coefficients, neatens square roots in the coefficients
    def canonicalizeRadicalsInCoefficients(self):
        for i in range(len(self.coefficients)):
            if type(self.coefficients[i]) == sage.symbolic.expression.Expression:
                self.coefficients[i] = self.coefficients[i].canonicalize_radical()
    
    #For an operator with symbolic coefficients, expands the coefficients
    def expandCoefficients(self):
        for i in range(len(self.coefficients)):
            self.coefficients[i] = self.coefficients[i].expand()
    
    #Takes the defining permutation of a trace and multiplies it on the left by another permutation
    def leftmultiply(self, perm):
        if type(perm) != sage.groups.perm_gps.permgroup_element.PermutationGroupElement:
            perm = PermutationGroupElement(perm, parent=SymmetricGroup(sum(self.fieldContent)))
        for tr in self.traces:
            tr.permutation = perm * tr.permutation
        return
    
    #Takes the defining permutation of a trace and multiplies it on the right by another permutation
    def rightmultiply(self, perm):
        if type(perm) != sage.groups.perm_gps.permgroup_element.PermutationGroupElement:
            perm = PermutationGroupElement(perm, parent=SymmetricGroup(sum(self.fieldContent)))
        for tr in self.traces:
            tr.permutation = tr.permutation * perm
        return
    
    #Takes the defining permutation of a trace and conjugates it by another permutation
    def conjugate(self, perm):
        if type(perm) != sage.groups.perm_gps.permgroup_element.PermutationGroupElement:
            perm = PermutationGroupElement(perm, parent=SymmetricGroup(sum(self.fieldContent)))
        self.leftmultiply(perm)
        self.rightmultiply(perm.inverse())
        return
        
        
        
#Class representing a basis of operators        
class Basis:
    def __init__(self, fieldContent, operators, labels=None):
        self.fieldContent = fieldContent
        if not all([op.fieldContent == fieldContent for op in operators]):
            raise Exception('Operators must match the given field content')
        self.operators = operators
        if labels != None and len(labels) != len(operators):
            raise Exception('Lists of labels and operators must have the same length')
        self.labels = labels
        self.labelled = (labels != None)
        
    def __len__(self):
        return len(self.operators)
        
    def __repr__(self):
        string = ''
        if self.labels == None:
            for op in self.operators:
                string += str(op) + '\n'
        else:
            maxlength = max([len(str(lbl)) for lbl in self.labels])
            for i in range(len(self.operators)):
                label = str(self.labels[i])
                string += label + ': ' + (' ' * (maxlength - len(label))) + str(self.operators[i]) + '\n' 
        return string
        
    def __iter__(self):
        if self.labelled:
            return iter([ (self.labels[i], self.operators[i]) for i in range(len(self.operators))])
        return iter(self.operators)
        
    def __getitem__(self,key):
        if type(key) == sage.rings.integer.Integer or type(key) == int:
            return self.operators[key]
        elif key in self.labels:
            return self.operators[self.labels.index(key)]
        else:
            return NotImplemented
    
    #Creates an identical copy of the basis
    def copy(self):
        if self.labelled:
            return Basis(self.fieldContent, [ op.copy() for op in self.operators ], self.labels[:])
        else:
            return Basis(self.fieldContent, [ op.copy() for op in self.operators ])
    
    # returns a linear combination of the basis
    def linearCombination(self, coefficients):
        if len(self) == len(coefficients):
            newoperator = Operator(self.fieldContent, [], [])
            for i in range(len(self)):
                newoperator += coefficients[i] * self.operators[i]
            return newoperator
        else:
            raise Exception('The length of the list of coefficients must match the size of the basis')
    
    # returns a new basis obtained from the old by transforming with the given matrix
    def transform(self, matrix):
        operators = [ self.linearCombination(matrix[i]) for i in range(len(matrix))]
        return Basis(self.fieldContent, operators)
    
    # permutes the order of the basis elements
    def permute(self, permutation):
        if type(permutation) != sage.groups.perm_gps.permgroup_element.PermutationGroupElement:
            permutation = PermutationGroupElement(permutation, parent=SymmetricGroup(len(self)))
        if self.labelled:
            return Basis(self.fieldContent, permutation(self.operators), permutation(self.labels))
        return Basis(self.fieldContent, permutation(self.operators))
    
    #expands the  given operator as a linear combination of the basis elements. 
    #If the given operator is not in the span, returns None
    def expand(self, operator):
        tracelist = []
        if self.labelled:
            for lbl, op in self:
                for cf, tr in op:
                    if tr not in tracelist:
                        tracelist.append(tr)
        else:
            for op in self:
                for cf, tr in op:
                    if tr not in tracelist:
                        tracelist.append(tr)
        tracecoefficients = [ [0 for tr in tracelist] for op in self]
        for i in range(len(self)):
            for cf, tr in self[i]:
                tracecoefficients[i][tracelist.index(tr)] = cf
        matrix = Matrix(tracecoefficients).T
        
        coefficients = [0 for tr in tracelist]
        for cf, tr in operator:
            coefficients[tracelist.index(tr)] = cf
        v = vector(coefficients)
        try:
            output = matrix.solve_right(v)
        except ValueError:
            output = None
        
        return output
    
# a helper class to calculate correlators 
class correlatorCalculator:
    def __init__(self, fieldContent):
        self.fieldContent = fieldContent
        m = fieldContent[0]
        n = fieldContent[1]
        self.G = SymmetricGroup(m+n)
        self.characters = self.G.irreducible_characters()
        self.conjClasses = self.G.conjugacy_classes()
        self.cycleType = [c[0].cycle_type() for c in self.conjClasses]
        self.ccSizes = [len(c) for c in self.conjClasses]
        self.ccNo = len(self.ccSizes)
        self.representPerms = self.G.conjugacy_classes_representatives()
        self.characterTable = [[R(g) for g in self.representPerms] for R in self.characters] 
    
        fact = factorial(m+n)
        self.omegaCharacters = [sum([N ** len(self.cycleType[i]) * 
                                     self.characterTable[j][i] * 
                                     self.ccSizes[i] for i in range(self.ccNo)]).factor() for j in range(self.ccNo)]
        self.Gcoefficients = [self.characterTable[i][0]^2 / (fact * self.omegaCharacters[i]) for i in range(self.ccNo)]
        self.Fcoefficients = [ 1/fact * self.omegaCharacters[i] for i in range(self.ccNo) ]
            
        subgroupGenerators=[]
        for i in range(m-1):
            permImage = [j+1 for j in range(m+n)]
            permImage[i] = i+2
            permImage[i+1] = i+1 
            subgroupGenerators.append(PermutationGroupElement(permImage))
        for i in range(n-1):
            permImage = [j+1 for j in range(m+n)]
            permImage[m+i] = m+i+2
            permImage[m+i+1] = m+i+1
            subgroupGenerators.append(PermutationGroupElement(permImage))
        self.H = self.G.subgroup(subgroupGenerators)
        
        self.cachedTraces = { 'F':[] , 'planar':[], 'G':[] }
        self.cachedCorrelators = { 'F':[] , 'planar':[], 'G':[] }
        
    # caches the correlator of two traces to speed up later calculations
    def addTraceToCache(self, trace, correlatorType):
        currentLength = len(self.cachedTraces[correlatorType])
        self.cachedTraces[correlatorType].append(trace)
        for i in range(currentLength):
            self.cachedCorrelators[correlatorType][i].append(None)
        self.cachedCorrelators[correlatorType].append([None for i in range(currentLength + 1)])
        
    # calculates the correlator of two traces, operators or bases (in this case it returns a matrix of inner products); if the 
    # inputs are different classes, it deals with this appropriately
    def calculate(self, input1, input2, correlatorType='G'):
        if self.fieldContent != input1.fieldContent or self.fieldContent != input2.fieldContent:
            raise Exception('Inputs must have the appropriate field content')
        if correlatorType not in ['F', 'planar', 'G']:
            raise Exception('Must choose the planar, F-weighted or G-weighted correlator')
        
        if isinstance(input2, Basis):
            return [ self.calculate(input1, op, correlatorType) for op in input2 ]
        
        if isinstance(input1, Basis):
            return [ self.calculate(op, input2, correlatorType) for op in input1 ]
        
        corr = 0
        
        if isinstance(input2, Operator):
            for cf, tr in input2:
                corr += cf * self.calculate(input1, tr, correlatorType)
            if correlatorType in ['F','G']:
                return corr.canonicalize_radical().factor()
            return corr
        
        if isinstance(input1, Operator):
            for cf, tr in input1:
                corr += cf * self.calculate(tr, input2, correlatorType)
            if correlatorType in ['F','G']:
                return corr.canonicalize_radical().factor()
            return corr
        
        if isinstance(input1, Trace) and isinstance(input2, Trace):
            if input1 not in self.cachedTraces[correlatorType]:
                self.addTraceToCache(input1, correlatorType)
                
            if input2 not in self.cachedTraces[correlatorType]:
                self.addTraceToCache(input2, correlatorType)
                
            index1 = self.cachedTraces[correlatorType].index(input1)
            index2 = self.cachedTraces[correlatorType].index(input2)
            if self.cachedCorrelators[correlatorType][index1][index2] != None:
                return self.cachedCorrelators[correlatorType][index1][index2]
            
            if correlatorType == 'F':
                perm1inv = input1.permutation.inverse()
                for h in self.H:
                    index = self.cycleType.index( (perm1inv * h * input2.permutation * h.inverse()).cycle_type() )
                    corr += sum([self.Fcoefficients[i] * self.characterTable[i][index] for i in range(self.ccNo)])
                corr = corr.factor()
            if correlatorType == 'planar':
                for h in self.H:
                    if input1.permutation == h * input2.permutation * h.inverse():
                        corr += 1
            if correlatorType == 'G':
                perm1inv = input1.permutation.inverse()
                for h in self.H:
                    index = self.cycleType.index( (perm1inv * h * input2.permutation * h.inverse()).cycle_type() )
                    corr += sum([self.Gcoefficients[i] * self.characterTable[i][index] for i in range(self.ccNo)])
                corr = corr.factor()
                    
            self.cachedCorrelators[correlatorType][index1][index2] = corr
            self.cachedCorrelators[correlatorType][index2][index1] = corr
            
            return corr
           
        raise Exception('Inputs must be traces, operators or bases')

# class to model a vector in (V_2)^{n}. Consists of a list of 1s and 2s that represent the positions of the X and Y fields
# respectively
class FieldPositionsVector:
    def __init__(self, fieldContent, fieldPositionsList, coefficients):
        if len(fieldPositionsList) != len(coefficients):
            raise Exception('Lists must be the same length')
        if not (all([fp.count(1) == fieldContent[0] for fp in fieldPositionsList]) 
                and all([fp.count(2) == fieldContent[1] for fp in fieldPositionsList])):
            raise Exception('Each set of field positions must match the given field content')
        self.fieldContent = fieldContent
        self.fieldPositionsList = [fieldPositionsList[i] for i in range(len(coefficients)) if coefficients[i] != 0]
        self.coefficients = [coefficients[i] for i in range(len(coefficients)) if coefficients[i] != 0]
        
    def __len__(self):
        return len(self.coefficients)
        
    def __add__(self, other):
        if isinstance(other, FieldPositionsVector):
            if self.fieldContent != other.fieldContent:
                raise Exception('Cannot add objects with different field contents')
            new = self.copy()
            for cf, fp in other:
                if fp in new.fieldPositionsList:
                    index = new.fieldPositionsList.index(fp)
                    new.coefficients[index] += cf
                    if new.coefficients[index] == 0:
                        new.coefficients.pop(index)
                        new.fieldPositionsList.pop(index)
                else:
                    new.fieldPositionsList.append(fp[:])
                    new.coefficients.append(cf)
            return new
        else:
            return NotImplemented
        
    def __sub__(self, other):
        return self + (-1 * other)
        
    def __mul__(self, other):
        if isscalar(other):
            coefficients = [ other * cf for cf in self.coefficients ]
            fieldPositionsList = [ fp[:] for fp in self.fieldPositionsList ]
            return FieldPositionsVector(self.fieldContent, fieldPositionsList, coefficients)
        else:
            return NotImplemented
        
    def __rmul__(self, other):
        return self * other
    
    def __div__(self, other):
        return self * (1/other)
        
    def __iter__(self):
        return iter([(self.coefficients[i], self.fieldPositionsList[i]) for i in range(len(self.coefficients))])
    
    def __contains__(self, fp):
        return fp in self.fieldPositionsList
    
    
    def __repr__(self):
        string = ''
        for cf, fp in self:
            fpstring = '(' + ''.join([str(i) for i in fp]) + ')'
            if cf == 1:
                string += ' + ' + fpstring
            elif cf == -1:
                string += ' - ' + fpstring
            elif cf >= 0:
                string += ' + ' + str(cf) + fpstring
            else:
                string += ' - ' + str(-cf) + fpstring
        if string[0:2] == ' -':
            string = string[1] + string[3:]
        else:
            string = string[3:]
        return string
        
    #creates an identical copy
    def copy(self):
        fieldPositionsList = [ fp[:] for fp in self.fieldPositionsList ]
        coefficients = self.coefficients[:]
        return FieldPositionsVector(self.fieldContent, fieldPositionsList, coefficients)
    
    # Inner product in (V_2)^n
    def dot(self, other):
        innerProduct = 0
        for cf, fp in other:
            if fp in self:
                index = self.fieldPositionsList.index(fp)
                innerProduct += cf * self.coefficients[index]
        return innerProduct
    
    # Apply the U(2) lowering operator to a vector in (V_2)^n
    def lower(self):
        if self.fieldContent[0] == 0:
            raise Exception('Cannot lower any further')
        new = FieldPositionsVector([self.fieldContent[0] - 1, self.fieldContent[1]+1], [], [])
        for cf, fp in self:
            coefficients = [ cf for i in range(self.fieldContent[0])]
            fpList = []
            for i in range(len(fp)):
                if fp[i] == 1:
                    newfp = fp[:]
                    newfp[i] = 2
                    fpList.append(newfp)
            new += FieldPositionsVector([self.fieldContent[0] - 1, self.fieldContent[1]+1], fpList, coefficients)
        return new
    
    # normalises the vector
    def normalise(self):
        norm = sqrt(self.dot(self))
        self.coefficients = [ cf/norm for cf in self.coefficients]
        return 
    
    # permutes the list of 1s and 2s
    def permute(self, permutation):
        if type(permutation) != sage.groups.perm_gps.permgroup_element.PermutationGroupElement:
            permutation = PermutationGroupElement(permutation, parent=SymmetricGroup(sum(self.fieldContent)))
        return FieldPositionsVector(self.fieldContent, [permutation(fp) for fp in self.fieldPositionsList],
                                       self.coefficients)

In [2]:
# Given a list of 1s and 2s, produces a permutation (of order 2) that places
#    the 1s at the beginning and the 2s at the end
def permFromFieldPositions(fieldPositions):
    m = fieldPositions.count(1)
    n = fieldPositions.count(2)
    pairsOutOfPlace = []
    for i in range(m):
        if fieldPositions[i] != 1:
            pairsOutOfPlace.append([i])
    index = 0
    for j in range(m,m+n):
        if fieldPositions[j] != 2:
            pairsOutOfPlace[index].append(j)
            index += 1
    pairsOutOfPlaceTuples = [(pair[0]+1,pair[1]+1) for pair in pairsOutOfPlace]
    perm = PermutationGroupElement(pairsOutOfPlaceTuples,parent=SymmetricGroup(m+n))
    return perm

# determines is a variable is a scalar
def isscalar(x):
    return (type(x) == sage.symbolic.expression.Expression or 
              type(x) == sage.rings.rational.Rational or 
              type(x) == sage.rings.integer.Integer)

# returns the single trace constructed from a lyndon word repeated an integer number of times
def singleTrace(repeats, lyndonword):
    m = repeats * lyndonword.count(1)
    n = repeats * lyndonword.count(2)
    permutation = []
    xcount = 0
    ycount = 0
    for i in range(repeats):
        index = xcount
        xcount += 1
        for letter in lyndonword[1:]:
            if letter == 1:
                permutation.insert(index, xcount + 1)
                xcount += 1
                index = xcount - 1
            else:
                permutation.insert(index, m + ycount + 1)
                ycount += 1
                index = len(permutation)
        permutation += [xcount + 1]
    permutation[-1] = 1
    return Trace([m,n], permutation)

# returns the possible single traces of a given field content
def singleTraces(fieldContent):
    traces = []
    for d in divisors(gcd(fieldContent)):
        for w in LyndonWords([Integer(fieldContent[0]/d), Integer(fieldContent[1]/d)]):
            traces.append(singleTrace(d,w))
    return traces

# returns the trace basis of operators
def traceBasis(fieldContent):
    operators = []
    for partition in VectorPartitions(fieldContent):
        currentTraces = singleTraces(partition[0])
        for vector in partition[1:]:
            newTraces = []
            for tr1 in singleTraces(vector):
                for tr2 in currentTraces:
                    newTrace = tr1 * tr2
                    if newTrace not in newTraces:
                        newTraces.append(newTrace)
            currentTraces = newTraces
        operators += [ 1 * tr for tr in currentTraces]
    return Basis(fieldContent, operators)

# returns the symmetrisation of a single trace with the given field content
def symmetrisedSingleTrace(fieldContent):
    m = fieldContent[0]
    n = fieldContent[1]
    
    if m <= 1 or n <= 1:
        permutation = [i+2 for i in range(m+n-1)] + [1]
        return 1 * Trace(fieldContent, permutation)
    
    coefficients = []
    traces = []
    binom = binomial(sum(fieldContent), fieldContent[0])
    for d in divisors(gcd(fieldContent)):
        for w in LyndonWords([Integer(fieldContent[0]/d), Integer(fieldContent[1]/d)]):
            traces.append(singleTrace(d,w))
            coefficients.append(sum(fieldContent)/(d * binom))
    return Operator(fieldContent, coefficients, traces)

# returns the symmetrised multi-trace corresponding to the given vector partition
def symmetrisedTrace(vectorPartition):
    operator = symmetrisedSingleTrace(vectorPartition[0])
    for vector in vectorPartition[1:]:
        operator *= symmetrisedSingleTrace(vector)
    return operator

# lists the vector partitions of a given field content
def VectorPartitionList(fieldContent, reverse=False):
    partitionList = [[ v for v in p] for p in VectorPartitions(fieldContent)]
    partitionList.sort(key=len,reverse=reverse)
    for p in partitionList:
        p.sort(reverse=True)
    return partitionList

# returns the symmetrised traces of a given field content
def symmetrisedTraceBasis(fieldContent):
    labels = VectorPartitionList(fieldContent)
    operators = [ symmetrisedTrace(p) for p in labels ]
    return Basis(fieldContent, operators, labels)



# Gives the matrix of coefficients transforming from the multi-symmetric function basis to the symmetrised trace basis
def cMatrix(fieldContent):
    m = fieldContent[0]
    n = fieldContent[1]
    partitionList = VectorPartitionList(fieldContent)
    noPartitions = len(partitionList)
        
    matrixCoefficients=np.zeros((noPartitions,noPartitions))

    for index in range(noPartitions):
        
        #The vector partition we're summing at the moment (p in our notation)
        partition=partitionList[index]

        plength=len(partition)

        #Cumulative sums of the parts of p
        cumulativeSum=np.cumsum(partition,axis=0)

        #The options for sums of p after each layer, along with their multiplicities. The 0th layer is initialised to be the 0 partition with multiplicity 1.
        psumoptions =[[[ [ [0,0] for i in range(plength) ], 1 ]]] + [ [] for i in range(plength) ]

        #Adds the separate parts of p one by one. At each stage, the possible partitions are found and listed with multiplicities in psumoptions[layer]
        for layer in range(plength):
            #Find the possible partitions after the next part of p has been added. Each parition is given trailing zeros so they all have the same length
            layerPartitionList=VectorPartitionList(cumulativeSum[layer])
            for i in range(len(layerPartitionList)):
                layerPartitionList[i] += [[0,0] for j in range(plength - len(layerPartitionList[i]))]
            layerPartitionMultiplicities=[[p,0] for p in layerPartitionList]
            #Takes the possible partitions from the previous layer and adds the next term in p to each position, thereby finding the multiplicities for this layer
            for j in range(len(psumoptions[layer])):
                for i in range(plength):
                    psumtemp=np.array(psumoptions[layer][j][0])
                    psumtemp[i] += np.array(partition[layer])
                    psumtemp = psumtemp.tolist()
                    psumtemp.sort(reverse=True)
                    layerPartitionMultiplicities[layerPartitionList.index(psumtemp)][1]+=psumoptions[layer][j][1]
            #If the multiplicity is greater than 1, add it to the list of options for this layer
            for m in layerPartitionMultiplicities:
                if m[1]>0:
                    psumoptions[layer+1].append(m)

        #Truncates the partitions by removing the trailing zeros
        for k in psumoptions[plength]:
            for i in range(k[0].count([0,0])):
                k[0].remove([0,0])
            #Dividing through by a redundancy factor
            factor=1
            for i in range(plength-len(k[0])+1,plength+1):
                factor*=i
            matrixCoefficients[partitionList.index(k[0]),index]=k[1]/factor
    
    return matrixCoefficients

# Gives the matrix of coefficients from the symmetrised trace basis to the multi-symmetric basis
def cTildeMatrix(fieldContent):
    c = np.linalg.inv(cMatrix(fieldContent))
    cInteger = [ [ 0 for i in range(len(c))] for j in range(len(c)) ]
    for i in range(len(c)):
        for j in range(len(c)):
            cInteger[i][j] = Integer(round(c[i,j]))
    return cInteger

# Generates the multi-symmetric function operator for a given partition
def multiSymmetricFunction(partition):
    m = np.sum(partition,axis=0)[0]
    n = np.sum(partition,axis=0)[1]
    cTilde = cTildeMatrix([m,n])
    partitionList = VectorPartitionList([m,n])
    operator = Operator([m,n], [], [])
    for i in range(len(partitionList)):
        c = cTilde[i][partitionList.index(partition)]
        if c != 0:
            operator += c * symmetrisedTrace(partitionList[i])
    return operator
    
# Finds the multi-symmetric function basis of operators
def multiSymmetricBasis(fieldContent):
    labels = VectorPartitionList(fieldContent)
    stBasis = symmetrisedTraceBasis(fieldContent)
    c = cTildeMatrix(fieldContent)
    cT = [ [c[j][i] for j in range(len(c)) ] for i in range(len(c))]
    operators = [ stBasis.linearCombination(cT[i]) for i in range(len(c))]
    return Basis(fieldContent, operators, labels)

# Determines the Lambda and R that label the U(2)-covariant free field basis along with their multiplicity 
def lambdaRmultiplicities(n):
    partitions = list(Partitions(n))
    partitions.reverse()
    NoPartitions = len(partitions)
    G = SymmetricGroup(n)
    characterTable = G.character_table()
    ccSizes = [ len(c) for c in G.conjugacy_classes() ]
    multiplicities = []
    for i in range(NoPartitions):
        if len(partitions[i]) <= 2:
            for j in range(NoPartitions):
                multiplicity = (1/factorial(n)) * sum([ ccSizes[k] * characterTable[i,k] * 
                                                        characterTable[j,k] * characterTable[j,k] 
                                                        for k in range(NoPartitions) ])
                if multiplicity != 0:
                    multiplicities.append((partitions[i],partitions[j], multiplicity))
    multiplicities.reverse()
    return multiplicities

# lists the possible vectors in (V_2)^n
def PossibleFieldPositions(fieldContent):
    fieldPositionsList = []
    combs = Combinations(range(sum(fieldContent)), fieldContent[1])
    for comb in combs:
        fp = [ 1 for j in range(fieldContent[0]) ]
        for j in comb:
            fp.insert(j,2)
        fieldPositionsList.append(fp)
    return fieldPositionsList

# Generates the Young basis for (V_2)^n
def VectorGenerator(n):    
    partitions = Partitions(n)
    lambdaPartitions = [ p for p in partitions if len(p) <= 2 ]
    
    basis = []
    labels = []

    for Lambda in lambdaPartitions:
        lambdatableaux = SemistandardTableaux(size=n, max_entry=2, shape=Lambda)
        topstate = lambdatableaux[0]
        fieldContent = topstate.evaluation()
        if len(fieldContent) == 1:
            fieldContent.append(0)
        symmetrisedFieldPositions = []
        for i in range(fieldContent[1]+1):
            firsthalf = PossibleFieldPositions([fieldContent[0]-i, i])
            secondhalf = PossibleFieldPositions([i,fieldContent[1]-i])
            for fp2 in secondhalf:
                fplist = [fp1 + fp2 for fp1 in firsthalf]
                coefficients = [1 for fp1 in firsthalf]
                symmetrisedFieldPositions.append(FieldPositionsVector(fieldContent, fplist, coefficients))
        
        Rtableaux = list(StandardTableaux(Lambda))
        Rtableaux.reverse()
        
        vectorsToOrthogonalise = [i for i in range(len(labels)) 
                                        if labels[i][0].evaluation() == fieldContent 
                                        and all([labels[i][1][0][j] == j+1 for j in range(fieldContent[0])])]
        
        dummy = True
        while dummy:
            randomSymmetrisedFieldPositions = FieldPositionsVector(fieldContent, [], [])
            for fp in symmetrisedFieldPositions:
                randomSymmetrisedFieldPositions += ZZ.random_element(1,10) * fp
            for i in vectorsToOrthogonalise:
                randomSymmetrisedFieldPositions -= randomSymmetrisedFieldPositions.dot(basis[i]) * basis[i]
            dummy = ( len(randomSymmetrisedFieldPositions) == 0 )
        
        randomSymmetrisedFieldPositions.normalise()
        labels.append([topstate, Rtableaux[0]])
        basis.append(randomSymmetrisedFieldPositions)
        
        permutations = []
        for i in range(n-2):
            perm = [j+1 for j in range(n)]
            perm[i+1] = i+3
            perm[i+2] = i+2
            permutations.append(perm)
        
        found = [True] + [ False for i in range(len(Rtableaux)-1) ]
        tested = [False for i in range(len(Rtableaux))]
        basisindices = [len(basis)-1] + [None for i in range(len(Rtableaux)-1)]
        testindex = 0
    
        while not all(found):
            r = Rtableaux[testindex]
            basiselement = basis[basisindices[testindex]]
            for i in range(len(permutations)):
                newr = r.symmetric_group_action_on_entries(permutations[i])
                if newr.is_standard():
                    tableauindex = Rtableaux.index(newr)
                    if not found[tableauindex]:
                        position1 = r.cells_containing(i+2)[0]
                        position2 = r.cells_containing(i+3)[0]
                        distance = Integer(position2[0] - position1[0] + position1[1] - position2[1])
                        
                        newbasiselement = 1/sqrt(distance^2 - 1) * ( distance *
                                                        basiselement.permute(permutations[i]) +  basiselement)
                        basis.append(newbasiselement)
                        labels.append([topstate, newr])
                        basisindices[tableauindex] = len(basis)-1
                        found[tableauindex] = True
            tested[testindex] = True
            while tested[testindex] or not found[testindex]:
                testindex += 1
                
        for state in lambdatableaux[1:]:
            for i in range(len(Rtableaux)):
                basiselement = basis[basisindices[i]]
                newbasiselement = basiselement.lower()
                newbasiselement.normalise()
                basis.append(newbasiselement)
                labels.append([state,Rtableaux[i]])
                basisindices[i] += len(Rtableaux)
    return (labels, basis)

# Gives the U(2)-covariant basis of free-field operators, with caveats. They are unnormalised, and if there is a non-trivial
# multiplicity for a given Lambda, R then this returns the appropriate number of vectors, but they won't be orthogonal
def GL2Basis(fieldContent):
    calculator = correlatorCalculator(fieldContent)
    print('Calculator created')

    fplabels, fpbasis = VectorGenerator(sum(fieldContent))
    multiplicities = lambdaRmultiplicities(sum(fieldContent))
    print('Basis labels and multiplicities are:')
    print(multiplicities)
    if fieldContent[1] == 0:
        fieldContent.pop()
    fpbasis = [fpbasis[i] for i in range(len(fplabels)) if fplabels[i][0].evaluation() == fieldContent] 
    fplabels = [lbl for lbl in fplabels if lbl[0].evaluation() == fieldContent]
    if len(fieldContent) == 1:
        fieldContent.append(0)
    partitions = list(Partitions(sum(fieldContent)))
    lambdaPartitions = [ p for p in partitions if len(p) == 1 or (len(p) == 2 and p[1] <= fieldContent[1])]
    fpvectors = []
    for Lambda in lambdaPartitions:
        fpvectors.append([ fpbasis[i] for i in range(len(fplabels)) if fplabels[i][0].shape() == Lambda])
    print('Setup lambda vectors')
    
    G = SymmetricGroup(sum(fieldContent))
    representations = [ SymmetricGroupRepresentation(p, 'orthogonal') for p in partitions]
    operators = []
    labels = []
    for Lambda, R, mult in multiplicities:
        print(Lambda, R)
        if Lambda in lambdaPartitions:
            lambdaindex = lambdaPartitions.index(Lambda)
            Rindex = partitions.index(R)
            rep = representations[Rindex]
            dim = rep(G[0]).dimensions()[0]
            fpvectorindex = len(fpvectors[lambdaindex]) - 1
            for multindex in range(mult):
                print(multindex)
                dummy = True
                while dummy:
                    fpvector = fpvectors[lambdaindex][fpvectorindex]
                    print('attempting with vector index ' + str(fpvectorindex))
                    randomcoefficients = [ ZZ.random_element(1,5) for i in range(dim) ]
                    coefficients = []
                    permutations = []
                    for cf, fp in fpvector:
                        h = permFromFieldPositions(fp)
                        for g in G:
                            matrix = rep(g)
                            coefficients.append(cf * sum([matrix[0,i] * randomcoefficients[i] for i in range(dim)]))
                            permutations.append(h * g * h)
                    op = Operator(fieldContent,[],[]) + Operator(fieldContent, coefficients, permutations=permutations)
                    op.canonicalizeRadicalsInCoefficients()
                    op.expandCoefficients()
                    dummy = (len(op) == 0)
                    fpvectorindex -= 1
                operators.append(op)
                labels.append((Lambda, R, multindex+1))
            print('')
    return Basis(fieldContent, operators=operators, labels=labels)

# given a string generated by the trace class __str__ function, reconstructs the trace
def traceFromString(string):
    wordlist = [ s[3:] for s in string.split(')') ][:-1]
    
    concat = ''.join(wordlist)
    fieldcontent = [concat.count('X'), concat.count('Y')]
    
    tuplelist = []
    Xs_used = 0
    Ys_used = 0
    for w in wordlist:
        if len(w) >= 2:
            totuple = []
            for letter in w:
                if letter == 'X':
                    totuple.append(Xs_used + 1)
                    Xs_used += 1
                elif letter == 'Y':
                    totuple.append(fieldcontent[0] + Ys_used + 1)
                    Ys_used += 1
                else:
                    raise Exception('Word should consist only of Xs and Ys')
            tuplelist.append(tuple(totuple))
    return Trace(fieldcontent, tuplelist)

# Applies the U(2) lowering operator to a trace
def lowerTrace(trace):
    if trace.fieldContent[0] == 0:
        raise Exception('Cannot lower operator any further')
    else:
        fieldContent = [trace.fieldContent[0]-1, trace.fieldContent[1]+1]
        op = Operator(fieldContent,[],[])
        trstring = str(trace)
        startindex = 0
        while trstring[startindex:].count('X') > 0:
            xindex = trstring[startindex:].index('X')
            startindex += xindex + 1
            newtrstring = trstring[:startindex-1] + 'Y' + trstring[startindex:]
            op += 1 * traceFromString(newtrstring)
        return op

# Applies the U(2) lowering operator to an operator
def lowerOperator(op):
    if op.fieldContent[0] == 0:
        raise Exception('Cannot lower operator any further')
    else:
        fieldContent = [op.fieldContent[0]-1, op.fieldContent[1]+1]
        newop = Operator(fieldContent,[],[])
        for cf, tr in op:
            newop += cf * lowerTrace(tr)
        return newop

# changes the sign of those traces whose defining permutation is odd
def changeSign(op):
    newop = op.copy()
    for i in range(len(newop)):
        newop.coefficients[i] *= newop.traces[i].permutation.sign()
    return newop

#Generates a string that will represent the trace, operator or basis in Latex. Some editing is sometimes still necessary
def latexString(op, operatorSymbol='O', labelType=None):
    if isinstance(op, Basis):
        basis = op
        
        if basis.labelled:
            output = '\\ytableausetup{boxsize=7pt}\n'
            output += '\\begin{align*}\n'
            for lbl, op1 in basis:
                output += operatorSymbol + '_{\n'
                for i in range(len(labelType)):
                    if labelType[i] == 'semi-standard tableau':
                        content = [1 if j < basis.fieldContent[0] else 2 for j in range(sum(basis.fieldContent))]
                        output += '\\fontsize{6pt}{0} \\begin{ytableau}\n'
                        for j in range(lbl[i][0]):
                            output += str(content[j]) + ' & '
                        output = output[:-2]
                        if len(lbl[i]) > 1 and lbl[i][1] > 0:
                            output += '\\\\ '
                            for j in range(lbl[i][1]):
                                output += str(content[j + lbl[i][0]]) + ' & '
                            output = output[:-2]
                        output += '\n\\end{ytableau} \\fontsize{12pt}{0} ,\n'
                    if labelType[i] == 'partition':
                        output += str(lbl[i])
                        output += ' ,\n'
                    if labelType[i] == 'multiplicity':
                        output += str(lbl[i])
                        output += ' ,\n'
                output = output[:-2]
                output += '} & =\n'
                cf = gcd(op1.coefficients)
                if cf != 1:
                    output += latex(cf) + ' \\left[\n'
                    output += latexString(1/cf * op1) + '\\right] \\\\\n'
                else:
                    output += latexString(op1) + '\\\\\n'
            output += '\\end{align*}\n'
            output += '\\ytableausetup{boxsize=normal}'
            return output
        else:
            return NotImplemented
    
    elif isinstance(op, Operator):
        output = ''
        for cf, tr in op:
            try: 
                cf = Rational(cf)
            except TypeError:
                cf = cf
            if type(cf) == sage.symbolic.expression.Expression:
                output += '+ \\left(' + latex(cf) + '\\right) '
            else: 
                if cf > 0:
                    output += '+ '
                else:
                    output += '- '
            
                if abs(cf) != 1:
                    output += latex(abs(cf)) + ' '
            output += latexString(tr) + '\n'
        if output[0:2] == '+ ':
            output = output[2:]
        return output
            
        
    elif isinstance(op, Trace):
        
        # Turn the permutation into a list of matrix words
        matrixWords = []
        for cycle in op.permutation.cycle_tuples(singletons=True):
            word = ''
            for i in cycle:
                if i <= op.fieldContent[0]:
                    word += 'X'
                else:
                    word += 'Y'
            
            wordRotations = [word]
            for i in range(len(word)-1):
                word = word[-1] + word[:-1]
                wordRotations.append(word)
            matrixWords.append(min(wordRotations))
        
        # Find multiplicity of each unique matrix word
        wordcounts = {}
        for w in matrixWords:
            if w in wordcounts:
                wordcounts[w] += 1
            else:
                wordcounts.update({w:1})
        
        output = ''
        words = wordcounts.keys()
        words.sort()
        for word in words:
            count = wordcounts[word]
            # Find Lyndon word and exponent for each unique matrix word. 
            for d in divisors(len(word)):
                prefix = word[:d]
                for i in range(len(word)/d -1):
                    prefix += word[:d]
                if prefix == word:
                    break
            lyndonword = word[:d]
            exp = len(word)/d
            
            # Turn each Lyndon word into a latex string (e.g. XXXYY -> X^3 Y^2)
            currentletter = lyndonword[0]
            index = 1
            letters = [currentletter]
            blocks = []
            while index < len(lyndonword):
                if lyndonword[index] != currentletter:
                    blocks.append(index - sum(blocks))
                    currentletter = lyndonword[index]
                    letters.append(currentletter)
                index += 1
            blocks.append(index - sum(blocks))
            string = ''
            for i in range(len(blocks)):
                string += letters[i]
                if blocks[i] > 1:
                    string += '^' + str(blocks[i])
                string += ' '
            string = string[:-1]
            
            # Add latex for brackets and exponents
            if exp > 1:
                if len(lyndonword) > 1:
                    string = '\\left( '+string+' \\right)'
                string += '^'+str(exp)
            string = '\\tr ' + string
            
            if count > 1:
                string = '\\left( '+ string + ' \\right)^' + str(count)
        
            output += string + ' '
            
        
        return output

# generates the U(2)-covariant basis of multi-symmetric functions
def GL2multiSymmetricBasis(fieldContent):
    totalFieldContent = sum(fieldContent)
    partitionList = list(Partitions(sum(fieldContent)))

    operators = []
    labels = []
    for ycontent in range(fieldContent[1]+1):
        currentFieldContent = [totalFieldContent-ycontent,ycontent]
        calculator = correlatorCalculator(currentFieldContent)
        for i in range(len(operators)):
            operators[i] = lowerOperator(operators[i])
            operators[i] /= sqrt(calculator.calculate(operators[i],operators[i],'planar'))
            operators[i].canonicalizeRadicalsInCoefficients()
        basis = multiSymmetricBasis(currentFieldContent)
        for p in partitionList:
            basisIndices = []
            for i in range(len(basis)):
                basisPartition = list(np.sum(np.array(basis.labels[i]), axis=1))
                basisPartition.sort(reverse=True)
                if p == basisPartition:
                    basisIndices.append(i)
            indices = []
            for i in range(len(labels)):
                if p == labels[i][1]:
                    indices.append(i)
            multiplicity = len(basisIndices) - len(indices)
            testIndex = 0
            for i in range(multiplicity):
                cont = True
                while cont:
                    op = basis[basisIndices[testIndex]]
                    for j in indices:
                        op -= calculator.calculate(operators[j], op, 'planar') * operators[j]
                    op.canonicalizeRadicalsInCoefficients()
                    cont =  len(op.coefficients) == 0
                    testIndex += 1
                op /= sqrt(calculator.calculate(op, op, 'planar'))
                op.canonicalizeRadicalsInCoefficients()
                operators.append(op)
                labels.append((currentFieldContent,p,i+1))
                indices.append(len(operators)-1)

    return Basis(fieldContent, operators, labels)

# orthogonalises the U(2)-covariant basis of multi-symmetric functions using the G-weighted inner product
def orthogonalisedBasis(fieldContent):
    basis = GL2multiSymmetricBasis(fieldContent)
    totalfieldContent = sum(fieldContent)
    calculator = correlatorCalculator(fieldContent)
    operators = []
    labels = []
    for lambda1 in range(fieldContent[1]+1):
        Lambda = [totalfieldContent-lambda1, lambda1]
        operatorsByLambda = []
        labelsByLambda = []
        indicesByPartition = {}
        for i in range(len(basis)):
            if basis.labels[i][0] == Lambda:
                if basis.labels[i][1] in indicesByPartition:
                    indicesByPartition[basis.labels[i][1]].append(i)
                else: 
                    indicesByPartition.update({basis.labels[i][1]:[i]})
        partitions = indicesByPartition.keys()
        partitions.sort(key=transposePartition,reverse=True)
        for p in partitions:
            for mult in range(len(indicesByPartition[p])):
                index = indicesByPartition[p][mult]
                op = basis[index]
                for i in range(len(operatorsByLambda)):
                    op1 = operatorsByLambda[i]
                    op -= calculator.calculate(op,op1,'G')/calculator.calculate(op1,op1,'G') * op1
                op.expandCoefficients()
                op.canonicalizeRadicalsInCoefficients()
                op.factorCoefficients()
                op /= sqrt(calculator.calculate(op,op,'planar'))
                op.expandCoefficients()
                op.canonicalizeRadicalsInCoefficients()
                op.factorCoefficients()
                operatorsByLambda.append(op)
                labelsByLambda.append((Lambda,p,mult+1))
        operators += operatorsByLambda
        labels += labelsByLambda

    return Basis(fieldContent, operators, labels)

# transposes a partition
def transposePartition(p):
    q = [sum([1 if i > j else 0 for i in p]) for j in range(sum(p))]
    try: 
        q = q[:q.index(0)]
    except ValueError:
        q = q
    return q




In [5]:
# Unit Tests (these are an early attempt to follow good coding practice. This didn't last long... sorry)

# Trace class

# Testing initialisation
trace1 = Trace([2,2], PermutationGroupElement([2,3,4,1]))
assert(str(trace1) == '(TrXXYY)')
trace2 = Trace([2,2], [(1,3),(2,4)])
assert(str(trace2) == '(TrXY)(TrXY)')
trace3 = Trace([10,6], [11,1,12,5,13,7,6,8,14,15,2,4,3,10,9,16])
assert(str(trace3) == '(TrX)(TrXX)(TrXXY)(TrXXYXY)(TrXYXY)(TrY)')
# Testing trace multiplication
trace4 = Trace([2,2], [2,3,4,1])
assert(str(trace4) == '(TrXXYY)')
multTrace = trace3 * trace4
assert(str(multTrace) == '(TrX)(TrXX)(TrXXY)(TrXXYXY)(TrXXYY)(TrXYXY)(TrY)')
# Testing equality checks and copying
trace5 = Trace([12,8], [1,3,2,5,13,7,14,15,10,16,18,19,4,8,6,17,9,12,11,20])
assert(trace5 == multTrace)
trace6 = trace1.copy()
assert(trace6 == trace1)
assert(not trace6 is trace1)
# Testing scalar multiplication and division
assert(isinstance(3 * trace1, Operator))
assert(str(3 * trace1) == '3(TrXXYY)')
assert(str(trace1 / 3) == '1/3(TrXXYY)')
# Testing initialisation using fieldPositions
trace7 = Trace([2,2], PermutationGroupElement([2,3,4,1]), fieldPositions=[1,2,1,2])
assert(str(trace7) == '(TrXYXY)')


# Operator class

# Testing initialisation
trace8 = Trace([2,2],[(1,2,3)])
trace9 = Trace([2,2],[(1,3,2,4)])
trace10 = Trace([2,2],[3,4,1,2])
operator1 = Operator([2,2], [5,1,-1,-3,(N+5)/(N-2)], traces=[trace1,trace2,trace8,trace9,trace10])
assert(str(operator1) == '5(TrXXYY) + (TrXY)(TrXY) - (TrXXY)(TrY) - 3(TrXYXY) + ((N + 5)/(N - 2))(TrXY)(TrXY)')
operator2 = Operator([2,2], [3,-1], permutations=[[2,3,4,1],[3,4,2,1]])
assert(str(operator2) == '3(TrXXYY) - (TrXYXY)')
operator3 = Operator([2,2], [-2, 4], permutations=[[2,3,4,1],[3,4,2,1]], fieldPositions=[1,2,1,2])
assert(str(operator3) == '-2(TrXYXY) + 4(TrXXYY)')
# Testing multiplication and division by scalars
assert(str(3 * operator2) == '9(TrXXYY) - 3(TrXYXY)')
assert(str(operator2 / 5) == '3/5(TrXXYY) - 1/5(TrXYXY)')
#Testing multiplication by traces
assert(str(trace1 * operator2) == '3(TrXXYY)(TrXXYY) - (TrXXYY)(TrXYXY)')
assert((trace1 * operator2).fieldContent == [4,4])
#Testing multiplication by other operators
assert(str(operator2 * operator3) == '-10(TrXXYY)(TrXYXY) + 12(TrXXYY)(TrXXYY) + 2(TrXYXY)(TrXYXY)')
assert((operator2 * operator3).fieldContent == [4,4])
# Testing addition and subtraction of other operators
assert(str(operator2 + operator3) == '7(TrXXYY) - 3(TrXYXY)')
assert(str(operator2 + 2 * trace2) == '3(TrXXYY) - (TrXYXY) + 2(TrXY)(TrXY)')
assert(str(operator2 - operator3) == '-(TrXXYY) + (TrXYXY)')

# Basis class



In [3]:
calculator22 = correlatorCalculator([2,2])
calculator32 = correlatorCalculator([3,2])
calculator42 = correlatorCalculator([4,2])
calculator33 = correlatorCalculator([3,3])

In [69]:
#Code generating the GL(2) basis with field content 2,2
labels = [([4],[4],1), ([4],[3,1],1), ([4],[2,2],1), ([4],[2,1,1],1), ([4],[1,1,1,1],1),
         ([3,1],[3,1],1), ([3,1],[2,1,1],1), ([2,2],[3,1],1), ([2,2],[2,2],1), ([2,2],[2,1,1],1)]

tracelist = [ Trace([2,2], [1,2,3,4]),
            Trace([2,2], [(1,2)]), Trace([2,2], [(1,3)]), Trace([2,2], [(3,4)]),
            Trace([2,2], [(1,2,3)]), Trace([2,2], [(1,3,4)]),
            Trace([2,2], [(1,2),(3,4)]), Trace([2,2],[(1,3),(2,4)]), 
            Trace([2,2], [(1,2,3,4)]), Trace([2,2],[(1,3,2,4)])]

coefficients = [[1,1,4,1,4,4,1,2,4,2], [3,1,4,1,0,0,-1,-2,-4,-2], [1,0,0,0,-2,-2,1,2,0,0], [3,-1,-4,-1,0,0,-1,-2,4,2],
               [1,-1,-4,-1,4,4,1,2,-4,-2], [0,1,0,-1,2,-2,0,0,0,0], [0,-1,0,1,2,-2,0,0,0,0], [0,1,-2,1,0,0,2,-2,2,-2],
               [0,-1,2,-1,0,0,0,0,2,-2], [0,-1,2,-1,0,0,2,-2,-2,2]]

norms = [sqrt(3)*sqrt(2)/24, sqrt(3)*sqrt(2)/24, sqrt(3)*sqrt(2)/12, sqrt(3)*sqrt(2)/24, sqrt(3)*sqrt(2)/24,
        1/4, 1/4, sqrt(3)/12, sqrt(3)*sqrt(2)/12, sqrt(3)/12]

for i in range(10):
    for j in range(10):
        coefficients[i][j] *= norms[i]

operators = [Operator([2,2],coefficients=cfs, traces=tracelist) for cfs in coefficients]

gl2basis_fc22 = Basis([2,2],operators,labels=labels)

In [83]:
msbasis22 = multiSymmetricBasis([2,2])



In [102]:
# Orthogonal basis for the Lambda = [2,2] sector in which the stringy exclusion principle is manifest.
# First gives the symmetrised trace basis, normalised so that the planar inner product is 1.
# Then gives the BPS basis, again normalised so that the planar inner product is 1
comm = Operator([2,2], coefficients=[1,-1], permutations=[[(1,2,3,4)],[1,3,2,4]])

st22_1 = gl2basis_fc22.linearCombination([0,0,0,0,0,0,0,sqrt(3)/6, sqrt(2)*sqrt(3)/6, sqrt(3)/2])
st22_1.canonicalizeRadicalsInCoefficients()

st22_2 = gl2basis_fc22.linearCombination([0,0,0,0,0,0,0,-(N+2)*(2*N-1), sqrt(2)*N*(N+1), -(N-2)])
st22_2 /= sqrt(2)*sqrt(3*N^4 + 8*N^3 + 2*N^2 - 8*N + 4)
st22_2.canonicalizeRadicalsInCoefficients()
st22_2.factorCoefficients()

bps22_1 = gl2basis_fc22.linearCombination([0,0,0,0,0,0,0,N*(N-2), sqrt(2)*(N-2)*(N+2), 3*N*(N+2)])
bps22_1 /= 2*sqrt(3*N^4 + 8*N^3 + 6*N^2 + 8)
bps22_1.canonicalizeRadicalsInCoefficients()
bps22_1.factorCoefficients()

bps22_2 = gl2basis_fc22.linearCombination([0,0,0,0,0,0,0,-(2*N-1), sqrt(2)*(N+1), -1])
bps22_2 /= sqrt(2)*sqrt(3*N^2 + 2)
bps22_2.canonicalizeRadicalsInCoefficients()
bps22_2.factorCoefficients()

labels = [([4],[4],1), ([4],[3,1],1), ([4],[2,2],1), ([4],[2,1,1],1), ([4],[1,1,1,1],1), ([3,1],[3,1],1), 
          ([3,1],[2,1,1],1), ([2,2],[2,1,1],1), ([2,2],[2,2],1)]

operators = gl2basis_fc22.operators[:7] + [bps22_1,bps22_2]

bpsbasis_fc22 = Basis([2,2], operators, labels)



In [4]:
#Code generating the GL(2) basis with field content 3,2
labels = [([5], [5], 1), ([5], [4, 1], 1), ([5], [3, 2], 1), ([5], [3, 1, 1], 1), ([5], [2, 2, 1], 1), 
          ([5], [2, 1, 1, 1], 1), ([5], [1, 1, 1, 1, 1], 1), ([4, 1], [4, 1], 1), ([4, 1], [3, 2], 1), 
          ([4, 1], [3, 1, 1], 1), ([4, 1], [2, 2, 1], 1), ([4, 1], [2, 1, 1, 1], 1), ([3, 2], [4, 1], 1), 
          ([3, 2], [3, 2], 1), ([3, 2], [3, 1, 1], 1), ([3, 2], [3, 1, 1], 2), ([3, 2], [2, 2, 1], 1), 
          ([3, 2], [2, 1, 1, 1], 1)]

tracelist = [ Trace([3,2], [1,2,3,4,5]), 
            Trace([3,2],[(1,2)]), Trace([3,2],[(1,4)]), Trace([3,2],[(4,5)]),
            Trace([3,2],[(1,2),(3,4)]), Trace([3,2],[(1,2),(4,5)]), Trace([3,2],[(1,4),(2,5)]),
            Trace([3,2],[(1,2,3)]), Trace([3,2],[(1,2,4)]), Trace([3,2],[(1,4,5)]),
            Trace([3,2],[(1,2,3),(4,5)]), Trace([3,2],[(1,2,4),(3,5)]), Trace([3,2],[(1,4,5),(2,3)]), 
            Trace([3,2],[(1,2,3,4)]), Trace([3,2],[(1,2,4,5)]), Trace([3,2],[(1,4,2,5)]),
            Trace([3,2],[(1,2,3,4,5)]), Trace([3,2],[(1,2,4,3,5)])]

coefficients = [[1/120*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2)], [1/30*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 0, 0, 0, 1/60*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, 0, 0, -1/10*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2)], [1/24*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, 0], [1/20*sqrt(5)*sqrt(2), 0, 0, 0, -1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/10*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2)], [1/24*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 0, 0], [1/30*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 0, 0, 0, 1/60*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 0, 0, 0, -1/10*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2)], [1/120*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2)], [0, -1/15*sqrt(5), 1/30*sqrt(5), 1/30*sqrt(5), -1/10*sqrt(5), 1/30*sqrt(5), 1/15*sqrt(5), -1/15*sqrt(5), -1/15*sqrt(5), 2/15*sqrt(5), 0, 0, 0, -1/5*sqrt(5), 2/15*sqrt(5), 1/15*sqrt(5), 0, 0], [0, -1/30*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 0, 0], [0, 1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, -1/30*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), -1/10*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 0, 0], [0, -1/30*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 0, 0], [0, -1/15*sqrt(5), 1/30*sqrt(5), 1/30*sqrt(5), 1/10*sqrt(5), -1/30*sqrt(5), -1/15*sqrt(5), 1/15*sqrt(5), 1/15*sqrt(5), -2/15*sqrt(5), 0, 0, 0, -1/5*sqrt(5), 2/15*sqrt(5), 1/15*sqrt(5), 0, 0], [0, 1/60*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 0, 1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 0, 1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2)], [0, 1/12*sqrt(2), -1/6*sqrt(2), 1/12*sqrt(2), 0, 1/12*sqrt(2), -1/12*sqrt(2), 1/12*sqrt(2), -1/6*sqrt(2), 1/12*sqrt(2), 0, 0, 0, 0, -1/6*sqrt(2), 1/6*sqrt(2), -1/4*sqrt(2), 1/4*sqrt(2)], [0, -1/60*sqrt(5)*sqrt(3)*sqrt(2), 1/30*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), 0, 0, 0, 0, 0, 0, 1/30*sqrt(5)*sqrt(3)*sqrt(2), -1/15*sqrt(5)*sqrt(3)*sqrt(2), 1/30*sqrt(5)*sqrt(3)*sqrt(2), 0, -1/15*sqrt(5)*sqrt(3)*sqrt(2), 1/15*sqrt(5)*sqrt(3)*sqrt(2), 0, 0], [0, 0, 0, 0, 0, -1/10*sqrt(5), 1/10*sqrt(5), 1/10*sqrt(5), -1/5*sqrt(5), 1/10*sqrt(5), 0, 0, 0, 0, 0, 0, 1/10*sqrt(5), -1/10*sqrt(5)], [0, -1/12*sqrt(2), 1/6*sqrt(2), -1/12*sqrt(2), 0, 1/12*sqrt(2), -1/12*sqrt(2), 1/12*sqrt(2), -1/6*sqrt(2), 1/12*sqrt(2), 0, 0, 0, 0, 1/6*sqrt(2), -1/6*sqrt(2), -1/4*sqrt(2), 1/4*sqrt(2)], [0, -1/60*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 0, 1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, -1/15*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2)]]


operators = [Operator([3,2],coefficients=cfs, traces=tracelist) for cfs in coefficients]

gl2basis_fc32 = Basis([3,2],operators,labels=labels)

In [5]:
# Generating bps basis for field content [3,2]
Gmatrix = diagonal_matrix([1/((N+4)*(N+3)*(N+2)*(N+1)*N), 1/((N+3)*(N+2)*(N+1)*N*(N-1)),
                          1/((N+2)*(N+1)*N^2*(N-1)), 1/((N+2)*(N+1)*N*(N-1)*(N-2)), 1/((N+1)*N^2*(N-1)*(N-2)),
                          1/((N+1)*N*(N-1)*(N-2)*(N-3)), 1/((N+1)*N*(N-1)*(N-2)*(N-3)*(N-4)),
                          1/((N+3)*(N+2)*(N+1)*N*(N-1)), 1/((N+2)*(N+1)*N^2*(N-1)), 1/((N+2)*(N+1)*N*(N-1)*(N-2)),
                          1/((N+1)*N^2*(N-1)*(N-2)), 1/((N+1)*N*(N-1)*(N-2)*(N-3)),
                          1/((N+3)*(N+2)*(N+1)*N*(N-1)), 1/((N+2)*(N+1)*N^2*(N-1)), 1/((N+2)*(N+1)*N*(N-1)*(N-2)),
                          1/((N+2)*(N+1)*N*(N-1)*(N-2)), 1/((N+1)*N^2*(N-1)*(N-2)), 1/((N+1)*N*(N-1)*(N-2)*(N-3))])

Fmatrix = diagonal_matrix([((N+4)*(N+3)*(N+2)*(N+1)*N), ((N+3)*(N+2)*(N+1)*N*(N-1)),
                          ((N+2)*(N+1)*N^2*(N-1)), ((N+2)*(N+1)*N*(N-1)*(N-2)), ((N+1)*N^2*(N-1)*(N-2)),
                          ((N+1)*N*(N-1)*(N-2)*(N-3)), ((N+1)*N*(N-1)*(N-2)*(N-3)*(N-4)),
                          ((N+3)*(N+2)*(N+1)*N*(N-1)), ((N+2)*(N+1)*N^2*(N-1)), ((N+2)*(N+1)*N*(N-1)*(N-2)),
                          ((N+1)*N^2*(N-1)*(N-2)), ((N+1)*N*(N-1)*(N-2)*(N-3)),
                          ((N+3)*(N+2)*(N+1)*N*(N-1)), ((N+2)*(N+1)*N^2*(N-1)), ((N+2)*(N+1)*N*(N-1)*(N-2)),
                          ((N+2)*(N+1)*N*(N-1)*(N-2)), ((N+1)*N^2*(N-1)*(N-2)), ((N+1)*N*(N-1)*(N-2)*(N-3))])

comm41 = Operator([3,2], [1,-1], permutations=[[(1,2,4,5)],[(1,4,2,5)]])
comm5 = Operator([3,2], [1,-1], permutations=[[(1,2,3,4,5)],[(1,2,4,3,5)]])
normalisedcommbasis = Basis([3,2], [comm41,comm5])
for i in range(len(normalisedcommbasis)):
    normalisedcommbasis.operators[i] /= sqrt(calculator32.calculate(normalisedcommbasis[i],normalisedcommbasis[i],'planar'))
    normalisedcommbasis[i].canonicalizeRadicalsInCoefficients()
commvectors = [gl2basis_fc32.expand(comm) for comm in normalisedcommbasis]

symtracevectors = []
Gnorms = []
indexorder = [17,16,14,12]
for index in indexorder:
    symtracevector = vector([0 for i in range(18)])
    symtracevector[index] = 1
    for commvec in commvectors:
        symtracevector -= (symtracevector * commvec) * commvec
    for i in range(len(symtracevectors)):
        symtracevector -= (symtracevector * Gmatrix * symtracevectors[i])/Gnorms[i] * symtracevectors[i]
        for j in range(len(symtracevector)):
            symtracevector[j] = symtracevector[j].canonicalize_radical().factor()
    planarNorm = (symtracevector * symtracevector).canonicalize_radical().factor()
    if planarNorm != 0:
        symtracevector *= 1/sqrt(planarNorm)
        for i in range(len(symtracevector)):
            symtracevector[i] = symtracevector[i].canonicalize_radical().factor()
        Gnorms.append((symtracevector * Gmatrix * symtracevector).canonicalize_radical().factor())
        symtracevectors.append(symtracevector)

labels = gl2basis_fc32.labels[:12] + [([3,2],[2,1,1,1],1),([3,2],[2,2,1],1),([3,2],[3,1,1],1),([3,2],[3,2],1)]
operators = gl2basis_fc32.operators[:12]
for vec in symtracevectors:
    operators.append(gl2basis_fc32.linearCombination(list(vec)))
# Changing signs of the operators so they match the sign conventions from the multi-symmetric function orthogonalisation method
signs = [1,1,1,1,1,1,1,-1,-1,1,-1,-1,-1,-1,-1,1]
for i in range(len(operators)):
    operators[i] *= signs[i]
symtracebasis_fc32 = Basis([3,2], operators=operators, labels=labels)


bpsvectors = [vector([(vec[i]*Gmatrix[i,i]).factor() for i in range(18)]) for vec in symtracevectors]
for i in range(len(bpsvectors)):
    planarNorm = (bpsvectors[i] * bpsvectors[i]).canonicalize_radical().factor()
    bpsvectors[i] *= 1/sqrt(planarNorm)
    for j in range(len(bpsvectors[i])):
        bpsvectors[i][j] = bpsvectors[i][j].canonicalize_radical().factor()
Fnorms = [(vec * Fmatrix * vec).canonicalize_radical().factor() for vec in bpsvectors]

labels = gl2basis_fc32.labels[:12] + [([3,2],[2,1,1,1],1),([3,2],[2,2,1],1),([3,2],[3,1,1],1),([3,2],[3,2],1)]
operators = gl2basis_fc32.operators[:12]
for vec in bpsvectors:
    operators.append(gl2basis_fc32.linearCombination(list(vec)))
# Changing signs of the operators so they match the sign conventions from the multi-symmetric function orthogonalisation method
signs = [1,1,1,1,1,1,1,-1,-1,1,-1,-1,-1,-1,-1,1]
for i in range(len(operators)):
    operators[i] *= signs[i]

bpsbasis_fc32 = Basis([3,2],operators=operators, labels=labels)

# Changing signs of the vectors so they match the sign conventions from the multi-symmetric function orthogonalisation method
signs = [-1,-1,-1,1]
for i in range(4):
    symtracevectors[i] *= signs[i]
    bpsvectors[i] *= signs[i]

In [6]:
msbasis_fc32 = orthogonalisedBasis([3,2])

In [145]:
#Code generating the GL(2) basis with field content 4,2 and Lambda = [6,0]
labels = [([6], [6], 1), ([6], [5, 1], 1), ([6], [4, 2], 1), ([6], [4, 1, 1], 1), ([6], [3, 3], 1), 
          ([6], [3, 2, 1], 1), ([6], [3, 1, 1, 1], 1), ([6], [2, 2, 2], 1), ([6], [2, 2, 1, 1], 1), 
          ([6], [2, 1, 1, 1, 1], 1), ([6], [1, 1, 1, 1, 1, 1], 1)]

tracelist = [ Trace([4,2], [1,2,3,4,5,6]), Trace([4,2],[(1,2)]), Trace([4,2],[(1,5)]), Trace([4,2],[(5,6)]),
            Trace([4,2],[(1,2),(3,4)]), Trace([4,2],[(1,2),(3,5)]), Trace([4,2],[(1,5),(2,6)]), 
            Trace([4,2],[(1,2),(5,6)]), Trace([4,2],[(1,2),(3,4),(5,6)]), Trace([4,2],[(1,2),(3,5),(4,6)]),
            Trace([4,2],[(1,2,3)]), Trace([4,2],[(1,2,5)]), Trace([4,2],[(1,5,6)]), 
            Trace([4,2],[(1,2,3),(4,5)]), Trace([4,2],[(1,2,3),(5,6)]), Trace([4,2],[(1,2,5),(3,4)]),
            Trace([4,2],[(1,2,5),(3,6)]), Trace([4,2],[(1,5,6),(2,3)]), Trace([4,2],[(1,2,3),(4,5,6)]),
            Trace([4,2],[(1,2,5),(3,4,6)]), Trace([4,2],[(1,2,3,4)]), Trace([4,2],[(1,2,3,5)]),
            Trace([4,2],[(1,2,5,6)]), Trace([4,2],[(1,5,2,6)]), Trace([4,2],[(1,2,3,4),(5,6)]),
            Trace([4,2],[(1,2,3,5),(4,6)]), Trace([4,2],[(1,2,5,6),(3,4)]), Trace([4,2],[(1,5,2,6),(3,4)]),
            Trace([4,2],[(1,2,3,4,5)]), Trace([4,2],[(1,2,3,5,6)]), Trace([4,2],[(1,2,5,3,6)]),
            Trace([4,2],[(1,2,3,4,5,6)]), Trace([4,2],[(1,2,3,5,4,6)]), Trace([4,2],[(1,2,5,3,4,6)])]

coefficients = [[1/720*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/720*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3)], [1/144*sqrt(5)*sqrt(3), 1/40*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), -1/240*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, -1/45*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 0, 0, 0, -1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3)], [1/80*sqrt(5)*sqrt(3), 1/40*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/80*sqrt(5)*sqrt(3), 1/20*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), 0, 0, 0], [1/72*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/360*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), -1/45*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3)], [1/144*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/720*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), -1/80*sqrt(5)*sqrt(3), -1/20*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 2/45*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0], [1/45*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/45*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/45*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, -2/45*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 0, 0, 0], [1/72*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), -1/45*sqrt(5)*sqrt(3), -1/360*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3)], [1/144*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/720*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/80*sqrt(5)*sqrt(3), 1/20*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/45*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 2/45*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0], [1/80*sqrt(5)*sqrt(3), -1/40*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/240*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), -1/80*sqrt(5)*sqrt(3), -1/20*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/120*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), 0, 0, 0], [1/144*sqrt(5)*sqrt(3), -1/40*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/240*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, -1/45*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 0, 0, 0, 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3)], [1/720*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/720*sqrt(5)*sqrt(3), 1/240*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), -1/240*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), -1/45*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3)]]

operators = [Operator([4,2],coefficients=cfs, traces=tracelist) for cfs in coefficients]

gl2basis_fc42_lambda6 = Basis([4,2],operators,labels=labels)

In [9]:
#Code generating the GL(2) basis with field content 4,2 and Lambda = [5,1]
labels = [([5, 1], [5, 1], 1), ([5, 1], [4, 2], 1), ([5, 1], [4, 1, 1], 1), ([5, 1], [3, 2, 1], 1), 
          ([5, 1], [3, 2, 1], 2), ([5, 1], [3, 1, 1, 1], 1), ([5, 1], [2, 2, 1, 1], 1), ([5, 1], [2, 1, 1, 1, 1], 1)]

tracelist = [ Trace([4,2], [1,2,3,4,5,6]), Trace([4,2],[(1,2)]), Trace([4,2],[(1,5)]), Trace([4,2],[(5,6)]),
            Trace([4,2],[(1,2),(3,4)]), Trace([4,2],[(1,2),(3,5)]), Trace([4,2],[(1,5),(2,6)]), 
            Trace([4,2],[(1,2),(5,6)]), Trace([4,2],[(1,2),(3,4),(5,6)]), Trace([4,2],[(1,2),(3,5),(4,6)]),
            Trace([4,2],[(1,2,3)]), Trace([4,2],[(1,2,5)]), Trace([4,2],[(1,5,6)]), 
            Trace([4,2],[(1,2,3),(4,5)]), Trace([4,2],[(1,2,3),(5,6)]), Trace([4,2],[(1,2,5),(3,4)]),
            Trace([4,2],[(1,2,5),(3,6)]), Trace([4,2],[(1,5,6),(2,3)]), Trace([4,2],[(1,2,3),(4,5,6)]),
            Trace([4,2],[(1,2,5),(3,4,6)]), Trace([4,2],[(1,2,3,4)]), Trace([4,2],[(1,2,3,5)]),
            Trace([4,2],[(1,2,5,6)]), Trace([4,2],[(1,5,2,6)]), Trace([4,2],[(1,2,3,4),(5,6)]),
            Trace([4,2],[(1,2,3,5),(4,6)]), Trace([4,2],[(1,2,5,6),(3,4)]), Trace([4,2],[(1,5,2,6),(3,4)]),
            Trace([4,2],[(1,2,3,4,5)]), Trace([4,2],[(1,2,3,5,6)]), Trace([4,2],[(1,2,5,3,6)]),
            Trace([4,2],[(1,2,3,4,5,6)]), Trace([4,2],[(1,2,3,5,4,6)]), Trace([4,2],[(1,2,5,3,4,6)])]

coefficients = [[0, -1/48*sqrt(3)*sqrt(2), 1/72*sqrt(3)*sqrt(2), 1/144*sqrt(3)*sqrt(2), -1/48*sqrt(3)*sqrt(2), -1/24*sqrt(3)*sqrt(2), 1/24*sqrt(3)*sqrt(2), 1/48*sqrt(3)*sqrt(2), 0, 0, -1/24*sqrt(3)*sqrt(2), 0, 1/24*sqrt(3)*sqrt(2), -1/18*sqrt(3)*sqrt(2), 1/72*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 1/24*sqrt(3)*sqrt(2), 0, 0, -1/24*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 1/24*sqrt(3)*sqrt(2), 0, 0, 0, 0, -1/6*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 0, 0, 0], [0, -1/80*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), 1/240*sqrt(5)*sqrt(3)*sqrt(2), 1/240*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/240*sqrt(5)*sqrt(3)*sqrt(2), 0, 0, -1/40*sqrt(5)*sqrt(3)*sqrt(2), 0, 1/40*sqrt(5)*sqrt(3)*sqrt(2), 0, -1/40*sqrt(5)*sqrt(3)*sqrt(2), 1/20*sqrt(5)*sqrt(3)*sqrt(2), -1/20*sqrt(5)*sqrt(3)*sqrt(2), 1/40*sqrt(5)*sqrt(3)*sqrt(2), 0, 0, -1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), 1/60*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/30*sqrt(5)*sqrt(3)*sqrt(2), 1/30*sqrt(5)*sqrt(3)*sqrt(2), 1/60*sqrt(5)*sqrt(3)*sqrt(2), 1/30*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), 0, 0, 0], [0, -1/8, 1/12, 1/24, -1/24, -1/12, 1/12, 1/24, 0, 0, -1/12, 0, 1/12, 1/6, 1/12, 0, 0, -1/4, 0, 0, 0, 0, 0, 0, 1/12, 1/6, -1/6, -1/12, 1/3, -1/6, -1/6, 0, 0, 0], [0, 0, 0, 0, -1/30*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), 0, 0, 1/30*sqrt(5)*sqrt(2), 0, -1/30*sqrt(5)*sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/30*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 0, 0, 0], [0, -1/30*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/18*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 1/30*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), 0, 0, 1/30*sqrt(5)*sqrt(3), 1/15*sqrt(5)*sqrt(3), -1/15*sqrt(5)*sqrt(3), -1/30*sqrt(5)*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1/8, -1/12, -1/24, -1/24, -1/12, 1/12, 1/24, 0, 0, -1/12, 0, 1/12, -1/6, -1/12, 0, 0, 1/4, 0, 0, 0, 0, 0, 0, 1/12, 1/6, -1/6, -1/12, 1/3, -1/6, -1/6, 0, 0, 0], [0, 1/80*sqrt(5)*sqrt(3)*sqrt(2), -1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/240*sqrt(5)*sqrt(3)*sqrt(2), 1/240*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/240*sqrt(5)*sqrt(3)*sqrt(2), 0, 0, -1/40*sqrt(5)*sqrt(3)*sqrt(2), 0, 1/40*sqrt(5)*sqrt(3)*sqrt(2), 0, 1/40*sqrt(5)*sqrt(3)*sqrt(2), -1/20*sqrt(5)*sqrt(3)*sqrt(2), 1/20*sqrt(5)*sqrt(3)*sqrt(2), -1/40*sqrt(5)*sqrt(3)*sqrt(2), 0, 0, 1/120*sqrt(5)*sqrt(3)*sqrt(2), 1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/30*sqrt(5)*sqrt(3)*sqrt(2), 1/30*sqrt(5)*sqrt(3)*sqrt(2), 1/60*sqrt(5)*sqrt(3)*sqrt(2), 1/30*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), 0, 0, 0], [0, 1/48*sqrt(3)*sqrt(2), -1/72*sqrt(3)*sqrt(2), -1/144*sqrt(3)*sqrt(2), -1/48*sqrt(3)*sqrt(2), -1/24*sqrt(3)*sqrt(2), 1/24*sqrt(3)*sqrt(2), 1/48*sqrt(3)*sqrt(2), 0, 0, -1/24*sqrt(3)*sqrt(2), 0, 1/24*sqrt(3)*sqrt(2), 1/18*sqrt(3)*sqrt(2), -1/72*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), -1/24*sqrt(3)*sqrt(2), 0, 0, 1/24*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), -1/24*sqrt(3)*sqrt(2), 0, 0, 0, 0, -1/6*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 0, 0, 0]]

operators = [Operator([4,2],coefficients=cfs, traces=tracelist) for cfs in coefficients]

gl2basis_fc42_lambda51 = Basis([4,2],operators,labels=labels)

In [4]:
#Code generating the GL(2) basis with field content 4,2 and Lambda = [4,2]
labels = [([4, 2], [5, 1], 1), ([4, 2], [4, 2], 1), ([4, 2], [4, 2], 2), ([4, 2], [4, 1, 1], 1), 
          ([4, 2], [4, 1, 1], 2), ([4, 2], [3, 3], 1), ([4, 2], [3, 2, 1], 1), ([4, 2], [3, 2, 1], 2), 
          ([4, 2], [3, 2, 1], 3), ([4, 2], [3, 1, 1, 1], 1), ([4, 2], [3, 1, 1, 1], 2), ([4, 2], [2, 2, 2], 1), 
          ([4, 2], [2, 2, 1, 1], 1), ([4, 2], [2, 2, 1, 1], 2), ([4, 2], [2, 1, 1, 1, 1], 1)]

tracelist = [ Trace([4,2], [1,2,3,4,5,6]), Trace([4,2],[(1,2)]), Trace([4,2],[(1,5)]), Trace([4,2],[(5,6)]),
            Trace([4,2],[(1,2),(3,4)]), Trace([4,2],[(1,2),(3,5)]), Trace([4,2],[(1,5),(2,6)]), 
            Trace([4,2],[(1,2),(5,6)]), Trace([4,2],[(1,2),(3,4),(5,6)]), Trace([4,2],[(1,2),(3,5),(4,6)]),
            Trace([4,2],[(1,2,3)]), Trace([4,2],[(1,2,5)]), Trace([4,2],[(1,5,6)]), 
            Trace([4,2],[(1,2,3),(4,5)]), Trace([4,2],[(1,2,3),(5,6)]), Trace([4,2],[(1,2,5),(3,4)]),
            Trace([4,2],[(1,2,5),(3,6)]), Trace([4,2],[(1,5,6),(2,3)]), Trace([4,2],[(1,2,3),(4,5,6)]),
            Trace([4,2],[(1,2,5),(3,4,6)]), Trace([4,2],[(1,2,3,4)]), Trace([4,2],[(1,2,3,5)]),
            Trace([4,2],[(1,2,5,6)]), Trace([4,2],[(1,5,2,6)]), Trace([4,2],[(1,2,3,4),(5,6)]),
            Trace([4,2],[(1,2,3,5),(4,6)]), Trace([4,2],[(1,2,5,6),(3,4)]), Trace([4,2],[(1,5,2,6),(3,4)]),
            Trace([4,2],[(1,2,3,4,5)]), Trace([4,2],[(1,2,3,5,6)]), Trace([4,2],[(1,2,5,3,6)]),
            Trace([4,2],[(1,2,3,4,5,6)]), Trace([4,2],[(1,2,3,5,4,6)]), Trace([4,2],[(1,2,5,3,4,6)])]

coefficients = [[0, -1/240*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), -1/240*sqrt(5)*sqrt(2), -1/240*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -7/240*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 0, -1/24*sqrt(5)*sqrt(2), 0, 1/12*sqrt(5)*sqrt(2), -1/24*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 0, -1/12*sqrt(5)*sqrt(2), 1/12*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2)], [0, 0, 0, 0, -1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 0, -1/40*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 0, 0, 0, 1/20*sqrt(5)*sqrt(2), 0, -1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 0, -1/40*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 0, 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2)], [0, -1/80*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -1/80*sqrt(5)*sqrt(2), -1/80*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -3/80*sqrt(5)*sqrt(2), 0, 0, -1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 0, -1/40*sqrt(5)*sqrt(2), 0, 1/20*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 0, 0, -1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 0, -1/40*sqrt(5)*sqrt(2), 0, 0, 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), 0], [0, 0, 0, 0, 1/36*sqrt(3), -1/18*sqrt(3), 1/12*sqrt(3), -1/18*sqrt(3), -1/36*sqrt(3), 1/36*sqrt(3), 1/18*sqrt(3), -1/9*sqrt(3), 1/18*sqrt(3), 1/18*sqrt(3), -1/18*sqrt(3), -1/18*sqrt(3), 1/18*sqrt(3), 0, 1/18*sqrt(3), -1/18*sqrt(3), 1/12*sqrt(3), -1/6*sqrt(3), 1/18*sqrt(3), 1/36*sqrt(3), -1/36*sqrt(3), 1/18*sqrt(3), 0, -1/36*sqrt(3), 0, 1/18*sqrt(3), -1/18*sqrt(3), 1/9*sqrt(3), -1/18*sqrt(3), -1/18*sqrt(3)], [0, 1/120*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 1/120*sqrt(5)*sqrt(3), 1/360*sqrt(5)*sqrt(3), -1/180*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 7/360*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/180*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), 1/180*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/180*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), -2/45*sqrt(5)*sqrt(3), 2/45*sqrt(5)*sqrt(3), 0, 0, 1/18*sqrt(5)*sqrt(3), -1/18*sqrt(5)*sqrt(3), -1/36*sqrt(5)*sqrt(3), 1/18*sqrt(5)*sqrt(3), 0, -1/36*sqrt(5)*sqrt(3), 0, 1/18*sqrt(5)*sqrt(3), -1/18*sqrt(5)*sqrt(3), -1/45*sqrt(5)*sqrt(3), -1/45*sqrt(5)*sqrt(3), 2/45*sqrt(5)*sqrt(3)], [0, -1/120*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 7/120*sqrt(5)*sqrt(2), 0, 1/12*sqrt(5)*sqrt(2), -1/12*sqrt(5)*sqrt(2), 0, 1/12*sqrt(5)*sqrt(2), -1/12*sqrt(5)*sqrt(2)], [0, 0, 0, 0, 1/30*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 0, 1/30*sqrt(5)*sqrt(2), 0, 0, -1/30*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 0, 0, 0, 0, 0, 1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 0, 0, 0, 0, -1/30*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), 0, -1/30*sqrt(5)*sqrt(2), 0, -1/30*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1/15*sqrt(5), -1/15*sqrt(5), 0, 0, 0, 1/15*sqrt(5), -1/15*sqrt(5), -1/15*sqrt(5), 1/15*sqrt(5), 0, 0, 0, 0, 0, 1/15*sqrt(5), -1/15*sqrt(5), 0, 0, 0, 0, 0, 0, 0, -1/15*sqrt(5), 2/15*sqrt(5), -1/15*sqrt(5)], [0, 1/30*sqrt(5), -1/15*sqrt(5), 1/30*sqrt(5), 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/30*sqrt(5), -1/30*sqrt(5), 1/30*sqrt(5), 1/10*sqrt(5), -1/15*sqrt(5), 0, 0, -1/30*sqrt(5), 1/15*sqrt(5), -1/15*sqrt(5), 1/30*sqrt(5), 0, 0, 0, 0, 0, 0, 0, 2/15*sqrt(5), 0, -2/15*sqrt(5)], [0, 0, 0, 0, 1/36*sqrt(3), -1/18*sqrt(3), 1/12*sqrt(3), -1/18*sqrt(3), 1/36*sqrt(3), -1/36*sqrt(3), 1/18*sqrt(3), -1/9*sqrt(3), 1/18*sqrt(3), -1/18*sqrt(3), 1/18*sqrt(3), 1/18*sqrt(3), -1/18*sqrt(3), 0, 1/18*sqrt(3), -1/18*sqrt(3), -1/12*sqrt(3), 1/6*sqrt(3), -1/18*sqrt(3), -1/36*sqrt(3), -1/36*sqrt(3), 1/18*sqrt(3), 0, -1/36*sqrt(3), 0, 1/18*sqrt(3), -1/18*sqrt(3), -1/9*sqrt(3), 1/18*sqrt(3), 1/18*sqrt(3)], [0, -1/120*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), -1/120*sqrt(5)*sqrt(3), 1/360*sqrt(5)*sqrt(3), -1/180*sqrt(5)*sqrt(3), -1/60*sqrt(5)*sqrt(3), 7/360*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), 1/180*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), 1/180*sqrt(5)*sqrt(3), 1/90*sqrt(5)*sqrt(3), 1/180*sqrt(5)*sqrt(3), -1/90*sqrt(5)*sqrt(3), -1/45*sqrt(5)*sqrt(3), 1/60*sqrt(5)*sqrt(3), -2/45*sqrt(5)*sqrt(3), 2/45*sqrt(5)*sqrt(3), 0, 0, -1/18*sqrt(5)*sqrt(3), 1/18*sqrt(5)*sqrt(3), -1/36*sqrt(5)*sqrt(3), 1/18*sqrt(5)*sqrt(3), 0, -1/36*sqrt(5)*sqrt(3), 0, 1/18*sqrt(5)*sqrt(3), -1/18*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), 1/45*sqrt(5)*sqrt(3), -2/45*sqrt(5)*sqrt(3)], [0, 1/120*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 7/120*sqrt(5)*sqrt(2), 0, 1/12*sqrt(5)*sqrt(2), -1/12*sqrt(5)*sqrt(2), 0, -1/12*sqrt(5)*sqrt(2), 1/12*sqrt(5)*sqrt(2)], [0, 0, 0, 0, -1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 0, 1/40*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 0, 0, 0, -1/20*sqrt(5)*sqrt(2), 0, 1/20*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, 1/40*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 0, 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, -1/20*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2)], [0, 1/80*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 1/80*sqrt(5)*sqrt(2), -1/80*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -3/80*sqrt(5)*sqrt(2), 0, 0, -1/40*sqrt(5)*sqrt(2), 1/20*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), 0, 1/40*sqrt(5)*sqrt(2), 0, -1/20*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), 0, 0, 1/40*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, 1/40*sqrt(5)*sqrt(2), 0, 0, 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 0, 1/20*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), -1/10*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), 0], [0, 1/240*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/240*sqrt(5)*sqrt(2), -1/240*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), 1/40*sqrt(5)*sqrt(2), -7/240*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), -1/120*sqrt(5)*sqrt(2), 0, 1/24*sqrt(5)*sqrt(2), 0, -1/12*sqrt(5)*sqrt(2), 1/24*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), 1/120*sqrt(5)*sqrt(2), -1/60*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), -1/40*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), -1/20*sqrt(5)*sqrt(2), 1/60*sqrt(5)*sqrt(2), 0, -1/12*sqrt(5)*sqrt(2), 1/12*sqrt(5)*sqrt(2), 1/10*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2)]]

operators = [Operator([4,2],coefficients=cfs, traces=tracelist) for cfs in coefficients]

gl2basis_fc42_lambda42 = Basis([4,2],operators,labels=labels)

In [5]:
# Generating bps basis for field content [4,2], lambda = [4,2]
Gmatrix = diagonal_matrix([1/((N+4)*(N+3)*(N+2)*(N+1)*N*(N-1)), 1/((N+3)*(N+2)*(N+1)*N^2*(N-1)),
                          1/((N+3)*(N+2)*(N+1)*N^2*(N-1)), 1/((N+3)*(N+2)*(N+1)*N*(N-1)*(N-2)),
                          1/((N+3)*(N+2)*(N+1)*N*(N-1)*(N-2)), 1/((N+2)*(N+1)^2*N^2*(N-1)),
                          1/((N+2)*(N+1)*N^2*(N-1)*(N-2)), 1/((N+2)*(N+1)*N^2*(N-1)*(N-2)),
                          1/((N+2)*(N+1)*N^2*(N-1)*(N-2)), 1/((N+2)*(N+1)*N*(N-1)*(N-2)*(N-3)),
                          1/((N+2)*(N+1)*N*(N-1)*(N-2)*(N-3)), 1/((N+1)*N^2*(N-1)^2*(N-2)),
                          1/((N+1)*N^2*(N-1)*(N-2)*(N-3)), 1/((N+1)*N^2*(N-1)*(N-2)*(N-3)),
                          1/((N+1)*N*(N-1)*(N-2)*(N-3)*(N-4))])

Fmatrix = diagonal_matrix([((N+4)*(N+3)*(N+2)*(N+1)*N*(N-1)), ((N+3)*(N+2)*(N+1)*N^2*(N-1)),
                          ((N+3)*(N+2)*(N+1)*N^2*(N-1)), ((N+3)*(N+2)*(N+1)*N*(N-1)*(N-2)),
                          ((N+3)*(N+2)*(N+1)*N*(N-1)*(N-2)), ((N+2)*(N+1)^2*N^2*(N-1)),
                          ((N+2)*(N+1)*N^2*(N-1)*(N-2)), ((N+2)*(N+1)*N^2*(N-1)*(N-2)),
                          ((N+2)*(N+1)*N^2*(N-1)*(N-2)), ((N+2)*(N+1)*N*(N-1)*(N-2)*(N-3)),
                          ((N+2)*(N+1)*N*(N-1)*(N-2)*(N-3)), ((N+1)*N^2*(N-1)^2*(N-2)),
                          ((N+1)*N^2*(N-1)*(N-2)*(N-3)), ((N+1)*N^2*(N-1)*(N-2)*(N-3)),
                          ((N+1)*N*(N-1)*(N-2)*(N-3)*(N-4))])

comm411 = Operator([4,2], [1,-1], permutations=[[(1,2,5,6)],[(1,5,2,6)]])
comm42 = Operator([4,2], [1,-1], permutations=[[(1,2,5,6),(3,4)],[(1,5,2,6),(3,4)]])
comm51 = Operator([4,2], [1,-1], permutations=[[(1,2,3,5,6)],[(1,2,5,3,6)]])
comm6_1 = Operator([4,2], [1,-1], permutations=[[(1,2,3,4,5,6)],[(1,2,3,5,4,6)]])
comm6_2 = Operator([4,2], [1,1,-2], permutations=[[(1,2,3,4,5,6)],[(1,2,3,5,4,6)],[(1,2,5,3,4,6)]])
normalisedcommbasis = Basis([4,2], [comm411,comm42,comm51,comm6_1,comm6_2])
for i in range(len(normalisedcommbasis)):
    normalisedcommbasis.operators[i] /= sqrt(calculator42.calculate(normalisedcommbasis[i],normalisedcommbasis[i],'planar'))
    normalisedcommbasis[i].canonicalizeRadicalsInCoefficients()
commvectors = [gl2basis_fc42_lambda42.expand(comm) for comm in normalisedcommbasis]

symtracevectors = []
Gnorms = []
#indexorder = [14,12,13,9,11,6,7,3,5,1]
indexorder = [14,13,12,10,9,11,8,7,6,4,3,5,2,1,0]
for index in indexorder:
    symtracevector = vector([0 for i in range(15)])
    symtracevector[index] = 1
    for commvec in commvectors:
        symtracevector -= (symtracevector * commvec) * commvec
    for i in range(len(symtracevectors)):
        symtracevector -= (symtracevector * Gmatrix * symtracevectors[i])/Gnorms[i] * symtracevectors[i]
        for j in range(len(symtracevector)):
            symtracevector[j] = symtracevector[j].canonicalize_radical().factor()
    planarNorm = (symtracevector * symtracevector).canonicalize_radical().factor()
    if planarNorm != 0:
        symtracevector *= 1/sqrt(planarNorm)
        for i in range(len(symtracevector)):
            symtracevector[i] = symtracevector[i].canonicalize_radical().factor()
        Gnorms.append((symtracevector * Gmatrix * symtracevector).canonicalize_radical().factor())
        symtracevectors.append(symtracevector)
        
#labels = [([4,2],5,1),([4,2],4,1),([4,2],4,2),([4,2],4,3),([4,2],3,1),([4,2],3,2),([4,2],3,3),([4,2],3,4),
#        ([4,2],2,1),([4,2],2,2)]
labels = [([4, 2], [2, 1, 1, 1, 1], 1), ([4, 2], [2, 2, 1, 1], 1), ([4, 2], [2, 2, 1, 1], 2), ([4, 2], [3, 1, 1, 1], 1),
          ([4, 2], [2, 2, 2], 1), ([4, 2], [3, 2, 1], 1), ([4, 2], [3, 2, 1], 2), ([4, 2], [4, 1, 1], 1), ([4, 2], [3, 3], 1),
          ([4, 2], [4, 2], 1)]
operators = []
signchangeindices = [3,8,9]
for i in signchangeindices:
    symtracevectors[i] = -symtracevectors[i]

for vec in symtracevectors:
    operators.append(gl2basis_fc42_lambda42.linearCombination(list(vec)))
symtracevectors_fc42_lambda42 = symtracevectors
symtracebasis_fc42_lambda42 = Basis([4,2], operators=operators, labels=labels)
    

bpsvectors = [vector([(vec[i]*Gmatrix[i,i]).factor() for i in range(15)]) for vec in symtracevectors]
for i in range(len(bpsvectors)):
    planarNorm = (bpsvectors[i] * bpsvectors[i]).canonicalize_radical().factor()
    bpsvectors[i] *= 1/sqrt(planarNorm)
    for j in range(len(bpsvectors[i])):
        bpsvectors[i][j] = bpsvectors[i][j].canonicalize_radical().factor()
Fnorms = [(vec * Fmatrix * vec).canonicalize_radical().factor() for vec in bpsvectors]

labels = [([4, 2], [2, 1, 1, 1, 1], 1), ([4, 2], [2, 2, 1, 1], 1), ([4, 2], [2, 2, 1, 1], 2), ([4, 2], [3, 1, 1, 1], 1),
          ([4, 2], [2, 2, 2], 1), ([4, 2], [3, 2, 1], 1), ([4, 2], [3, 2, 1], 2), ([4, 2], [4, 1, 1], 1), ([4, 2], [3, 3], 1),
          ([4, 2], [4, 2], 1)]
operators = []
for vec in bpsvectors:
    operators.append(gl2basis_fc42_lambda42.linearCombination(list(vec)))
bpsvectors_fc42_lambda42 = bpsvectors
bpsbasis_fc42_lambda42 = Basis([4,2],operators=operators, labels=labels)

In [6]:
msbasis_fc42= orthogonalisedBasis([4,2])

In [12]:
#Code generating the GL(2) basis with field content 3,3 and Lambda = [4,2]
labels = [([4, 2], [5, 1], 1), ([4, 2], [4, 2], 1), ([4, 2], [4, 2], 2), ([4, 2], [4, 1, 1], 1), 
          ([4, 2], [4, 1, 1], 2), ([4, 2], [3, 3], 1), ([4, 2], [3, 2, 1], 1), ([4, 2], [3, 2, 1], 2), 
          ([4, 2], [3, 2, 1], 3), ([4, 2], [3, 1, 1, 1], 1), ([4, 2], [3, 1, 1, 1], 2), ([4, 2], [2, 2, 2], 1), 
          ([4, 2], [2, 2, 1, 1], 1), ([4, 2], [2, 2, 1, 1], 2), ([4, 2], [2, 1, 1, 1, 1], 1)]

tracelist = [ Trace([3,3], [1,2,3,4,5,6]), 
            Trace([3,3],[(1,2)]), Trace([3,3],[(1,4)]), Trace([3,3],[(4,5)]),
            Trace([3,3],[(1,2),(3,4)]), Trace([3,3],[(1,2),(4,5)]), Trace([3,3],[(1,4),(2,5)]), Trace([3,3],[(1,4),(5,6)]),
            Trace([3,3],[(1,2),(3,4),(5,6)]), Trace([3,3],[(1,4),(2,5),(3,6)]),
            Trace([3,3],[(1,2,3)]), Trace([3,3],[(1,2,4)]), Trace([3,3],[(1,4,5)]),  Trace([3,3],[(4,5,6)]),
            Trace([3,3],[(1,2,3),(4,5)]), Trace([3,3],[(1,2,4),(3,5)]), Trace([3,3],[(1,2,4),(5,6)]),
            Trace([3,3],[(1,4,5),(2,3)]), Trace([3,3],[(1,4,5),(2,6)]), Trace([3,3],[(1,2),(4,5,6)]),
            Trace([3,3],[(1,2,3),(4,5,6)]), Trace([3,3],[(1,2,4),(3,5,6)]), 
            Trace([3,3],[(1,2,3,4)]), Trace([3,3],[(1,2,4,5)]), Trace([3,3],[(1,4,2,5)]), Trace([3,3],[(1,4,5,6)]), 
            Trace([3,3],[(1,2,3,4),(5,6)]), Trace([3,3],[(1,2,4,5),(3,6)]), Trace([3,3],[(1,4,2,5),(3,6)]), 
            Trace([3,3],[(1,4,5,6),(2,3)]),
            Trace([3,3],[(1,2,3,4,5)]), Trace([3,3],[(1,2,4,3,5)]), Trace([3,3],[(1,2,4,5,6)]), Trace([3,3],[(1,4,2,5,6)]),
            Trace([3,3],[(1,2,3,4,5,6)]), Trace([3,3],[(1,2,4,3,5,6)]), Trace([3,3],[(1,2,4,5,3,6)]), 
            Trace([3,3],[(1,4,2,5,3,6)])]

coefficients = [[0, -1/120*sqrt(5), 1/60*sqrt(5), -1/120*sqrt(5), -1/120*sqrt(5), -1/20*sqrt(5), 1/15*sqrt(5), -1/120*sqrt(5), -1/30*sqrt(5), 1/30*sqrt(5), -1/120*sqrt(5), 1/120*sqrt(5), 1/120*sqrt(5), -1/120*sqrt(5), -1/24*sqrt(5), 1/12*sqrt(5), -1/24*sqrt(5), -1/24*sqrt(5), 1/12*sqrt(5), -1/24*sqrt(5), -1/30*sqrt(5), 1/30*sqrt(5), -1/60*sqrt(5), -1/30*sqrt(5), 1/15*sqrt(5), -1/60*sqrt(5), -1/15*sqrt(5), 1/30*sqrt(5), 1/10*sqrt(5), -1/15*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5), -2/15*sqrt(5), 1/30*sqrt(5), 1/30*sqrt(5), 1/15*sqrt(5)], [0, 0, 0, 0, -1/20*sqrt(5), 1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), 0, 0, 0, 0, 1/20*sqrt(5), 0, -1/20*sqrt(5), -1/20*sqrt(5), 0, 1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), -1/20*sqrt(5), 1/10*sqrt(5), 0, -1/20*sqrt(5), -1/20*sqrt(5), 0, 1/10*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5)], [0, -1/40*sqrt(5), 1/20*sqrt(5), -1/40*sqrt(5), -1/40*sqrt(5), -1/20*sqrt(5), 1/10*sqrt(5), -1/40*sqrt(5), 0, 0, -1/40*sqrt(5), 1/40*sqrt(5), 1/40*sqrt(5), -1/40*sqrt(5), -1/40*sqrt(5), 1/20*sqrt(5), -1/40*sqrt(5), -1/40*sqrt(5), 1/20*sqrt(5), -1/40*sqrt(5), 0, 0, -1/20*sqrt(5), 1/10*sqrt(5), 0, -1/20*sqrt(5), 0, 1/10*sqrt(5), -1/10*sqrt(5), 0, 1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), 1/10*sqrt(5), 0, 0, -1/10*sqrt(5)], [0, 0, 0, 0, 1/36*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), 0, 1/18*sqrt(3)*sqrt(2), -1/18*sqrt(3)*sqrt(2), -1/18*sqrt(3)*sqrt(2), 1/18*sqrt(3)*sqrt(2), 0, 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), -1/9*sqrt(3)*sqrt(2), -1/18*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/18*sqrt(3)*sqrt(2), 0, -1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2)], [0, 1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/60*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), 1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/45*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), -1/90*sqrt(5)*sqrt(3)*sqrt(2), 1/90*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), -1/360*sqrt(5)*sqrt(3)*sqrt(2), -1/360*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), -1/120*sqrt(5)*sqrt(3)*sqrt(2), 1/180*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), 1/180*sqrt(5)*sqrt(3)*sqrt(2), -1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/45*sqrt(5)*sqrt(3)*sqrt(2), 1/45*sqrt(5)*sqrt(3)*sqrt(2), 0, 1/18*sqrt(5)*sqrt(3)*sqrt(2), -1/18*sqrt(5)*sqrt(3)*sqrt(2), 0, -1/36*sqrt(5)*sqrt(3)*sqrt(2), 1/18*sqrt(5)*sqrt(3)*sqrt(2), 0, -1/36*sqrt(5)*sqrt(3)*sqrt(2), 1/36*sqrt(5)*sqrt(3)*sqrt(2), -1/36*sqrt(5)*sqrt(3)*sqrt(2), 1/36*sqrt(5)*sqrt(3)*sqrt(2), -1/36*sqrt(5)*sqrt(3)*sqrt(2), -1/30*sqrt(5)*sqrt(3)*sqrt(2), 1/45*sqrt(5)*sqrt(3)*sqrt(2), 1/45*sqrt(5)*sqrt(3)*sqrt(2), -1/90*sqrt(5)*sqrt(3)*sqrt(2)], [0, -1/60*sqrt(5), 1/30*sqrt(5), -1/60*sqrt(5), 1/30*sqrt(5), -1/20*sqrt(5), -1/60*sqrt(5), 1/30*sqrt(5), -1/60*sqrt(5), 1/60*sqrt(5), -1/60*sqrt(5), 1/60*sqrt(5), 1/60*sqrt(5), -1/60*sqrt(5), -1/30*sqrt(5), -1/30*sqrt(5), 1/15*sqrt(5), 1/15*sqrt(5), -1/30*sqrt(5), -1/30*sqrt(5), -1/60*sqrt(5), 1/60*sqrt(5), 1/60*sqrt(5), 1/30*sqrt(5), -1/15*sqrt(5), 1/60*sqrt(5), 1/60*sqrt(5), -2/15*sqrt(5), 1/10*sqrt(5), 1/60*sqrt(5), 1/12*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5), -1/12*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5)], [0, 0, 0, 0, 1/15*sqrt(5), 0, -2/15*sqrt(5), 1/15*sqrt(5), 0, 0, -1/30*sqrt(5), 1/30*sqrt(5), 1/30*sqrt(5), -1/30*sqrt(5), 0, 0, 0, 0, 0, 0, 1/15*sqrt(5), -1/15*sqrt(5), 0, 0, 0, 0, -1/15*sqrt(5), 2/15*sqrt(5), 0, -1/15*sqrt(5), -1/30*sqrt(5), 1/30*sqrt(5), -1/30*sqrt(5), 1/30*sqrt(5), 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 0, 0, 0, 0, 0, 1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2), 0, 0, 0, 0, 1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/30*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 1/15*sqrt(5)*sqrt(2)], [0, 1/30*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/30*sqrt(5)*sqrt(2), 1/30*sqrt(5)*sqrt(2), 0, 0, 1/30*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 0, 0, -1/30*sqrt(5)*sqrt(2), 0, 1/15*sqrt(5)*sqrt(2), -1/30*sqrt(5)*sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 2/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), -1/15*sqrt(5)*sqrt(2), 0], [0, 0, 0, 0, 1/36*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), 0, -1/18*sqrt(3)*sqrt(2), 1/18*sqrt(3)*sqrt(2), 1/18*sqrt(3)*sqrt(2), -1/18*sqrt(3)*sqrt(2), 0, 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), 1/9*sqrt(3)*sqrt(2), 1/18*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/18*sqrt(3)*sqrt(2), 0, -1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), -1/36*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2), 1/36*sqrt(3)*sqrt(2)], [0, -1/120*sqrt(5)*sqrt(3)*sqrt(2), 1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/120*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), 1/60*sqrt(5)*sqrt(3)*sqrt(2), -1/45*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), 1/90*sqrt(5)*sqrt(3)*sqrt(2), -1/90*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), -1/360*sqrt(5)*sqrt(3)*sqrt(2), -1/360*sqrt(5)*sqrt(3)*sqrt(2), 1/360*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/180*sqrt(5)*sqrt(3)*sqrt(2), -1/360*sqrt(5)*sqrt(3)*sqrt(2), -1/360*sqrt(5)*sqrt(3)*sqrt(2), -1/180*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/45*sqrt(5)*sqrt(3)*sqrt(2), 1/45*sqrt(5)*sqrt(3)*sqrt(2), 0, -1/18*sqrt(5)*sqrt(3)*sqrt(2), 1/18*sqrt(5)*sqrt(3)*sqrt(2), 0, -1/36*sqrt(5)*sqrt(3)*sqrt(2), 1/18*sqrt(5)*sqrt(3)*sqrt(2), 0, -1/36*sqrt(5)*sqrt(3)*sqrt(2), 1/36*sqrt(5)*sqrt(3)*sqrt(2), -1/36*sqrt(5)*sqrt(3)*sqrt(2), 1/36*sqrt(5)*sqrt(3)*sqrt(2), -1/36*sqrt(5)*sqrt(3)*sqrt(2), 1/30*sqrt(5)*sqrt(3)*sqrt(2), -1/45*sqrt(5)*sqrt(3)*sqrt(2), -1/45*sqrt(5)*sqrt(3)*sqrt(2), 1/90*sqrt(5)*sqrt(3)*sqrt(2)], [0, 1/60*sqrt(5), -1/30*sqrt(5), 1/60*sqrt(5), 1/30*sqrt(5), -1/20*sqrt(5), -1/60*sqrt(5), 1/30*sqrt(5), 1/60*sqrt(5), -1/60*sqrt(5), -1/60*sqrt(5), 1/60*sqrt(5), 1/60*sqrt(5), -1/60*sqrt(5), 1/30*sqrt(5), 1/30*sqrt(5), -1/15*sqrt(5), -1/15*sqrt(5), 1/30*sqrt(5), 1/30*sqrt(5), -1/60*sqrt(5), 1/60*sqrt(5), -1/60*sqrt(5), -1/30*sqrt(5), 1/15*sqrt(5), -1/60*sqrt(5), 1/60*sqrt(5), -2/15*sqrt(5), 1/10*sqrt(5), 1/60*sqrt(5), 1/12*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5), -1/12*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5), 1/12*sqrt(5), -1/12*sqrt(5)], [0, 0, 0, 0, -1/20*sqrt(5), 1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), 0, 0, 0, 0, -1/20*sqrt(5), 0, 1/20*sqrt(5), 1/20*sqrt(5), 0, -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/10*sqrt(5), 0, 1/20*sqrt(5), -1/20*sqrt(5), 0, 1/10*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5)], [0, 1/40*sqrt(5), -1/20*sqrt(5), 1/40*sqrt(5), -1/40*sqrt(5), -1/20*sqrt(5), 1/10*sqrt(5), -1/40*sqrt(5), 0, 0, -1/40*sqrt(5), 1/40*sqrt(5), 1/40*sqrt(5), -1/40*sqrt(5), 1/40*sqrt(5), -1/20*sqrt(5), 1/40*sqrt(5), 1/40*sqrt(5), -1/20*sqrt(5), 1/40*sqrt(5), 0, 0, 1/20*sqrt(5), -1/10*sqrt(5), 0, 1/20*sqrt(5), 0, 1/10*sqrt(5), -1/10*sqrt(5), 0, 1/20*sqrt(5), -1/20*sqrt(5), 1/20*sqrt(5), -1/20*sqrt(5), -1/10*sqrt(5), 0, 0, 1/10*sqrt(5)], [0, 1/120*sqrt(5), -1/60*sqrt(5), 1/120*sqrt(5), -1/120*sqrt(5), -1/20*sqrt(5), 1/15*sqrt(5), -1/120*sqrt(5), 1/30*sqrt(5), -1/30*sqrt(5), -1/120*sqrt(5), 1/120*sqrt(5), 1/120*sqrt(5), -1/120*sqrt(5), 1/24*sqrt(5), -1/12*sqrt(5), 1/24*sqrt(5), 1/24*sqrt(5), -1/12*sqrt(5), 1/24*sqrt(5), -1/30*sqrt(5), 1/30*sqrt(5), 1/60*sqrt(5), 1/30*sqrt(5), -1/15*sqrt(5), 1/60*sqrt(5), -1/15*sqrt(5), 1/30*sqrt(5), 1/10*sqrt(5), -1/15*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5), -1/12*sqrt(5), 1/12*sqrt(5), 2/15*sqrt(5), -1/30*sqrt(5), -1/30*sqrt(5), -1/15*sqrt(5)]]

operators = [Operator([3,3],coefficients=cfs, traces=tracelist) for cfs in coefficients]

gl2basis_fc33_lambda42 = Basis([3,3],operators,labels=labels)

In [7]:
#Code generating the GL(2) basis with field content 3,3 and Lambda = [3,3]
labels = [([3, 3], [4, 1, 1], 1), ([3, 3], [3, 2, 1], 1), ([3, 3], [3, 2, 1], 2), ([3, 3], [3, 1, 1, 1], 1)]

tracelist = [ Trace([3,3], [1,2,3,4,5,6]), 
            Trace([3,3],[(1,2)]), Trace([3,3],[(1,4)]), Trace([3,3],[(4,5)]),
            Trace([3,3],[(1,2),(3,4)]), Trace([3,3],[(1,2),(4,5)]), Trace([3,3],[(1,4),(2,5)]), Trace([3,3],[(1,4),(5,6)]),
            Trace([3,3],[(1,2),(3,4),(5,6)]), Trace([3,3],[(1,4),(2,5),(3,6)]),
            Trace([3,3],[(1,2,3)]), Trace([3,3],[(1,2,4)]), Trace([3,3],[(1,4,5)]),  Trace([3,3],[(4,5,6)]),
            Trace([3,3],[(1,2,3),(4,5)]), Trace([3,3],[(1,2,4),(3,5)]), Trace([3,3],[(1,2,4),(5,6)]),
            Trace([3,3],[(1,4,5),(2,3)]), Trace([3,3],[(1,4,5),(2,6)]), Trace([3,3],[(1,2),(4,5,6)]),
            Trace([3,3],[(1,2,3),(4,5,6)]), Trace([3,3],[(1,2,4),(3,5,6)]), 
            Trace([3,3],[(1,2,3,4)]), Trace([3,3],[(1,2,4,5)]), Trace([3,3],[(1,4,2,5)]), Trace([3,3],[(1,4,5,6)]), 
            Trace([3,3],[(1,2,3,4),(5,6)]), Trace([3,3],[(1,2,4,5),(3,6)]), Trace([3,3],[(1,4,2,5),(3,6)]), 
            Trace([3,3],[(1,4,5,6),(2,3)]),
            Trace([3,3],[(1,2,3,4,5)]), Trace([3,3],[(1,2,4,3,5)]), Trace([3,3],[(1,2,4,5,6)]), Trace([3,3],[(1,4,2,5,6)]),
            Trace([3,3],[(1,2,3,4,5,6)]), Trace([3,3],[(1,2,4,3,5,6)]), Trace([3,3],[(1,2,4,5,3,6)]), 
            Trace([3,3],[(1,4,2,5,3,6)])]

coefficients = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/36*sqrt(3), -1/12*sqrt(3), 1/12*sqrt(3), -1/36*sqrt(3), 1/12*sqrt(3), -1/6*sqrt(3), -1/12*sqrt(3), 1/12*sqrt(3), 1/6*sqrt(3), -1/12*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/6*sqrt(3), -1/6*sqrt(3), -1/6*sqrt(3), 1/6*sqrt(3), 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/18*sqrt(3), -1/6*sqrt(3), 1/6*sqrt(3), -1/18*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/6*sqrt(3), 1/6*sqrt(3), 1/6*sqrt(3), -1/6*sqrt(3), 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/2*sqrt(2), 1/2*sqrt(2), 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/36*sqrt(3), -1/12*sqrt(3), 1/12*sqrt(3), -1/36*sqrt(3), -1/12*sqrt(3), 1/6*sqrt(3), 1/12*sqrt(3), -1/12*sqrt(3), -1/6*sqrt(3), 1/12*sqrt(3), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/6*sqrt(3), -1/6*sqrt(3), -1/6*sqrt(3), 1/6*sqrt(3), 0, 0, 0, 0]]

operators = [Operator([3,3],coefficients=cfs, traces=tracelist) for cfs in coefficients]

gl2basis_fc33_lambda33 = Basis([3,3],operators,labels=labels)

In [8]:
# Generating bps basis for field content [3,3], lambda = [3,3]
Gmatrix = diagonal_matrix([1/((N+3)*(N+2)*(N+1)*N*(N-1)*(N-2)), 1/((N+2)*(N+1)*N^2*(N-1)*(N-2)),
                          1/((N+2)*(N+1)*N^2*(N-1)*(N-2)), 1/((N+2)*(N+1)*N*(N-1)*(N-2)*(N-3))])

Fmatrix = diagonal_matrix([((N+3)*(N+2)*(N+1)*N*(N-1)*(N-2)), ((N+2)*(N+1)*N^2*(N-1)*(N-2)),
                          ((N+2)*(N+1)*N^2*(N-1)*(N-2)), ((N+2)*(N+1)*N*(N-1)*(N-2)*(N-3))])

comm42_51 = lowerOperator(Operator([4,2], [1,-1], permutations=[[(1,2,3,5,6)],[(1,2,5,3,6)]]))
comm42_6_1 = lowerOperator(Operator([4,2], [1,-1], permutations=[[(1,2,3,4,5,6)],[(1,2,3,5,4,6)]]))
comm42_6_2 = lowerOperator(Operator([4,2], [1,1,-2], permutations=[[(1,2,3,4,5,6)],[(1,2,3,5,4,6)],[(1,2,5,3,4,6)]]))

comm51_1 = Operator([3,3],[1,-1],permutations=[[(1,2,3,4,5)],[(1,2,4,3,5)]])
comm51_2 = Operator([3,3],[1,-1],permutations=[[(2,3,4,5,6)],[(2,4,3,5,6)]])
comm6_1 = Operator([3,3],[1,-1],permutations=[[(1,2,3,4,5,6)],[(1,2,4,3,5,6)]])
comm6_2 = Operator([3,3],[1,1,-2],permutations=[[(1,2,3,4,5,6)],[(1,2,4,3,5,6)],[(1,2,4,5,3,6)]])
comm6_3 = Operator([3,3],[1,1,1,-3],permutations=[[(1,2,3,4,5,6)],[(1,2,4,3,5,6)],[(1,2,4,5,3,6)],[(1,4,2,5,3,6)]])

comm33_51 = comm51_1 - (calculator33.calculate(comm51_1,comm42_51,'planar')/calculator33.calculate(comm42_51,comm42_51,'planar')) * comm42_51
comm33_6 = ( comm6_1 - (calculator33.calculate(comm6_1,comm42_6_1,'planar')/calculator33.calculate(comm42_6_1,comm42_6_1,'planar')) * comm42_6_1
            - (calculator33.calculate(comm6_1,comm42_6_2,'planar')/calculator33.calculate(comm42_6_2,comm42_6_2,'planar')) * comm42_6_2)

normalisedcommbasis = Basis([3,3], [comm33_51,comm33_6])
for i in range(len(normalisedcommbasis)):
    normalisedcommbasis.operators[i] /= sqrt(calculator33.calculate(normalisedcommbasis[i],normalisedcommbasis[i],'planar'))
    normalisedcommbasis[i].canonicalizeRadicalsInCoefficients()
commvectors = [gl2basis_fc33_lambda33.expand(comm) for comm in normalisedcommbasis]

symtracevectors = []
Gnorms = []
indexorder = [3,2,1,0]
for index in indexorder:
    symtracevector = vector([0 for i in range(4)])
    symtracevector[index] = 1
    for commvec in commvectors:
        symtracevector -= (symtracevector * commvec) * commvec
    for i in range(len(symtracevectors)):
        symtracevector -= (symtracevector * Gmatrix * symtracevectors[i])/Gnorms[i] * symtracevectors[i]
        for j in range(len(symtracevector)):
            symtracevector[j] = symtracevector[j].canonicalize_radical().factor()
    planarNorm = (symtracevector * symtracevector).canonicalize_radical().factor()
    if planarNorm != 0:
        print(index)
        print(gl2basis_fc33_lambda33.labels[index])
        symtracevector *= 1/sqrt(planarNorm)
        for i in range(len(symtracevector)):
            symtracevector[i] = symtracevector[i].canonicalize_radical().factor()
        Gnorms.append((symtracevector * Gmatrix * symtracevector).canonicalize_radical().factor())
        symtracevectors.append(symtracevector)
        

labels = [([3, 3], [3, 1, 1, 1], 1),([3, 3], [3, 2, 1], 1)]
operators = []
for vec in symtracevectors:
    operators.append(gl2basis_fc33_lambda33.linearCombination(list(vec)))
symtracevectors_fc33_lambda33 = symtracevectors
symtracebasis_fc33_lambda33 = Basis([3,3], operators=operators, labels=labels)
    

bpsvectors = [vector([(vec[i]*Gmatrix[i,i]).factor() for i in range(4)]) for vec in symtracevectors]
for i in range(len(bpsvectors)):
    planarNorm = (bpsvectors[i] * bpsvectors[i]).canonicalize_radical().factor()
    bpsvectors[i] *= 1/sqrt(planarNorm)
    for j in range(len(bpsvectors[i])):
        bpsvectors[i][j] = bpsvectors[i][j].canonicalize_radical().factor()
Fnorms = [(vec * Fmatrix * vec).canonicalize_radical().factor() for vec in bpsvectors]

labels = [([3, 3], [3, 1, 1, 1], 1),([3, 3], [3, 2, 1], 1)]
operators = []
for vec in bpsvectors:
    operators.append(gl2basis_fc33_lambda33.linearCombination(list(vec)))
bpsvectors_fc33_lambda33 = bpsvectors
bpsbasis_fc33_lambda33 = Basis([3,3],operators=operators, labels=labels)

3
([3, 3], [3, 1, 1, 1], 1)
1
([3, 3], [3, 2, 1], 1)


In [ ]:
msbasis_fc33= orthogonalisedBasis([3,3])

In [ ]:
#Combining the different lambda sectors into one basis (for field content 3,3)
labels = gl2basis_fc33_lambda42.labels + gl2basis_fc33_lambda33.labels
operators = gl2basis_fc33_lambda42.operators + gl2basis_fc33_lambda33.operators

gl2basis_fc33 = Basis ([3,3], operators, labels)


In [ ]:
#Everything from here onwards is rough working. There may be some useful code in there, but it's not in any order.

In [6]:
GL2msbasis32 = GL2multiSymmetricBasis([3,2])

In [19]:
gl2msbasisvectors32 = [gl2basis_fc32.expand(op) for lbl,op in GL2msbasis32]

bpsvectors32 = [vector([vec[i]/Fmatrix[i,i] for i in range(18)]) for vec in gl2msbasisvectors32]
Divisors = [sqrt(vec * vec).canonicalize_radical().factor() for vec in bpsvectors32]

bpsoperatorlist32 = [gl2basis_fc32.linearCombination(vec) for vec in bpsvectors32]
for op in bpsoperatorlist32:
    op.canonicalizeRadicalsInCoefficients()
    op.factorCoefficients()

gl2bpsbasis32 = Basis([3,2],bpsoperatorlist32,labels)

In [81]:
i=0
j=1
((bpsvectors32[i] * bpsvectors32[j])/Divisors[i]/Divisors[j]).canonicalize_radical().factor()

-(3*N^6 - 24*N^5 + 106*N^4 - 428*N^3 + 1123*N^2 - 1104*N + 2520)*(N + 4)*N/(sqrt(5*N^8 - 40*N^7 + 112*N^6 - 184*N^5 + 767*N^4 - 1128*N^3 + 3528*N^2 - 5184*N + 10368)*sqrt(4*N^8 - 20*N^7 + 131*N^6 - 782*N^5 + 1535*N^4 - 3464*N^3 + 14728*N^2 - 1152*N + 40320))

In [22]:
gl2bpsbasis32.labels

[([5], [5], 1),
 ([5], [4, 1], 1),
 ([5], [3, 2], 1),
 ([5], [3, 1, 1], 1),
 ([5], [2, 2, 1], 1),
 ([5], [2, 1, 1, 1], 1),
 ([5], [1, 1, 1, 1, 1], 1),
 ([4, 1], [4, 1], 1),
 ([4, 1], [3, 2], 1),
 ([4, 1], [3, 1, 1], 1),
 ([4, 1], [2, 2, 1], 1),
 ([4, 1], [2, 1, 1, 1], 1),
 ([3, 2], [2, 1, 1, 1], 1),
 ([3, 2], [2, 2, 1], 1),
 ([3, 2], [3, 1, 1], 1),
 ([3, 2], [3, 2], 1)]

In [75]:
i = 0
for cf, tr in gl2bpsbasis32[i]:
    print(cf/Divisors[i])
    print(tr)

-1/120*sqrt(5)*sqrt(2)*(N^4 + 9*N^3 + 26*N^2 + 24*N - 1680)/sqrt(4*N^8 - 20*N^7 + 131*N^6 - 782*N^5 + 1535*N^4 - 3464*N^3 + 14728*N^2 - 1152*N + 40320)
(TrX)(TrX)(TrX)(TrY)(TrY)
1/40*sqrt(5)*sqrt(2)*(N^3 + 9*N^2 + 26*N - 396)*N/sqrt(4*N^8 - 20*N^7 + 131*N^6 - 782*N^5 + 1535*N^4 - 3464*N^3 + 14728*N^2 - 1152*N + 40320)
(TrX)(TrXX)(TrY)(TrY)
1/20*sqrt(5)*sqrt(2)*(N^3 + 9*N^2 + 26*N - 396)*N/sqrt(4*N^8 - 20*N^7 + 131*N^6 - 782*N^5 + 1535*N^4 - 3464*N^3 + 14728*N^2 - 1152*N + 40320)
(TrX)(TrX)(TrXY)(TrY)
1/120*sqrt(5)*sqrt(2)*(N^3 + 9*N^2 + 26*N - 396)*N/sqrt(4*N^8 - 20*N^7 + 131*N^6 - 782*N^5 + 1535*N^4 - 3464*N^3 + 14728*N^2 - 1152*N + 40320)
(TrX)(TrX)(TrX)(TrYY)
-1/20*sqrt(5)*sqrt(2)*(N^3 + 11*N^2 - 72*N - 120)*(N - 2)/sqrt(4*N^8 - 20*N^7 + 131*N^6 - 782*N^5 + 1535*N^4 - 3464*N^3 + 14728*N^2 - 1152*N + 40320)
(TrXX)(TrXY)(TrY)
-1/40*sqrt(5)*sqrt(2)*(N^3 + 11*N^2 - 72*N - 120)*(N - 2)/sqrt(4*N^8 - 20*N^7 + 131*N^6 - 782*N^5 + 1535*N^4 - 3464*N^3 + 14728*N^2 - 1152*N + 40320)
(TrX)(TrXX)

In [79]:
i = 15
for cf in bpsvectors32[i]:
    print(cf/Divisors[i])

0
0
0
0
0
0
0
0
0
0
0
0
1/15*sqrt(5)*sqrt(3)*(N - 2)*(N - 3)*N/sqrt(10*N^6 + 74*N^5 + 199*N^4 + 252*N^3 + 351*N^2 + 648*N + 702)
-1/6*sqrt(3)*(N + 3)*(N - 2)*(N - 3)/sqrt(10*N^6 + 74*N^5 + 199*N^4 + 252*N^3 + 351*N^2 + 648*N + 702)
2/5*sqrt(5)*(N + 3)*(N - 3)*N/sqrt(10*N^6 + 74*N^5 + 199*N^4 + 252*N^3 + 351*N^2 + 648*N + 702)
1/10*sqrt(5)*sqrt(3)*sqrt(2)*(N + 3)*(N - 3)*N/sqrt(10*N^6 + 74*N^5 + 199*N^4 + 252*N^3 + 351*N^2 + 648*N + 702)
-5/6*sqrt(3)*(N + 3)*(N + 2)*(N - 3)/sqrt(10*N^6 + 74*N^5 + 199*N^4 + 252*N^3 + 351*N^2 + 648*N + 702)
-2/3*sqrt(5)*sqrt(3)*(N + 3)*(N + 2)*N/sqrt(10*N^6 + 74*N^5 + 199*N^4 + 252*N^3 + 351*N^2 + 648*N + 702)


In [67]:
op = msbasis22.linearCombination(1/6*vector([0,0,0,0,0,1,-2,1,0]))
print(op)

-1/3(TrXX)(TrYY) + 1/6(TrXX)(TrY)(TrY) + 1/3(TrXY)(TrXY) - 1/3(TrX)(TrXY)(TrY) + 1/6(TrX)(TrX)(TrYY)


In [65]:
GL2msbasis22[8]

(-1/3)(TrXX)(TrYY) + (1/6)(TrXX)(TrY)(TrY) + (1/3)(TrXY)(TrXY) + (-1/3)(TrX)(TrXY)(TrY) + (1/6)(TrX)(TrX)(TrYY)

In [66]:
msbasis22.expand(GL2msbasis22[8])

(0, 0, 0, 0, 0, 1/6, -1/3, 1/6, 0)

In [16]:
msbasis22 = multiSymmetricBasis([2,2])
msbasis22.labels

[[[2, 2]],
 [[2, 1], [0, 1]],
 [[2, 0], [0, 2]],
 [[1, 2], [1, 0]],
 [[1, 1], [1, 1]],
 [[2, 0], [0, 1], [0, 1]],
 [[1, 1], [1, 0], [0, 1]],
 [[1, 0], [1, 0], [0, 2]],
 [[1, 0], [1, 0], [0, 1], [0, 1]]]

In [15]:
GL2msbasis22 = GL2multiSymmetricBasis([2,2])
GL2msbasis22.labels

[([4, 0], [4], 1),
 ([4, 0], [3, 1], 1),
 ([4, 0], [2, 2], 1),
 ([4, 0], [2, 1, 1], 1),
 ([4, 0], [1, 1, 1, 1], 1),
 ([3, 1], [3, 1], 1),
 ([3, 1], [2, 1, 1], 1),
 ([2, 2], [2, 2], 1),
 ([2, 2], [2, 1, 1], 1)]

In [19]:
gl2basis_fc22.labels

[([4], [4], 1),
 ([4], [3, 1], 1),
 ([4], [2, 2], 1),
 ([4], [2, 1, 1], 1),
 ([4], [1, 1, 1, 1], 1),
 ([3, 1], [3, 1], 1),
 ([3, 1], [2, 1, 1], 1),
 ([2, 2], [3, 1], 1),
 ([2, 2], [2, 2], 1),
 ([2, 2], [2, 1, 1], 1)]

In [85]:
gl2msbasisvectors = [gl2basis_fc22.expand(op) for lbl,op in GL2msbasis22]
F = diagonal_matrix([ N*(N+1)*(N+2)*(N+3) , (N-1)*N*(N+1)*(N+2) , (N-1)*N^2*(N+1) , (N-2)*(N-1)*N*(N+1) , (N-3)*(N-2)*(N-1)*N ,
    (N-1)*N*(N+1)*(N+2) , (N-2)*(N-1)*N*(N+1) , (N-1)*N*(N+1)*(N+2) , (N-1)*N^2*(N+1) , (N-2)*(N-1)*N*(N+1) ])

G = diagonal_matrix([1/F[i,i] for i in range(10)])

bpsvectors = [vector([vec[i]/F[i,i] for i in range(10)]) for vec in gl2msbasisvectors]
Divisors = [sqrt(vec * vec).canonicalize_radical().factor() for vec in bpsvectors]

bpsoperatorlist = [gl2basis_fc22.linearCombination(vec) for vec in bpsvectors]
for op in bpsoperatorlist:
    op.canonicalizeRadicalsInCoefficients()
    op.factorCoefficients()

gl2bpsbasis = Basis([2,2],bpsoperatorlist,[([4, 0], [4], 1), ([4, 0], [3, 1], 1), ([4, 0], [2, 2], 1), ([4, 0], [2, 1, 1], 1),
 ([4, 0], [1, 1, 1, 1], 1), ([3, 1], [3, 1], 1), ([3, 1], [2, 1, 1], 1), ([2, 2], [2, 2], 1), ([2, 2], [2, 1, 1], 1)])

In [59]:
gl2bpsbasis[0]*(N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N

(-5/2*sqrt(3)*sqrt(2))(TrX)(TrX)(TrY)(TrY) + (5/6*sqrt(3)*sqrt(2)*N)(TrXX)(TrY)(TrY) + (10/3*sqrt(3)*sqrt(2)*N)(TrX)(TrXY)(TrY) + (5/6*sqrt(3)*sqrt(2)*N)(TrX)(TrX)(TrYY) + (-sqrt(3)*sqrt(2)*(N^2 + 1))(TrXXY)(TrY) + (-sqrt(3)*sqrt(2)*(N^2 + 1))(TrX)(TrXYY) + (-1/6*sqrt(3)*sqrt(2)*(2*N^2 - 3))(TrXX)(TrYY) + (-1/3*sqrt(3)*sqrt(2)*(2*N^2 - 3))(TrXY)(TrXY) + (1/3*sqrt(3)*sqrt(2)*(N^2 + 1)*N)(TrXXYY) + (1/6*sqrt(3)*sqrt(2)*(N^2 + 1)*N)(TrXYXY)

In [71]:
gl2msbasisvectors

[(1/2, -1/2, 0, 1/2, -1/2, 0, 0, 0, 0, 0),
 (0, 1/7*sqrt(7), -1/7*sqrt(7), -1/7*sqrt(7), 2/7*sqrt(7), 0, 0, 0, 0, 0),
 (0, 0, 1/3*sqrt(3), -1/3*sqrt(3), 1/3*sqrt(3), 0, 0, 0, 0, 0),
 (0, 0, 0, 1/10*sqrt(5)*sqrt(2), -3/10*sqrt(5)*sqrt(2), 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 1/2*sqrt(2), 1/2*sqrt(2), 0, 0, 0),
 (0, 0, 0, 0, 0, 0, -1, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 1/2*sqrt(2), 0, 1/2*sqrt(2)),
 (0, 0, 0, 0, 0, 0, 0, -1/6*sqrt(3), -1/6*sqrt(3)*sqrt(2), -1/2*sqrt(3))]

In [73]:
vec1 = gl2msbasisvectors[-2]
vec2 = gl2msbasisvectors[-1]

In [84]:
newvec = sqrt(3)* (vec1 - (vec1 * vec2) * vec2)
print(newvec)
print(3*sqrt(2)*gl2basis_fc22.linearCombination(newvec))

(0, 0, 0, 0, 0, 0, 0, 1/3*sqrt(3)*sqrt(2), -1/3*sqrt(3), 0)
(1)(TrXX)(TrY)(TrY) + (-2)(TrX)(TrXY)(TrY) + (1)(TrX)(TrX)(TrYY) + (1)(TrXX)(TrYY) + (-1)(TrXY)(TrXY)


In [96]:
((G*vec2) * (G*vec2)).factor()

1/3*(3*N^4 + 8*N^3 + 6*N^2 + 8)/((N + 2)^2*(N + 1)^2*(N - 1)^2*(N - 2)^2*N^4)

In [97]:
(3*N^4 + 8*N^3 + 6*N^2 + 8).factor()

3*N^4 + 8*N^3 + 6*N^2 + 8

In [100]:
vec = vector([0, 0, 0, 0, 0, 0, 0, -1/6*sqrt(3), -1/6*sqrt(3)*sqrt(2), 0])
((G*vec) * (G*vec)).factor()

1/12*(3*N^2 + 8*N + 8)/((N + 2)^2*(N + 1)^2*(N - 1)^2*N^4)

In [116]:
calculator22.calculate(bps22_2,bps22_2,'F').factor()

(3*N^2 + 4*N - 2)*(N + 1)*(N - 1)*N^2/(3*N^2 + 2)

In [114]:
P1 = 3*N^4 + 8*N^3 + 6*N^2 + 8
P2 = 3*N^2 + 2
Q1 = 3*N^2 + 4*N - 2
D = N^2 - 4*N - 2

In [117]:
(((N+2)*(N-2)*P2 - P1)^2 + 8*D^2 * (N+2) *(N-2)).expand().factor()

8*(3*N^2 + 4*N - 2)^2*(N^2 + 4)

In [123]:
(P1*P2-2*D).expand().factor()

9*N^6 + 24*N^5 + 24*N^4 + 16*N^3 + 34*N^2 + 8*N + 20

In [120]:
GL2msbasis22

([4, 0], [4], 1):          (1/3*sqrt(3)*sqrt(2))(TrXXYY) + (1/6*sqrt(3)*sqrt(2))(TrXYXY)
([4, 0], [3, 1], 1):       (-2/21*sqrt(7)*sqrt(3)*sqrt(2))(TrXXYY) + (-1/21*sqrt(7)*sqrt(3)*sqrt(2))(TrXYXY) + (1/14*sqrt(7)*sqrt(3)*sqrt(2))(TrXXY)(TrY) + (1/14*sqrt(7)*sqrt(3)*sqrt(2))(TrX)(TrXYY)
([4, 0], [2, 2], 1):       (-1/3*sqrt(2))(TrXXYY) + (-1/6*sqrt(2))(TrXYXY) + (1/3*sqrt(2))(TrXY)(TrXY) + (1/6*sqrt(2))(TrXX)(TrYY)
([4, 0], [2, 1, 1], 1):    (2/15*sqrt(5)*sqrt(3))(TrXXYY) + (1/15*sqrt(5)*sqrt(3))(TrXYXY) + (-1/10*sqrt(5)*sqrt(3))(TrXXY)(TrY) + (-1/10*sqrt(5)*sqrt(3))(TrX)(TrXYY) + (-1/15*sqrt(5)*sqrt(3))(TrXY)(TrXY) + (-1/30*sqrt(5)*sqrt(3))(TrXX)(TrYY) + (1/60*sqrt(5)*sqrt(3))(TrXX)(TrY)(TrY) + (1/15*sqrt(5)*sqrt(3))(TrX)(TrXY)(TrY) + (1/60*sqrt(5)*sqrt(3))(TrX)(TrX)(TrYY)
([4, 0], [1, 1, 1, 1], 1): (-1/6*sqrt(3)*sqrt(2))(TrXXYY) + (-1/12*sqrt(3)*sqrt(2))(TrXYXY) + (1/6*sqrt(3)*sqrt(2))(TrXXY)(TrY) + (1/6*sqrt(3)*sqrt(2))(TrX)(TrXYY) + (1/12*sqrt(3)*sqrt(2))(TrXY)(TrXY) + (1/24*sqrt(3

In [8]:
for lbl,op in stbasis:
    print(op)

([[4, 2]], 2/5(TrXXXXYY) + 2/5(TrXXXYXY) + 1/5(TrXXYXXY))
([[4, 1], [0, 1]], (TrXXXXY)(TrY))
([[4, 0], [0, 2]], (TrXXXX)(TrYY))
([[3, 2], [1, 0]], 1/2(TrX)(TrXXXYY) + 1/2(TrX)(TrXXYXY))
([[3, 1], [1, 1]], (TrXXXY)(TrXY))
([[3, 0], [1, 2]], (TrXXX)(TrXYY))
([[2, 2], [2, 0]], 2/3(TrXX)(TrXXYY) + 1/3(TrXX)(TrXYXY))
([[2, 1], [2, 1]], (TrXXY)(TrXXY))
([[4, 0], [0, 1], [0, 1]], (TrXXXX)(TrY)(TrY))
([[3, 1], [1, 0], [0, 1]], (TrX)(TrXXXY)(TrY))
([[3, 0], [1, 1], [0, 1]], (TrXXX)(TrXY)(TrY))
([[2, 1], [2, 0], [0, 1]], (TrXX)(TrXXY)(TrY))
([[3, 0], [1, 0], [0, 2]], (TrX)(TrXXX)(TrYY))
([[2, 0], [2, 0], [0, 2]], (TrXX)(TrXX)(TrYY))
([[2, 2], [1, 0], [1, 0]], 2/3(TrX)(TrX)(TrXXYY) + 1/3(TrX)(TrX)(TrXYXY))
([[2, 1], [1, 1], [1, 0]], (TrX)(TrXXY)(TrXY))
([[2, 0], [1, 2], [1, 0]], (TrX)(TrXX)(TrXYY))
([[2, 0], [1, 1], [1, 1]], (TrXX)(TrXY)(TrXY))
([[3, 0], [1, 0], [0, 1], [0, 1]], (TrX)(TrXXX)(TrY)(TrY))
([[2, 0], [2, 0], [0, 1], [0, 1]], (TrXX)(TrXX)(TrY)(TrY))
([[2, 1], [1, 0], [1, 0], [0, 1]], (

In [12]:
stbasis = symmetrisedTraceBasis([4,2])
msbasis = multiSymmetricBasis([4,2])

In [11]:
for lbl,op in stbasis:
    print(str(lbl) + '     ' + str(op) + '        ' + str(calculator42.calculate(op,op,'planar')))

[[4, 2]]     2/5(TrXXXXYY) + 2/5(TrXXXYXY) + 1/5(TrXXYXXY)        2/5
[[4, 1], [0, 1]]     (TrXXXXY)(TrY)        1
[[4, 0], [0, 2]]     (TrXXXX)(TrYY)        8
[[3, 2], [1, 0]]     1/2(TrX)(TrXXXYY) + 1/2(TrX)(TrXXYXY)        1/2
[[3, 1], [1, 1]]     (TrXXXY)(TrXY)        1
[[3, 0], [1, 2]]     (TrXXX)(TrXYY)        3
[[2, 2], [2, 0]]     2/3(TrXX)(TrXXYY) + 1/3(TrXX)(TrXYXY)        4/3
[[2, 1], [2, 1]]     (TrXXY)(TrXXY)        2
[[4, 0], [0, 1], [0, 1]]     (TrXXXX)(TrY)(TrY)        8
[[3, 1], [1, 0], [0, 1]]     (TrX)(TrXXXY)(TrY)        1
[[3, 0], [1, 1], [0, 1]]     (TrXXX)(TrXY)(TrY)        3
[[2, 1], [2, 0], [0, 1]]     (TrXX)(TrXXY)(TrY)        2
[[3, 0], [1, 0], [0, 2]]     (TrX)(TrXXX)(TrYY)        6
[[2, 0], [2, 0], [0, 2]]     (TrXX)(TrXX)(TrYY)        16
[[2, 2], [1, 0], [1, 0]]     2/3(TrX)(TrX)(TrXXYY) + 1/3(TrX)(TrX)(TrXYXY)        4/3
[[2, 1], [1, 1], [1, 0]]     (TrX)(TrXXY)(TrXY)        1
[[2, 0], [1, 2], [1, 0]]     (TrX)(TrXX)(TrXYY)        2
[[2, 0], [1, 1], [1, 1

In [14]:
for lbl,op in msbasis:
    print(str(lbl) + '     ' + str(calculator42.calculate(op,op,'planar')))

[[4, 2]]     2/5
[[4, 1], [0, 1]]     7/5
[[4, 0], [0, 2]]     42/5
[[3, 2], [1, 0]]     9/10
[[3, 1], [1, 1]]     7/5
[[3, 0], [1, 2]]     17/5
[[2, 2], [2, 0]]     26/15
[[2, 1], [2, 1]]     12/5
[[4, 0], [0, 1], [0, 1]]     108/5
[[3, 1], [1, 0], [0, 1]]     51/10
[[3, 0], [1, 1], [0, 1]]     48/5
[[2, 1], [2, 0], [0, 1]]     119/15
[[3, 0], [1, 0], [0, 2]]     191/10
[[2, 0], [2, 0], [0, 2]]     464/15
[[2, 2], [1, 0], [1, 0]]     94/15
[[2, 1], [1, 1], [1, 0]]     61/10
[[2, 0], [1, 2], [1, 0]]     253/30
[[2, 0], [1, 1], [1, 1]]     164/15
[[3, 0], [1, 0], [0, 1], [0, 1]]     462/5
[[2, 0], [2, 0], [0, 1], [0, 1]]     2096/15
[[2, 1], [1, 0], [1, 0], [0, 1]]     796/15
[[2, 0], [1, 1], [1, 0], [0, 1]]     821/15
[[2, 0], [1, 0], [1, 0], [0, 2]]     2036/15
[[1, 2], [1, 0], [1, 0], [1, 0]]     462/5
[[1, 1], [1, 1], [1, 0], [1, 0]]     1096/15
[[2, 0], [1, 0], [1, 0], [0, 1], [0, 1]]     16936/15
[[1, 1], [1, 0], [1, 0], [1, 0], [0, 1]]     5502/5
[[1, 0], [1, 0], [1, 0], [1, 0], 

In [330]:
symtracebasis_fc33_lambda33.labels

[([3, 3], [3, 1, 1, 1], 1), ([3, 3], [3, 2, 1], 1)]

In [345]:
op = 12 * sqrt(2*N^2 + 3)/sqrt(3) * bpsbasis_fc33_lambda33[1]
op.canonicalizeRadicalsInCoefficients()
print(op)

(N)(TrXXX)(TrY)(TrY)(TrY) + (-3*N)(TrX)(TrXXY)(TrY)(TrY) + (3*N)(TrX)(TrX)(TrXYY)(TrY) + (-N)(TrX)(TrX)(TrX)(TrYYY) + (N)(TrXXX)(TrY)(TrYY) + (-2*N)(TrXXY)(TrXY)(TrY) + (-N)(TrX)(TrXXY)(TrYY) + (N)(TrXX)(TrXYY)(TrY) + (2*N)(TrX)(TrXY)(TrXYY) + (-N)(TrX)(TrXX)(TrYYY) + (-6)(TrXXXYY)(TrY) + (6)(TrXXYXY)(TrY) + (6)(TrX)(TrXXYYY) + (-6)(TrX)(TrXYXYY)


In [306]:
t3111 = Operator([3,3], coefficients=[1,-3,3,-1], permutations=[[(1,2,3)],[(1,2,4)],[(1,4,5)],[(4,5,6)]])
t321 = Operator([3,3], coefficients=[1,-2,-1,1,2,-1], permutations=[[(1,2,3),(4,5)],[(1,2,4),(3,5)],[(1,2,4),(5,6)],
                                                                   [(1,4,5),(2,3)],[(1,4,5),(2,6)],[(1,2),(4,5,6)]])
c51 = Operator([3,3], coefficients=[1,-1,-1,1], permutations=[[(1,2,3,4,5)],[(1,2,4,3,5)],[(1,2,4,5,6)],[(1,4,2,5,6)]])
c6 = Operator([3,3], coefficients=[1,-1], permutations=[[(1,2,4,3,5,6)],[(1,2,4,5,3,6)]])

shortbasis = Basis([3,3], operators=[t3111,t321,c51,c6])

In [349]:
latex(Fnorms[1])

\frac{{\left(2 \, N^{2} + 3 \, N - 3\right)} {\left(N + 2\right)} {\left(N + 1\right)} {\left(N - 1\right)} {\left(N - 2\right)} N^{2}}{2 \, N^{2} + 3}

In [333]:
gl2basis_fc33_lambda33.labels

[([3, 3], [4, 1, 1], 1),
 ([3, 3], [3, 2, 1], 1),
 ([3, 3], [3, 2, 1], 2),
 ([3, 3], [3, 1, 1, 1], 1)]

In [337]:
(3*N^2 + 9*N - 18).factor()

3*N^2 + 9*N - 18

In [214]:
indices = [0,3,4,7,8,9]
i = indices[0]
print('\\begin{align}')
print('\\left| S^{BPS}_{')
print(bpsbasis_fc42_lambda42.labels[i][1])
print('} \\right|^2_F & = ')
print('\\frac{')
print('\\begin{gathered}')
print('\\vspace{-5pt}')
print(latex(Fnorms_fc42_lambda42[i].numerator()))
print('\\end{gathered}')
print('}{')
print('P^{BPS}_' + str(i))
print('}')
print('\\end{align}')

[([4, 2], [2, 1, 1, 1, 1], 1),
 ([4, 2], [2, 2, 1, 1], 1),
 ([4, 2], [2, 2, 1, 1], 2),
 ([4, 2], [3, 1, 1, 1], 1),
 ([4, 2], [2, 2, 2], 1),
 ([4, 2], [3, 2, 1], 1),
 ([4, 2], [3, 2, 1], 2),
 ([4, 2], [4, 1, 1], 1),
 ([4, 2], [3, 3], 1),
 ([4, 2], [4, 2], 1)]

In [264]:
indices = [0,3,4,7,8,9]
i = indices[5]
print('\\fontsize{12pt}{0}')
print('The norm of the BPS operator is')
print('\\fontsize{8pt}{0}')
print('\\begin{align}')
print('\\left| S^{BPS}_{')
print(bpsbasis_fc42_lambda42.labels[i][1])
print('} \\right|^2_F & = ')
print(latex(Fnorms_fc42_lambda42[i]))
print('\\end{align}')
print('\\fontsize{12pt}{0}')
print('Where the polynomials in the numerator are')
print('\\fontsize{8pt}{0}')
print('\\begin{align}')
print('Q_{' + str(i) + ', 1} & = \\\\')
print('Q_{' + str(i) + ', 2} & = ')
print('\\end{align}')

\fontsize{12pt}{0}
The norm of the BPS operator is
\fontsize{8pt}{0}
\begin{align}
\left| S^{BPS}_{
[4, 2]
} \right|^2_F & = 
\frac{{\left(99 \, N^{6} + 162 \, N^{5} - 324 \, N^{4} + 102 \, N^{3} + 152 \, N^{2} - 260 \, N + 120\right)} {\left(3 \, N^{2} - 2\right)} {\left(N + 3\right)} {\left(N + 2\right)} {\left(N + 1\right)} {\left(N - 1\right)} N^{4}}{297 \, N^{10} + 378 \, N^{8} - 1260 \, N^{7} + 390 \, N^{6} + 1080 \, N^{5} - 1256 \, N^{4} + 640 \, N^{3} + 760 \, N^{2} - 1920 \, N + 1440}
\end{align}
\fontsize{12pt}{0}
Where the polynomials in the numerator are
\fontsize{8pt}{0}
\begin{align}
Q_{9, 1} & = \\
Q_{9, 2} & = 
\end{align}


In [191]:
op = symtracebasis_fc33_lambda33[1]
op *= sqrt(N^4 + 3*N^3 - 9*N + 9)
op.factorCoefficients()
for cf in op.coefficients:
    print(cf)

1/24*sqrt(3)*sqrt(2)*(N + 1)*N
-1/8*sqrt(3)*sqrt(2)*(N + 1)*N
1/8*sqrt(3)*sqrt(2)*(N + 1)*N
-1/24*sqrt(3)*sqrt(2)*(N + 1)*N
1/24*sqrt(3)*sqrt(2)*(N^2 + 3*N - 6)
-1/12*sqrt(3)*sqrt(2)*(N^2 + 3*N - 6)
-1/24*sqrt(3)*sqrt(2)*(N^2 + 3*N - 6)
1/24*sqrt(3)*sqrt(2)*(N^2 + 3*N - 6)
1/12*sqrt(3)*sqrt(2)*(N^2 + 3*N - 6)
-1/24*sqrt(3)*sqrt(2)*(N^2 + 3*N - 6)


In [183]:
vectorindex = 0
vec = symtracevectors_fc42_lambda42[vectorindex]
print(vec[0])

1/390*sqrt(13)*sqrt(5)*sqrt(3)


In [181]:
#P = (195*N^10 + 2298*N^9 + 9767*N^8 + 17008*N^7 + 21041*N^6 + 74974*N^5 + 135005*N^4 - 144704*N^3 - 399936*N^2 - 62976*N + 707328)
#vec *= sqrt(P)
for i in range(len(vec)):
#    vec[i]= vec[i].simplify()
    print(vec[i]) 

1/390*sqrt(13)*sqrt(5)*sqrt(3)
0
-1/390*sqrt(13)*sqrt(5)*sqrt(3)
-1/117*sqrt(13)*sqrt(2)
11/1170*sqrt(13)*sqrt(5)*sqrt(2)
0
-1/585*sqrt(13)*sqrt(5)*sqrt(3)
0
-4/585*sqrt(13)*sqrt(5)*sqrt(3)*sqrt(2)
5/117*sqrt(13)*sqrt(2)
29/1170*sqrt(13)*sqrt(5)*sqrt(2)
1/117*sqrt(13)*sqrt(5)*sqrt(3)
1/195*sqrt(13)*sqrt(5)*sqrt(3)
1/30*sqrt(13)*sqrt(5)*sqrt(3)
1/18*sqrt(13)*sqrt(5)*sqrt(3)


In [184]:
c2 = 1170/(sqrt(13)*sqrt(5))
vec *= c2
for i in range(len(vec)):
    vec[i]= vec[i].factor()
    print(vec[i]) 

3*sqrt(3)
0
-3*sqrt(3)
-2*sqrt(5)*sqrt(2)
11*sqrt(2)
0
-2*sqrt(3)
0
-8*sqrt(3)*sqrt(2)
10*sqrt(5)*sqrt(2)
29*sqrt(2)
10*sqrt(3)
6*sqrt(3)
39*sqrt(3)
65*sqrt(3)


In [170]:
polyCfs = [(81*N^5 - 129*N^4 + 51*N^3 + 76*N^2 - 130*N + 60),
    (9*N^5 + 3*N^4 - 3*N^3 - 13*N^2 + 13*N - 6),
    (9*N^3 + 13*N^2 - 13*N + 6)*(3*N^2 - 2),
    (5*N^3 - 2*N^2 - 5*N + 3),
    (29*N^4 - 65*N^3 + 24*N^2 + 70*N - 60),
    (3*N^2 - 3*N + 2),
    (N^2 - 2*N + 2),
    (N^2 - 2*N + 2),
    (3*N^2 - 2),
    1,
    (11*N^3 - 18*N^2 - 10*N + 20),
    1,
    1,
    (3*N^2 - 2),
    (9*N^2 - 10)]
for i in range(len(vec)):
    vec[i] /= polyCfs[i]
    print(vec[i])

-sqrt(3)*sqrt(2)
-6*sqrt(3)*sqrt(2)
-3*sqrt(3)*sqrt(2)
-12*sqrt(5)*N
-6
-10*sqrt(3)*sqrt(2)*(N + 3)*(N - 1)
6*sqrt(3)*sqrt(2)*(N + 3)*N
24*sqrt(3)*(N + 3)
-16*sqrt(3)*(N + 3)*(N - 1)
12*sqrt(5)*(N + 3)*(N + 1)*(N - 1)*N
-6*N - 18
-10*sqrt(3)*sqrt(2)*(N + 3)*(N + 2)
-6*sqrt(3)*sqrt(2)*(N + 3)*(N + 2)
3*sqrt(3)*sqrt(2)*(N + 3)*(N + 2)
-sqrt(3)*sqrt(2)*(N + 3)*(N + 2)


In [152]:
polyCfs = []
for v in vec:
    string = str(v)
    index = string.find('N^2')
    startindex = index
    while startindex > 0 and string[startindex] != '(':
        startindex -= 1
    endindex = index
    while endindex < len(string) and string[endindex] != ')':
        endindex += 1
    code = 'a = ' + string[startindex:endindex+1]
    while code.find('^') > -1:
        i = code.index('^')
        code = code[:i] + '**' + code[i+1:]
    exec(code)
    polyCfs.append(a)
for i in range(len(vec)):
    vec[i] /= polyCfs[i]
    print(vec[i])

-60*sqrt(2)*N
9*sqrt(2)
-18*sqrt(2)
-2*sqrt(5)*sqrt(3)*N
-20*sqrt(3)*N
-3*sqrt(2)
12*sqrt(2)
1
-96*N + 96
2*sqrt(5)*sqrt(3)*N
-20*sqrt(3)*N
-15*sqrt(2)*(N + 2)
-9*sqrt(2)*(N + 2)
18*sqrt(2)*(N + 2)
-60*sqrt(2)*(N + 2)*N


In [361]:
bpsvectors_fc42_lambda42[-1][-4]

-5/9*sqrt(3)*sqrt(2)*(N + 3)*(N + 2)/sqrt(297*N^10 + 378*N^8 - 1260*N^7 + 390*N^6 + 1080*N^5 - 1256*N^4 + 640*N^3 + 760*N^2 - 1920*N + 1440)

In [358]:
bpsbasis_fc42_lambda42.labels

[([4, 2], [2, 1, 1, 1, 1], 1),
 ([4, 2], [2, 2, 1, 1], 1),
 ([4, 2], [2, 2, 1, 1], 2),
 ([4, 2], [3, 1, 1, 1], 1),
 ([4, 2], [2, 2, 2], 1),
 ([4, 2], [3, 2, 1], 1),
 ([4, 2], [3, 2, 1], 2),
 ([4, 2], [4, 1, 1], 1),
 ([4, 2], [3, 3], 1),
 ([4, 2], [4, 2], 1)]

In [359]:
gl2basis_fc42_lambda42.labels

[([4, 2], [5, 1], 1),
 ([4, 2], [4, 2], 1),
 ([4, 2], [4, 2], 2),
 ([4, 2], [4, 1, 1], 1),
 ([4, 2], [4, 1, 1], 2),
 ([4, 2], [3, 3], 1),
 ([4, 2], [3, 2, 1], 1),
 ([4, 2], [3, 2, 1], 2),
 ([4, 2], [3, 2, 1], 3),
 ([4, 2], [3, 1, 1, 1], 1),
 ([4, 2], [3, 1, 1, 1], 2),
 ([4, 2], [2, 2, 2], 1),
 ([4, 2], [2, 2, 1, 1], 1),
 ([4, 2], [2, 2, 1, 1], 2),
 ([4, 2], [2, 1, 1, 1, 1], 1)]

In [171]:
for i in range(len(polyCfs)):
    if polyCfs[i] != 1:
        for j in range(i+1,len(polyCfs)):
            if polyCfs[j] != 1:
                a = (polyCfs[i]/polyCfs[j]).simplify()
                if len(str(a)) < 20 and i != j:
                    print('i=' + str(i) + ', j=' + str(j))
                    print(str(a)[:20])

i=6, j=7
1
i=8, j=13
1


In [172]:
polyCfStrings = [ 'P_{' + str(vectorindex) + ',' + str(i+1) + '}' for i in range(len(polyCfs))]
pairs = [[6,7],[8,13]]
for i in range(len(pairs)):
    pairs[i].reverse()
pairs.sort()
for pair in pairs:
    polyCfStrings[pair[0]] = polyCfStrings[pair[1]]
    for i in range(pair[0]+1,len(polyCfStrings)):
        string = polyCfStrings[i]
        startindex = string.find(',')
        endindex = string.find('}')
        number = int(string[startindex+1:endindex]) - 1
        polyCfStrings[i] = 'P_{' + str(vectorindex) + ',' + str(number) + '}'
pairs.reverse()
for pair in pairs:
    polyCfs.pop(pair[0])

In [178]:
cfs = [latex(v) for v in vec]
sqrtlists = []
numberlists = []
#Square roots
for i in range(len(cfs)):
    cf = cfs[i]
    sqrts = []
    numbers = []
    if '\\sqrt' in cf:
        currentindex = cf.find('\\sqrt')
        sqrts = [currentindex] 
        while '\\sqrt' in cf[currentindex+1:]:
            currentindex += cf[currentindex+1:].find('\\sqrt') + 1
            sqrts.append(currentindex)
    
        for index in sqrts:
            endindex = cf[index:].find('}')
            numbers.append(Integer(cf[index+6:index+endindex]))
        prefix = cf[:sqrts[0]]
        suffix = cf[index+endindex+1:]
        prod = 1
        for n in numbers:
            prod *= n
        cfs[i] = prefix + '\\sqrt{' + str(prod) + '}' + suffix

#Signs
signs = ['-' if cf[0] == '-' else '+' for cf in cfs]
for i in range(len(cfs)):
    if signs[i] == '+':
        cfs[i] = '+ ' + cfs[i]
    else:
        cfs[i] = '- ' + cfs[i][1:]
if cfs[0][0] == '+':
    cfs[0] = cfs[0][2:]

In [179]:
symtracelabels = [([2, 1, 1, 1, 1],0), ([2, 2, 1, 1], 1), ([2, 2, 1, 1], 2), ([3, 1, 1, 1],0), ([2, 2, 2],0),
                  ([3, 2, 1], 1), ([3, 2, 1], 2), ([4, 1, 1],0), ([3, 3],0), ([4, 2],0)]
operatorlabels = [([5, 1],0), ([4, 2], 1), ([4, 2], 2), ([4, 1, 1], 1), ([4, 1, 1], 2), ([3, 3],0), ([3, 2, 1], 1),
                  ([3, 2, 1], 2), ([3, 2, 1], 3), ([3, 1, 1, 1], 1), ([3, 1, 1, 1], 2), ([2, 2, 2],0), ([2, 2, 1, 1], 1),
                  ([2, 2, 1, 1], 2), ([2, 1, 1, 1, 1],0)]
string = '\\begin{align}\n'
string += 'S^{BPS}_{\n'
string += str(symtracelabels[vectorindex][0]) + '\n'
if symtracelabels[vectorindex][1] > 0:
    string += ' , ' + str(symtracelabels[vectorindex][1]) + '\n'
string += '} & = \\frac{1}{' + latex(c2) + '\\sqrt{P^{BPS}_' + str(vectorindex) + '}} \left(\n'
for i in range(len(vec)):
    if i > 0:
        string += '\\nonumber \\\\\n'
        string += '& \\hspace{20pt}\n'
        if i == len(vec)-1:
            string += '\left.\n'
    string += cfs[i] + ' ' + '\n'# + polyCfStrings[i]
    string += '\\mathcal{O}_{\n'
    string += '\\ydiagram{' + str(operatorlabels[i][0])[1:-1] + '}\n'
    if operatorlabels[i][1] > 0:
        string += ' , ' + str(operatorlabels[i][1]) + '\n'
    string += '}\n'
    if i == 0:
        string += '\\right.\n'
    if i == len(vec)-1:
        string += '\\right)\n'
string += '\\end{align}\n'
string += '\\fontsize{12pt}{0}\n'
string += 'where the normalisation and coefficient polynomials are\n'
string += '\\fontsize{8pt}{0}\n'
string += '\\begin{align}\n'
string += 'P^{BPS}_{' + str(vectorindex) + '} & = ' + latex(P) + '\\\\\n'
for i in range(len(polyCfs)):
    string += 'P_{' + str(vectorindex) + ',' + str(i+1) + '} & = ' + latex(polyCfs[i]) + ' \\\\\n'
string = string[:-3]
string += '\n\\end{align}'

print(string)

\begin{align}
S^{BPS}_{
[2, 1, 1, 1, 1]
 } & = \frac{1}{ 18 \sqrt{P^{BPS}_ 0 }} \left(
 9 \, {\left(N - 1\right)} {\left(N - 2\right)} {\left(N - 3\right)} {\left(N - 4\right)} N 
 \mathcal{O}_{
 \ydiagram{5, 1}
 }
 \right.
 \nonumber \\
 & \hspace{20pt}
 + 0 
 \mathcal{O}_{
 \ydiagram{4, 2}
 , 1
 }
 \nonumber \\
 & \hspace{20pt}
 - 9 \, {\left(N + 4\right)} {\left(N - 1\right)} {\left(N - 2\right)} {\left(N - 3\right)} {\left(N - 4\right)} 
 \mathcal{O}_{
 \ydiagram{4, 2}
 , 2
 }
 \nonumber \\
 & \hspace{20pt}
 - 2 \, \sqrt{30} {\left(N + 4\right)} {\left(N - 1\right)} {\left(N - 3\right)} {\left(N - 4\right)} N 
 \mathcal{O}_{
 \ydiagram{4, 1, 1}
 , 1
 }
 \nonumber \\
 & \hspace{20pt}
 + 11 \, \sqrt{6} {\left(N + 4\right)} {\left(N - 1\right)} {\left(N - 3\right)} {\left(N - 4\right)} N 
 \mathcal{O}_{
 \ydiagram{4, 1, 1}
 , 2
 }
 \nonumber \\
 & \hspace{20pt}
 + 0 
 \mathcal{O}_{
 \ydiagram{3, 3}
 }
 \nonumber \\
 & \hspace{20pt}
 - 6 \, {\left(N + 4\right)} {\left(N + 3\right)} {\l

In [45]:
polyCfs

[78*N^4 + 180*N^3 - 411*N^2 - 510*N + 788,
 195*N^5 + 1149*N^4 + 687*N^3 - 3927*N^2 - 1552*N + 4448,
 195*N^5 + 1257*N^4 + 801*N^3 - 5871*N^2 - 3656*N + 9024,
 975*N^5 + 6177*N^4 + 3891*N^3 - 27411*N^2 - 16176*N + 40544,
 507*N^5 + 3225*N^4 + 2037*N^3 - 14487*N^2 - 8664*N + 21632,
 195*N^5 + 1041*N^4 + 573*N^3 - 1983*N^2 + 552*N - 128,
 1443*N^5 + 9129*N^4 + 5745*N^3 - 40335*N^2 - 23688*N + 59456,
 117*N^4 + 720*N^3 + 1041*N^2 + 240*N + 992,
 429*N^5 + 2247*N^4 + 1215*N^3 - 3585*N^2 + 2056*N - 2112,
 54*N^3 + 273*N^2 + 120*N - 572]

In [46]:
polyCfStrings

['P_{1,1}',
 'P_{1,2}',
 'P_{1,3}',
 'P_{1,4}',
 'P_{1,5}',
 'P_{1,2}',
 'P_{1,6}',
 'P_{1,2}',
 'P_{1,3}',
 'P_{1,4}',
 'P_{1,7}',
 'P_{1,8}',
 'P_{1,8}',
 'P_{1,9}',
 'P_{1,10}']

In [284]:
op1 = bpsbasis_fc42_lambda42[([4, 2], [2,2,1,1], 1)]
op2 = bpsbasis_fc42_lambda42[([4, 2], [2,2,1,1], 2)]
norm1 = calculator42.calculate(op1,op1,'F')
norm2 = calculator42.calculate(op2,op2,'F')
op1 /= sqrt(norm1)
op1.canonicalizeRadicalsInCoefficients()
op1.factorCoefficients()
op2 /= sqrt(norm2)
op2.canonicalizeRadicalsInCoefficients()
op2.factorCoefficients()



In [285]:
invariants = []
squareinvariants = []

In [286]:
c = 1
s = 0
op3 = c*op1 + s*op2
op4 = -s*op1 + c*op2

innerprodmatrix = [[0,0],[0,0]]
innerprodmatrix[0][0] = calculator42.calculate(op3,op3,'planar').canonicalize_radical().factor()
innerprodmatrix[0][1] = calculator42.calculate(op3,op4,'planar').canonicalize_radical().factor()
innerprodmatrix[1][0] = innerprodmatrix[0][1]
innerprodmatrix[1][1] = calculator42.calculate(op4,op4,'planar').canonicalize_radical().factor()

invariants.append((innerprodmatrix[0][0] + innerprodmatrix[1][1]).canonicalize_radical().factor())
squareinvariants.append((innerprodmatrix[0][0]^2 + 2*innerprodmatrix[0][1]^2 + innerprodmatrix[1][1]^2).canonicalize_radical().factor())

In [302]:
10035^2*195^2*2

7658328161250

In [301]:
squareinvariants

[(7658328161250*N^32 + 308379397920750*N^31 + 5678040590961075*N^30 + 62861883407800200*N^29 + 461553133569402069*N^28 + 2323880655128992368*N^27 + 7893896923770889320*N^26 + 16200841926037924512*N^25 + 9738474984510581700*N^24 - 43140893567922372492*N^23 - 100830809456338189482*N^22 + 66300678032545590264*N^21 + 576422366985618028290*N^20 + 587496624365125252152*N^19 - 1266939757691694906384*N^18 - 3370314414344723267400*N^17 - 14422779155617085790*N^16 + 8873284172309294711934*N^15 + 9228283693975324117807*N^14 - 8309143471774592802944*N^13 - 21871661389590847910159*N^12 - 3725069874701998817592*N^11 + 25451491117140266214976*N^10 + 18757146605106723110568*N^9 - 15395309506022451870416*N^8 - 22339442519546818907728*N^7 + 3985689055612424950064*N^6 + 16657691069689910952704*N^5 + 3604888800092578331072*N^4 - 4775351642112978422784*N^3 - 82696688563225374720*N^2 + 2740871464097166655488*N + 1006239182315089379328)/((10035*N^8 + 94914*N^7 + 264876*N^6 + 17268*N^5 - 819309*N^4 - 487830*N

In [27]:
lbl1 = ([4, 2], [3, 2, 1], 1)#([4, 2], [2, 2, 1, 1], 1)
lbl2 = ([4, 2], [3, 2, 1], 2)#([4, 2], [2, 2, 1, 1], 2)

msop1 = msbasis_fc42[lbl1]
msop2 = msbasis_fc42[lbl2]
stop1 = symtracebasis_fc42_lambda42[lbl1]
stop2 = symtracebasis_fc42_lambda42[lbl2]

for n in range(10):
    ops = [[msop1.copy(),msop2.copy()],[stop1.copy(),stop2.copy()]]
    for i in range(2):
        for j in range(2):
            for k in range(len(ops[i][j].coefficients)):
                ops[i][j].coefficients[k] = ops[i][j].coefficients[k](N=n)
    shortmsbasis = Basis([4,2],ops[0])
    shortstbasis = Basis([4,2],ops[1])
    cfs = [[shortmsbasis.expand(shortstbasis[0]),shortmsbasis.expand(shortstbasis[1])],
           [shortstbasis.expand(shortmsbasis[0]),shortstbasis.expand(shortmsbasis[1])]]
    print("N="+str(n))
    for i in range(2):
        for j in range(2):
            print(cfs[i][j])
    

N=0
(1693/60410607856227*sqrt(5998471637)*sqrt(123845753)*sqrt(5), 57112/60410607856227*sqrt(5998471637)*sqrt(186590563))
(-688/79819795197*sqrt(123845753)*sqrt(39628535), -1693/399098975985*sqrt(186590563)*sqrt(39628535)*sqrt(5))
(-1693/2239750443005*sqrt(5998471637)*sqrt(123845753)*sqrt(5), -57112/2239750443005*sqrt(123845753)*sqrt(39628535))
(688/674898066371*sqrt(5998471637)*sqrt(186590563), 1693/3374490331855*sqrt(186590563)*sqrt(39628535)*sqrt(5))
N=1
(3661/20164747239639*sqrt(866490605)*sqrt(495801609)*sqrt(5), 67168/20164747239639*sqrt(495801609)*sqrt(105204331))
(3578/125357489305*sqrt(866490605)*sqrt(5548019), -3661/125357489305*sqrt(105204331)*sqrt(5548019)*sqrt(5))
(32949/58930892536655*sqrt(866490605)*sqrt(495801609)*sqrt(5), 67168/11786178507331*sqrt(866490605)*sqrt(5548019))
(32202/7155051755641*sqrt(495801609)*sqrt(105204331), -18305/7155051755641*sqrt(105204331)*sqrt(5548019)*sqrt(5))
N=2
(773/6218746565675*sqrt(624229705)*sqrt(328824917)*sqrt(5), 101604/68406212222425

In [57]:
[3913650,78795855,656781957]

[3913650, 78795855, 656781957]

In [60]:
((3913650 * N^2 + 78795855 * N + 656781957)^2/2).expand()

7658328161250*N^4 + 308379397920750*N^3 + 11349616177207125/2*N^2 + 51751695850388235*N + 431362539040749849/2

In [61]:
5678040590961075*2

11356081181922150

In [55]:
3913650^2

15316656322500

In [56]:
7658328161250/15316656322500

1/2

In [19]:
bpsbasis_fc32.labels

[([5], [5], 1),
 ([5], [4, 1], 1),
 ([5], [3, 2], 1),
 ([5], [3, 1, 1], 1),
 ([5], [2, 2, 1], 1),
 ([5], [2, 1, 1, 1], 1),
 ([5], [1, 1, 1, 1, 1], 1),
 ([4, 1], [4, 1], 1),
 ([4, 1], [3, 2], 1),
 ([4, 1], [3, 1, 1], 1),
 ([4, 1], [2, 2, 1], 1),
 ([4, 1], [2, 1, 1, 1], 1),
 ([3, 2], [2, 1, 1, 1], 1),
 ([3, 2], [2, 2, 1], 1),
 ([3, 2], [3, 1, 1], 1),
 ([3, 2], [3, 2], 1)]

In [45]:
op = bpsbasis_fc32[ ([3, 2], [3,1,1], 1)]
op.canonicalizeRadicalsInCoefficients()
op.factorCoefficients()
for cf, tr in op:
    print(str(cf*sqrt(15*N^6 + 50*N^5 + 17*N^4 - 66*N^3 + 115*N^2 - 60*N + 20)*12/sqrt(3)/sqrt(2)) + str(tr))

1/12*sqrt(3)*sqrt(2)*(N^3 + 3*N^2 - 5*N + 2)(TrX)(TrXX)(TrY)(TrY)
-1/6*sqrt(3)*sqrt(2)*(N^3 + 3*N^2 - 5*N + 2)(TrX)(TrX)(TrXY)(TrY)
1/12*sqrt(3)*sqrt(2)*(N^3 + 3*N^2 - 5*N + 2)(TrX)(TrX)(TrX)(TrYY)
-1/3*sqrt(3)*sqrt(2)*(N + 1)*N^2(TrX)(TrXX)(TrYY)
1/3*sqrt(3)*sqrt(2)*(N + 1)*N^2(TrX)(TrXY)(TrXY)
1/6*sqrt(3)*sqrt(2)*(2*N^3 + 4*N^2 - 5*N + 2)(TrXXX)(TrY)(TrY)
-1/3*sqrt(3)*sqrt(2)*(2*N^3 + 4*N^2 - 5*N + 2)(TrX)(TrXXY)(TrY)
1/6*sqrt(3)*sqrt(2)*(2*N^3 + 4*N^2 - 5*N + 2)(TrX)(TrX)(TrXYY)
-1/6*sqrt(3)*sqrt(2)*(N + 1)*N^2(TrXXX)(TrYY)
1/3*sqrt(3)*sqrt(2)*(N + 1)*N^2(TrXXY)(TrXY)
-1/6*sqrt(3)*sqrt(2)*(N + 1)*N^2(TrXX)(TrXYY)
2/3*sqrt(3)*sqrt(2)*(N + 1)*(N - 1)(TrX)(TrXXYY)
-2/3*sqrt(3)*sqrt(2)*(N + 1)*(N - 1)(TrX)(TrXYXY)
sqrt(3)*sqrt(2)*(N + 1)*N(TrXXXYY)
-sqrt(3)*sqrt(2)*(N + 1)*N(TrXXYXY)


In [54]:
a = (bpsvectors[3]*sqrt(3*N^4 + 5*N^2 - 4*N + 2)*sqrt(5)*sqrt(3)*2)[12:]
for n in a:
    print(n)

10*N^2 - 10*N + 4
sqrt(5)*(4*N^2 + 5*N - 2)
4*sqrt(3)*(N - 1)
-3*sqrt(2)*N
sqrt(5)*(N + 2)
-2*N - 4


In [12]:
fieldContent = [4,2]
calculator = correlatorCalculator(fieldContent)
print('Calculator created')

fplabels, fpbasis = VectorGenerator(sum(fieldContent))
multiplicities = lambdaRmultiplicities(sum(fieldContent))
print('Basis labels and multiplicities are:')
print(multiplicities)
if fieldContent[1] == 0:
    fieldContent.pop()
fpbasis = [fpbasis[i] for i in range(len(fplabels)) if fplabels[i][0].evaluation() == fieldContent] 
fplabels = [lbl for lbl in fplabels if lbl[0].evaluation() == fieldContent]
if len(fieldContent) == 1:
    fieldContent.append(0)
partitions = list(Partitions(sum(fieldContent)))
lambdaPartitions = [ p for p in partitions if len(p) == 1 or (len(p) == 2 and p[1] <= fieldContent[1])]
fpvectors = []
for Lambda in lambdaPartitions:
    fpvectors.append([ fpbasis[i] for i in range(len(fplabels)) if fplabels[i][0].shape() == Lambda])
print('Setup lambda vectors')

G = SymmetricGroup(sum(fieldContent))
representations = [ SymmetricGroupRepresentation(p, 'orthogonal') for p in partitions]

Calculator created
Basis labels and multiplicities are:
[([6], [6], 1), ([6], [5, 1], 1), ([6], [4, 2], 1), ([6], [4, 1, 1], 1), ([6], [3, 3], 1), ([6], [3, 2, 1], 1), ([6], [3, 1, 1, 1], 1), ([6], [2, 2, 2], 1), ([6], [2, 2, 1, 1], 1), ([6], [2, 1, 1, 1, 1], 1), ([6], [1, 1, 1, 1, 1, 1], 1), ([5, 1], [5, 1], 1), ([5, 1], [4, 2], 1), ([5, 1], [4, 1, 1], 1), ([5, 1], [3, 2, 1], 2), ([5, 1], [3, 1, 1, 1], 1), ([5, 1], [2, 2, 1, 1], 1), ([5, 1], [2, 1, 1, 1, 1], 1), ([4, 2], [5, 1], 1), ([4, 2], [4, 2], 2), ([4, 2], [4, 1, 1], 2), ([4, 2], [3, 3], 1), ([4, 2], [3, 2, 1], 3), ([4, 2], [3, 1, 1, 1], 2), ([4, 2], [2, 2, 2], 1), ([4, 2], [2, 2, 1, 1], 2), ([4, 2], [2, 1, 1, 1, 1], 1), ([3, 3], [4, 1, 1], 1), ([3, 3], [3, 2, 1], 2), ([3, 3], [3, 1, 1, 1], 1)]
Setup lambda vectors


In [66]:
operators = []
labels = []
for Lambda, R, mult in multiplicities[11:]:
    print(Lambda, R)
    if Lambda in lambdaPartitions:
        lambdaindex = lambdaPartitions.index(Lambda)
        Rindex = partitions.index(R)
        rep = representations[Rindex]
        dim = rep(G[0]).dimensions()[0]
        fpvectorindex = len(fpvectors[lambdaindex]) - 1
        for multindex in range(mult):
            print(multindex)
            dummy = True
            while dummy:
                fpvector = fpvectors[lambdaindex][fpvectorindex]
                print('attempting with vector index ' + str(fpvectorindex))
                randomcoefficients = [ ZZ.random_element(1,5) for i in range(dim) ]
                coefficients = []
                permutations = []
                for cf, fp in fpvector:
                    h = permFromFieldPositions(fp)
                    for g in G:
                        matrix = rep(g)
                        coefficients.append(cf * sum([matrix[0,i] * randomcoefficients[i] for i in range(dim)]))
                        permutations.append(h * g * h)
                op = Operator(fieldContent,[],[]) + Operator(fieldContent, coefficients, permutations=permutations)
                op.canonicalizeRadicalsInCoefficients()
                op.expandCoefficients()
                dummy = (len(op) == 0)
                fpvectorindex -= 1
            operators.append(op)
            labels.append((Lambda, R, multindex+1))
        print('')
#basis33 = Basis(fieldContent, operators=operators, labels=labels)

([5, 1], [5, 1])
0
attempting with vector index 4

([5, 1], [4, 2])
0
attempting with vector index 4

([5, 1], [4, 1, 1])
0
attempting with vector index 4

([5, 1], [3, 2, 1])
0
attempting with vector index 4
1
attempting with vector index 3

([5, 1], [3, 1, 1, 1])
0
attempting with vector index 4

([5, 1], [2, 2, 1, 1])
0
attempting with vector index 4

([5, 1], [2, 1, 1, 1, 1])
0
attempting with vector index 4

([4, 2], [5, 1])
0
attempting with vector index 8
attempting with vector index 7
attempting with vector index 6


KeyboardInterrupt: 

In [218]:
sqrt(calculator42.calculate(op1, op2,'planar'))


0

In [215]:
op2 /= 3/4*sqrt(15)*abs(2*sqrt(5)*sqrt(3) - 3*sqrt(5) + 6*sqrt(3) + 39)
op2.canonicalizeRadicalsInCoefficients()

In [209]:
op1 = gl2basis_fc42_lambda51[3] + changeSign(gl2basis_fc42_lambda51[3])
op2 = gl2basis_fc42_lambda51[3] - changeSign(gl2basis_fc42_lambda51[3])

In [222]:
gl2basis_fc42_lambda51.operators[3] = op1
gl2basis_fc42_lambda51.operators[4] = op2

In [207]:
calculator42.calculate(gl2basis_fc42_lambda51,gl2basis_fc42_lambda51,'planar')

KeyboardInterrupt: 

In [225]:
for op in gl2basis_fc42_lambda51:
    print(calculator42.calculate(op, op, 'F'))

(N + 4)*(N + 3)*(N + 2)*(N + 1)*(N - 1)*N
(N + 3)*(N + 2)*(N + 1)*(N - 1)*N^2
(N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*N
(N + 2)*(N + 1)*(N - 1)*(N - 2)*N^2
(N + 2)*(N + 1)*(N - 1)*(N - 2)*N^2
(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N
(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2
(N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*N


In [748]:
comm33_1 = basis33[2]
comm33_2 = sqrt(3)/3 * (basis33[0] + basis33[3] - basis33[1])

In [730]:
st33_1 = sqrt(3)*sqrt(2)/6 * (basis33[0] - 2 * basis33[3] - basis33[1])
st33_2 = 1/sqrt(N^4 + 3*N^3 - 9*N + 9) * ( sqrt(2)/2 * (N+3)*(N-1)*basis33[0] 
                                          + sqrt(2)/2 *N*(N+1)*basis33[1] - sqrt(2)/2 * (N-3)*basis33[3] )
st33_2.factorCoefficients()

In [745]:
print(calculator33.calculate(st33_1,st33_1))
print(calculator33.calculate(st33_2,st33_2))

1/2*(2*N^2 + 3*N - 3)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)
1/2*(2*N^2 + 3*N - 3)/((N^4 + 3*N^3 - 9*N + 9)*(N + 2)*(N + 1)*(N - 1)*(N - 2))


In [755]:
bps33_1 = 1/sqrt(6*N^4 + 18*N^3 + 27*N^2 + 81)*( N*(N-3)*basis33[0] - (N-3)*(N+3)*basis33[1] - 2*N*(N+3)*basis33[3])
bps33_2 = sqrt(2)/sqrt(2*N^2+3) * ( sqrt(2)/2*(N-1)*basis33[0] + sqrt(2)/2*(N+1)*basis33[1] - sqrt(2)/2*basis33[3] )

In [759]:
print(calculator33.calculate(bps33_1,bps33_1,'F'))
print(calculator33.calculate(bps33_2,bps33_2,'F'))

(2*N^2 + 3*N - 3)*(N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2/(2*N^4 + 6*N^3 + 9*N^2 + 27)
(2*N^2 + 3*N - 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*N^2/(2*N^2 + 3)


In [819]:
for i in range(15):
    print(loweredbasis33[i] - newbasis[i])

In [223]:
basis = gl2basis_fc42_lambda51

tracelist = [ Trace([4,2], [1,2,3,4,5,6]), Trace([4,2],[(1,2)]), Trace([4,2],[(1,5)]), Trace([4,2],[(5,6)]),
            Trace([4,2],[(1,2),(3,4)]), Trace([4,2],[(1,2),(3,5)]), Trace([4,2],[(1,5),(2,6)]), 
            Trace([4,2],[(1,2),(5,6)]), Trace([4,2],[(1,2),(3,4),(5,6)]), Trace([4,2],[(1,2),(3,5),(4,6)]),
            Trace([4,2],[(1,2,3)]), Trace([4,2],[(1,2,5)]), Trace([4,2],[(1,5,6)]), 
            Trace([4,2],[(1,2,3),(4,5)]), Trace([4,2],[(1,2,3),(5,6)]), Trace([4,2],[(1,2,5),(3,4)]),
            Trace([4,2],[(1,2,5),(3,6)]), Trace([4,2],[(1,5,6),(2,3)]), Trace([4,2],[(1,2,3),(4,5,6)]),
            Trace([4,2],[(1,2,5),(3,4,6)]), Trace([4,2],[(1,2,3,4)]), Trace([4,2],[(1,2,3,5)]),
            Trace([4,2],[(1,2,5,6)]), Trace([4,2],[(1,5,2,6)]), Trace([4,2],[(1,2,3,4),(5,6)]),
            Trace([4,2],[(1,2,3,5),(4,6)]), Trace([4,2],[(1,2,5,6),(3,4)]), Trace([4,2],[(1,5,2,6),(3,4)]),
            Trace([4,2],[(1,2,3,4,5)]), Trace([4,2],[(1,2,3,5,6)]), Trace([4,2],[(1,2,5,3,6)]),
            Trace([4,2],[(1,2,3,4,5,6)]), Trace([4,2],[(1,2,3,5,4,6)]), Trace([4,2],[(1,2,5,3,4,6)])]
print(basis.labels)

coefficientMatrix = []

for index in range(len(basis)):
    op = basis[index]

    coefficients = [0 for tr in tracelist]
    for cf, tr in op:
        coefficients[tracelist.index(tr)] = cf
    coefficientMatrix.append(coefficients)
print(coefficientMatrix)

[([5, 1], [5, 1], 1), ([5, 1], [4, 2], 1), ([5, 1], [4, 1, 1], 1), ([5, 1], [3, 2, 1], 1), ([5, 1], [3, 2, 1], 2), ([5, 1], [3, 1, 1, 1], 1), ([5, 1], [2, 2, 1, 1], 1), ([5, 1], [2, 1, 1, 1, 1], 1)]
[[0, -1/48*sqrt(3)*sqrt(2), 1/72*sqrt(3)*sqrt(2), 1/144*sqrt(3)*sqrt(2), -1/48*sqrt(3)*sqrt(2), -1/24*sqrt(3)*sqrt(2), 1/24*sqrt(3)*sqrt(2), 1/48*sqrt(3)*sqrt(2), 0, 0, -1/24*sqrt(3)*sqrt(2), 0, 1/24*sqrt(3)*sqrt(2), -1/18*sqrt(3)*sqrt(2), 1/72*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 1/24*sqrt(3)*sqrt(2), 0, 0, -1/24*sqrt(3)*sqrt(2), -1/12*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 1/24*sqrt(3)*sqrt(2), 0, 0, 0, 0, -1/6*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 1/12*sqrt(3)*sqrt(2), 0, 0, 0], [0, -1/80*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), 1/240*sqrt(5)*sqrt(3)*sqrt(2), 1/240*sqrt(5)*sqrt(3)*sqrt(2), 1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/120*sqrt(5)*sqrt(3)*sqrt(2), -1/240*sqrt(5)*sqrt(3)*sqrt(2), 0, 0, -1/40*sqrt(5)*sqrt(3)*sqrt(2), 0, 1/40*sqrt(5)*sqrt(3)*sqrt

In [148]:
op1 = orthogonalisedBasis42[14]
op2 = orthogonalisedBasis42[15]
op3 = gl2basis_fc42_lambda51[3]
op4 = gl2basis_fc42_lambda51[4]
tracelist = [ Trace([4,2], [1,2,3,4,5,6]), Trace([4,2],[(1,2)]), Trace([4,2],[(1,5)]), Trace([4,2],[(5,6)]),
            Trace([4,2],[(1,2),(3,4)]), Trace([4,2],[(1,2),(3,5)]), Trace([4,2],[(1,5),(2,6)]), 
            Trace([4,2],[(1,2),(5,6)]), Trace([4,2],[(1,2),(3,4),(5,6)]), Trace([4,2],[(1,2),(3,5),(4,6)]),
            Trace([4,2],[(1,2,3)]), Trace([4,2],[(1,2,5)]), Trace([4,2],[(1,5,6)]), 
            Trace([4,2],[(1,2,3),(4,5)]), Trace([4,2],[(1,2,3),(5,6)]), Trace([4,2],[(1,2,5),(3,4)]),
            Trace([4,2],[(1,2,5),(3,6)]), Trace([4,2],[(1,5,6),(2,3)]), Trace([4,2],[(1,2,3),(4,5,6)]),
            Trace([4,2],[(1,2,5),(3,4,6)]), Trace([4,2],[(1,2,3,4)]), Trace([4,2],[(1,2,3,5)]),
            Trace([4,2],[(1,2,5,6)]), Trace([4,2],[(1,5,2,6)]), Trace([4,2],[(1,2,3,4),(5,6)]),
            Trace([4,2],[(1,2,3,5),(4,6)]), Trace([4,2],[(1,2,5,6),(3,4)]), Trace([4,2],[(1,5,2,6),(3,4)]),
            Trace([4,2],[(1,2,3,4,5)]), Trace([4,2],[(1,2,3,5,6)]), Trace([4,2],[(1,2,5,3,6)]),
            Trace([4,2],[(1,2,3,4,5,6)]), Trace([4,2],[(1,2,3,5,4,6)]), Trace([4,2],[(1,2,5,3,4,6)])]

coefficients1 = [0 for tr in tracelist]
coefficients2 = [0 for tr in tracelist]
coefficients3 = [0 for tr in tracelist]
coefficients4 = [0 for tr in tracelist]
for cf, tr in op1:
    coefficients1[tracelist.index(tr)] = cf#.canonicalize_radical().expand()
for cf, tr in op2:
    coefficients2[tracelist.index(tr)] = cf.canonicalize_radical().expand()
for cf, tr in op3:
    coefficients3[tracelist.index(tr)] = cf.canonicalize_radical().expand()
for cf, tr in op4:
    coefficients4[tracelist.index(tr)] = cf.canonicalize_radical().expand()    
for i in range(len(coefficients1)):
    s1 = str(coefficients1[i]*1560/sqrt(13)/sqrt(3)/sqrt(2))
    s2 = str(coefficients2[i])
    s3 = str(coefficients3[i])
    s4 = str(coefficients4[i])
    print(s1 + (' ' * (30-len(s1))) + s2 + (' ' * (20-len(s2))) + s3 + (' ' * (25-len(s3))) + s4)

0                             0                   0                        0
16/59*sqrt(59)*sqrt(13)       3/590*sqrt(59)      0                        -1/30*sqrt(5)*sqrt(3)
-32/177*sqrt(59)*sqrt(13)     -1/295*sqrt(59)     0                        1/45*sqrt(5)*sqrt(3)
-16/177*sqrt(59)*sqrt(13)     -1/590*sqrt(59)     0                        1/90*sqrt(5)*sqrt(3)
12/59*sqrt(59)*sqrt(13)       -4/885*sqrt(59)     -1/30*sqrt(5)*sqrt(2)    0
24/59*sqrt(59)*sqrt(13)       -8/885*sqrt(59)     -1/15*sqrt(5)*sqrt(2)    0
-24/59*sqrt(59)*sqrt(13)      8/885*sqrt(59)      1/15*sqrt(5)*sqrt(2)     0
-12/59*sqrt(59)*sqrt(13)      4/885*sqrt(59)      1/30*sqrt(5)*sqrt(2)     0
0                             0                   0                        0
0                             0                   0                        0
-12/59*sqrt(59)*sqrt(13)      4/885*sqrt(59)      1/30*sqrt(5)*sqrt(2)     0
0                             0                   0                        0
12/59*sqrt(59)*sqr

In [104]:
orthogonalisedBasis42 = basis42.copy()

In [159]:
orthogonalisedBasis42.labels[19:29]

[([4, 2], [4, 2], 1),
 ([4, 2], [4, 1, 1], 1),
 ([4, 2], [3, 3], 1),
 ([4, 2], [3, 2, 1], 1),
 ([4, 2], [3, 2, 1], 2),
 ([4, 2], [3, 1, 1, 1], 1),
 ([4, 2], [2, 2, 2], 1),
 ([4, 2], [2, 2, 1, 1], 1),
 ([4, 2], [2, 2, 1, 1], 2),
 ([4, 2], [2, 1, 1, 1, 1], 1)]

In [167]:
index=26
op = orthogonalisedBasis42[index]
norm = calculator42.calculate(op,op)
for i in range(19,index):
    op1 = orthogonalisedBasis42[i]
    orthogonalisedBasis42.operators[i] -= calculator42.calculate(op,op1)/norm * op

In [178]:
op = orthogonalisedBasis42[25]
print(calculator42.calculate(op,op,'planar').canonicalize_radical().factor())
print(calculator42.calculate(op,op))

3/5*(149226300*N^16 + 2574045540*N^15 + 17126314218*N^14 + 48780132444*N^13 + 10649800683*N^12 - 260799105402*N^11 - 414404753235*N^10 + 509352006396*N^9 + 1458046648509*N^8 - 677071004478*N^7 - 2841929143455*N^6 + 1359603561900*N^5 + 4498710198068*N^4 - 813280519104*N^3 - 2891933192736*N^2 + 800771042304*N + 1167006642048)/(18630*N^8 + 160677*N^7 + 371643*N^6 - 204495*N^5 - 1326729*N^4 - 15804*N^3 + 1726178*N^2 - 442368*N - 1298232)^2
3/5*(8010*N^7 + 56214*N^6 + 79800*N^5 - 132315*N^4 - 158273*N^3 + 296994*N^2 + 33500*N - 171336)/((18630*N^8 + 160677*N^7 + 371643*N^6 - 204495*N^5 - 1326729*N^4 - 15804*N^3 + 1726178*N^2 - 442368*N - 1298232)*(N + 1)*(N - 1)*(N - 2)*N^2)


In [177]:
op = orthogonalisedBasis42[24]
norm = calculator42.calculate(op,op)
op1 = orthogonalisedBasis42[25]
orthogonalisedBasis42.operators[25] -= calculator42.calculate(op,op1)/norm * op

In [173]:
for op in orthogonalisedBasis42:
    op.canonicalizeRadicalsInCoefficients()
    op.factorCoefficients()

In [19]:
bpsbasis_fc32.labels[12:]

[([3, 2], 4, 1), ([3, 2], 3, 1), ([3, 2], 3, 2), ([3, 2], 2, 1)]

In [18]:
msbasis_fc32.labels[12:]

[[[3, 2], [2, 1, 1, 1], 1],
 [[3, 2], [2, 2, 1], 1],
 [[3, 2], [3, 1, 1], 1],
 [[3, 2], [3, 2], 1]]

In [48]:
calculator32.calculate(symtracebasis_fc32[12],symtracebasis_fc32[12],'planar').expand().simplify_rational()

1

In [57]:
op = symtracebasis_fc32[([3, 2], 3, 2)] + msbasis_fc32[([3, 2], [3,1,1], 1)]
op.expandCoefficients()
op.canonicalizeRadicalsInCoefficients()
op.factorCoefficients()
print(op)

In [50]:
op1 = symtracebasis_fc32[([3, 2], 4, 1)]
op2 = msbasis_fc32[([3, 2], [2, 1, 1, 1], 1)]

In [51]:
print(op1.traces[0])
print(op2.traces[6])
c = 1
op1 *= c
op1.canonicalizeRadicalsInCoefficients()
op1.factorCoefficients()
op1 /= c
print(op1.coefficients[0])
print(op2.coefficients[6])

(TrX)(TrXX)(TrY)(TrY)
(TrX)(TrXX)(TrY)(TrY)
-1/60*sqrt(5)*sqrt(3)
1/60*sqrt(5)*sqrt(3)


In [69]:
cfs = gl2basis_fc22.expand(bps22_2)
for i in range(len(cfs)):
    cfs[i] = cfs[i].canonicalize_radical()
    cfs[i] = cfs[i].factor()
norm = gcd(cfs)
cfs = [cf/norm for cf in cfs ]
print(norm)
print(cfs)

1/sqrt(3*N^2 + 2)
[0, 0, 0, 0, 0, 0, 0, -1/2*sqrt(2)*(2*N - 1), N + 1, -1/2*sqrt(2)]


In [144]:
fieldContent = [2,2]
totalFieldContent = sum(fieldContent)
partitionList = list(Partitions(sum(fieldContent)))

operators = []
labels = []
for ycontent in range(fieldContent[1]+1):
    #print('Y content: ' + str(ycontent))
    currentFieldContent = [totalFieldContent-ycontent,ycontent]
    calculator = correlatorCalculator(currentFieldContent)
    #print('Calculator created')
    for i in range(len(operators)):
        operators[i] = lowerOperator(operators[i])
        operators[i] /= sqrt(calculator.calculate(operators[i],operators[i],'planar'))
        operators[i].canonicalizeRadicalsInCoefficients()
    #print('Previous operators lowered')
    basis = multiSymmetricBasis(currentFieldContent)
    #print(basis)
    for p in partitionList:
        #print('Looking at ' + str(p) + ' sector')
        basisIndices = []
        for i in range(len(basis)):
            basisPartition = list(np.sum(np.array(basis.labels[i]), axis=1))
            basisPartition.sort(reverse=True)
            if p == basisPartition:
                basisIndices.append(i)
        #print(basisIndices)
        indices = []
        for i in range(len(labels)):
            if p == labels[i][1]:
                indices.append(i)
        #print(indices)
        multiplicity = len(basisIndices) - len(indices)
        #print(multiplicity)
        testIndex = 0
        for i in range(multiplicity):
            cont = True
            while cont:
                op = basis[basisIndices[testIndex]]
                for j in indices:
                    op -= calculator.calculate(operators[j], op, 'planar') * operators[j]
                op.canonicalizeRadicalsInCoefficients()
                cont =  len(op.coefficients) == 0
                testIndex += 1
            op /= sqrt(calculator.calculate(op, op, 'planar'))
            op.canonicalizeRadicalsInCoefficients()
            operators.append(op)
            labels.append((currentFieldContent,p,i+1))
            indices.append(len(operators)-1)

basis22 = Basis(fieldContent, operators, labels)

In [145]:
basis = basis22
operatorSymbol = 'M'
labelType = ['semi-standard tableau', 'partition', 'multiplicity']

output = '\\ytableausetup{boxsize=7pt}\n'
output += '\\begin{align*}\n'
for lbl, op in basis:
    output += operatorSymbol + '_{\n'
    for i in range(len(labelType)):
        if labelType[i] == 'semi-standard tableau':
            content = [1 if j < basis.fieldContent[0] else 2 for j in range(sum(basis.fieldContent))]
            output += '\\fontsize{6pt}{0} \\begin{ytableau}\n'
            for j in range(lbl[0][0]):
                output += str(content[j]) + ' & '
            output = output[:-2]
            if lbl[0][1] > 0:
                output += '\\\\ '
                for j in range(lbl[0][1]):
                    output += str(content[j + lbl[0][0]]) + ' & '
                output = output[:-2]
            output += '\n\\end{ytableau} \\fontsize{12pt}{0} ,\n'
        if labelType[i] == 'partition':
            output += str(lbl[i])
            output += ' ,\n'
        if labelType[i] == 'multiplicity':
            output += str(lbl[i])
            output += ' ,\n'
    output = output[:-2]
    output += '} & =\n'
    cf = gcd(op.coefficients)
    if cf != 1:
        output += latex(cf) + ' \\left[\n'
        output += latexString(1/cf * op) + '\\right] \\\\\n'
    else:
        output += latexString(op) + '\\\\\n'
        
    
output += '\\end{align*}\n'
output += '\\ytableausetup{boxsize=normal}'
print(output)

\ytableausetup{boxsize=7pt}
\begin{align*}
M_{
\fontsize{6pt}{0} \begin{ytableau}
1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[4] ,
1 } & =
 \frac{1}{6} \, \sqrt{3} \sqrt{2} \left[
 2 \tr X^2 Y^2 
 + \tr \left( X Y \right)^2 
\right] \\
 M_{
 \fontsize{6pt}{0} \begin{ytableau}
 1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[3, 1] ,
1 } & =
 \frac{1}{42} \, \sqrt{7} \sqrt{3} \sqrt{2} \left[
 - 4 \tr X^2 Y^2 
 - 2 \tr \left( X Y \right)^2 
 + 3 \tr X^2 Y \tr Y 
 + 3 \tr X \tr X Y^2 
 \right] \\
 M_{
 \fontsize{6pt}{0} \begin{ytableau}
 1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[2, 2] ,
1 } & =
 \frac{1}{6} \, \sqrt{2} \left[
 - 2 \tr X^2 Y^2 
 - \tr \left( X Y \right)^2 
 + 2 \left( \tr X Y \right)^2 
 + \tr X^2 \tr Y^2 
 \right] \\
 M_{
 \fontsize{6pt}{0} \begin{ytableau}
 1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[2, 1, 1] ,
1 } & =
 \frac{1}{60} \, \sqrt{5} \sqrt{3} \left[
 8 \tr X^2 Y^2 
 + 4 \tr \left( X Y \right)^2 
 - 6 \tr X^2 Y \tr Y 
 - 6 \tr X \tr

In [3]:
fieldContent=[2,2]
basis = GL2multiSymmetricBasis(fieldContent)
totalfieldContent = sum(fieldContent)
calculator = correlatorCalculator(fieldContent)

In [4]:
fieldContent=[2,2]
basis = GL2multiSymmetricBasis(fieldContent)
totalfieldContent = sum(fieldContent)
calculator = correlatorCalculator(fieldContent)
operators = []
labels = []
for lambda1 in range(fieldContent[1]+1):
    Lambda = [totalfieldContent-lambda1, lambda1]
    operatorsByLambda = []
    labelsByLambda = []
    print(Lambda)
    print('')
    indicesByPartition = {}
    for i in range(len(basis)):
        if basis.labels[i][0] == Lambda:
            if basis.labels[i][1] in indicesByPartition:
                indicesByPartition[basis.labels[i][1]].append(i)
            else: 
                indicesByPartition.update({basis.labels[i][1]:[i]})
    partitions = indicesByPartition.keys()
    partitions.sort(key=transposePartition,reverse=True)
    print(partitions)
    for p in partitions:
        print('partition: ' + str(p))
        for mult in range(len(indicesByPartition[p])):
            print('multiplicity: ' + str(mult+1))
            index = indicesByPartition[p][mult]
            print('basis index: ' + str(index))
            op = basis[index]
            print('before orthogonalising')
            print(op)
            for i in range(len(operatorsByLambda)):
                op1 = operatorsByLambda[i]
                print('Orthogonalising against: ' + str(labelsByLambda[i]))
                op -= calculator.calculate(op,op1,'G')/calculator.calculate(op1,op1,'G') * op1
            op.expandCoefficients()
            op.canonicalizeRadicalsInCoefficients()
            op.factorCoefficients()
            op /= sqrt(calculator.calculate(op,op,'planar'))
            op.expandCoefficients()
            op.canonicalizeRadicalsInCoefficients()
            op.factorCoefficients()
            operatorsByLambda.append(op)
            labelsByLambda.append([Lambda,p,mult+1])
            print('after orthogonalising')
            print(op)
    operators += operatorsByLambda
    labels += labelsByLambda
    print('')

finalBasis = Basis(fieldContent, operators, labels)
    

[4, 0]

[[1, 1, 1, 1], [2, 1, 1], [2, 2], [3, 1], [4]]
partition: [1, 1, 1, 1]
multiplicity: 1
basis index: 4
before orthogonalising
(-1/6*sqrt(3)*sqrt(2))(TrXXYY) + (-1/12*sqrt(3)*sqrt(2))(TrXYXY) + (1/6*sqrt(3)*sqrt(2))(TrXXY)(TrY) + (1/6*sqrt(3)*sqrt(2))(TrX)(TrXYY) + (1/12*sqrt(3)*sqrt(2))(TrXY)(TrXY) + (1/24*sqrt(3)*sqrt(2))(TrXX)(TrYY) + (-1/24*sqrt(3)*sqrt(2))(TrXX)(TrY)(TrY) + (-1/6*sqrt(3)*sqrt(2))(TrX)(TrXY)(TrY) + (-1/24*sqrt(3)*sqrt(2))(TrX)(TrX)(TrYY) + (1/24*sqrt(3)*sqrt(2))(TrX)(TrX)(TrY)(TrY)
after orthogonalising
(-1/6*sqrt(3)*sqrt(2))(TrXXYY) + (-1/12*sqrt(3)*sqrt(2))(TrXYXY) + (1/6*sqrt(3)*sqrt(2))(TrXXY)(TrY) + (1/6*sqrt(3)*sqrt(2))(TrX)(TrXYY) + (1/12*sqrt(3)*sqrt(2))(TrXY)(TrXY) + (1/24*sqrt(3)*sqrt(2))(TrXX)(TrYY) + (-1/24*sqrt(3)*sqrt(2))(TrXX)(TrY)(TrY) + (-1/6*sqrt(3)*sqrt(2))(TrX)(TrXY)(TrY) + (-1/24*sqrt(3)*sqrt(2))(TrX)(TrX)(TrYY) + (1/24*sqrt(3)*sqrt(2))(TrX)(TrX)(TrY)(TrY)
partition: [2, 1, 1]
multiplicity: 1
basis index: 3
before orthogonalising
(2/15*sq

In [11]:
#Fully orthogonalised basis for field content (2,2)
latexString(finalBasis,'O',['semi-standard tableau','partition','multiplicity'])

\ytableausetup{boxsize=7pt}
\begin{align*}
O_{
\fontsize{6pt}{0} \begin{ytableau}
1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[1, 1, 1, 1] ,
1 } & =
 \frac{1}{24} \, \sqrt{3} \sqrt{2} \left[
 - 4 \tr X^2 Y^2 
 - 2 \tr \left( X Y \right)^2 
 + 4 \tr X^2 Y \tr Y 
 + 4 \tr X \tr X Y^2 
 + 2 \left( \tr X Y \right)^2 
 + \tr X^2 \tr Y^2 
 - \tr X^2 \left( \tr Y \right)^2 
 - 4 \tr X \tr X Y \tr Y 
 - \left( \tr X \right)^2 \tr Y^2 
 + \left( \tr X \right)^2 \left( \tr Y \right)^2 
 \right] \\
 O_{
 \fontsize{6pt}{0} \begin{ytableau}
 1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[2, 1, 1] ,
1 } & =
 \frac{1}{24} \, \sqrt{3} \sqrt{2} \left[
 4 \tr X^2 Y^2 
 + 2 \tr \left( X Y \right)^2 
 - 2 \left( \tr X Y \right)^2 
 - \tr X^2 \tr Y^2 
 - \tr X^2 \left( \tr Y \right)^2 
 - 4 \tr X \tr X Y \tr Y 
 - \left( \tr X \right)^2 \tr Y^2 
 + 3 \left( \tr X \right)^2 \left( \tr Y \right)^2 
\right] \\
 O_{
 \fontsize{6pt}{0} \begin{ytableau}
 1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{

In [28]:
#Checking [n-1,1] orthogonalisation for field content (2,2)
n=4
for lbl, op in finalBasis:
    if lbl[0] == [n-1,1]:
        p = lbl[1]
        norm = 1
        for i in range(len(p)):
            for j in range(p[i]):
                norm *= 1/(N-i+j)
        calculatedNorm = calculator.calculate(op,op,'G')
        matching = [Rational(norm(N=n+i)) == Rational(calculatedNorm(N=n+i)) for i in range(n+1)]
            
        print(all(matching),lbl,norm,calculatedNorm )


(True, [[3, 1], [2, 1, 1], 1], 1/((N + 1)*(N - 1)*(N - 2)*N), 1/((N + 1)*(N - 1)*(N - 2)*N))
(True, [[3, 1], [3, 1], 1], 1/((N + 2)*(N + 1)*(N - 1)*N), 1/((N + 2)*(N + 1)*(N - 1)*N))


In [31]:
#Fully orthogonalised basis for field content (3,2)
latexString(finalBasis,'O',['semi-standard tableau','partition','multiplicity'])

\ytableausetup{boxsize=7pt}
\begin{align*}
O_{
\fontsize{6pt}{0} \begin{ytableau}
1 & 1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[1, 1, 1, 1, 1] ,
1 } & =
 \frac{1}{120} \, \sqrt{5} \sqrt{2} \left[
 12 \tr X^3 Y^2 
 + 12 \tr X^2 Y X Y 
 - 12 \tr X^3 Y \tr Y 
 - 12 \tr X \tr X^2 Y^2 
 - 6 \tr X \tr \left( X Y \right)^2 
 - 12 \tr X^2 Y \tr X Y 
 - 2 \tr X^3 \tr Y^2 
 - 6 \tr X^2 \tr X Y^2 
 + 2 \tr X^3 \left( \tr Y \right)^2 
 + 12 \tr X \tr X^2 Y \tr Y 
 + 6 \left( \tr X \right)^2 \tr X Y^2 
 + 6 \tr X^2 \tr X Y \tr Y 
 + 6 \tr X \left( \tr X Y \right)^2 
 + 3 \tr X \tr X^2 \tr Y^2 
 - 3 \tr X \tr X^2 \left( \tr Y \right)^2 
 - 6 \left( \tr X \right)^2 \tr X Y \tr Y 
 - \left( \tr X \right)^3 \tr Y^2 
 + \left( \tr X \right)^3 \left( \tr Y \right)^2 
\right] \\
 O_{
 \fontsize{6pt}{0} \begin{ytableau}
 1 & 1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[2, 1, 1, 1] ,
1 } & =
 \frac{1}{60} \, \sqrt{5} \sqrt{2} \left[
 - 6 \tr X^3 Y^2 
 - 6 \tr X^2 Y X Y 
 + 6 \tr X^2 Y \tr X

In [32]:
#Checking [n-1,1] orthogonalisation for field content (3,2)
n=5
for lbl, op in finalBasis:
    if lbl[0] == [n-1,1]:
        p = lbl[1]
        norm = 1
        for i in range(len(p)):
            for j in range(p[i]):
                norm *= 1/(N-i+j)
        calculatedNorm = calculator.calculate(op,op,'G')
        matching = [Rational(norm(N=n+i)) == Rational(calculatedNorm(N=n+i)) for i in range(n+1)]
            
        print(all(matching),lbl,norm,calculatedNorm )

(True, [[4, 1], [2, 1, 1, 1], 1], 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*N), 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*N))
(True, [[4, 1], [2, 2, 1], 1], 1/((N + 1)*(N - 1)*(N - 2)*N^2), 1/((N + 1)*(N - 1)*(N - 2)*N^2))
(True, [[4, 1], [3, 1, 1], 1], 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*N), 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*N))
(True, [[4, 1], [3, 2], 1], 1/((N + 2)*(N + 1)*(N - 1)*N^2), 1/((N + 2)*(N + 1)*(N - 1)*N^2))
(True, [[4, 1], [4, 1], 1], 1/((N + 3)*(N + 2)*(N + 1)*(N - 1)*N), 1/((N + 3)*(N + 2)*(N + 1)*(N - 1)*N))


In [35]:
#Fully orthogonalised basis for field content (4,2)
latexString(finalBasis,'O',['semi-standard tableau','partition','multiplicity'])

\ytableausetup{boxsize=7pt}
\begin{align*}
O_{
\fontsize{6pt}{0} \begin{ytableau}
1 & 1 & 1 & 1 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[1, 1, 1, 1, 1, 1] ,
1 } & =
 \frac{1}{720} \, \sqrt{5} \sqrt{3} \left[
 - 48 \tr X^4 Y^2 
 - 48 \tr X^3 Y X Y 
 - 24 \tr \left( X^2 Y \right)^2 
 + 48 \tr X^4 Y \tr Y 
 + 48 \tr X \tr X^3 Y^2 
 + 48 \tr X \tr X^2 Y X Y 
 + 48 \tr X^3 Y \tr X Y 
 + 6 \tr X^4 \tr Y^2 
 + 24 \tr X^2 \tr X^2 Y^2 
 + 12 \tr X^2 \tr \left( X Y \right)^2 
 + 24 \left( \tr X^2 Y \right)^2 
 + 16 \tr X^3 \tr X Y^2 
 - 6 \tr X^4 \left( \tr Y \right)^2 
 - 48 \tr X \tr X^3 Y \tr Y 
 - 24 \left( \tr X \right)^2 \tr X^2 Y^2 
 - 12 \left( \tr X \right)^2 \tr \left( X Y \right)^2 
 - 16 \tr X^3 \tr X Y \tr Y 
 - 24 \tr X^2 \tr X^2 Y \tr Y 
 - 48 \tr X \tr X^2 Y \tr X Y 
 - 8 \tr X \tr X^3 \tr Y^2 
 - 24 \tr X \tr X^2 \tr X Y^2 
 - 12 \tr X^2 \left( \tr X Y \right)^2 
 - 3 \left( \tr X^2 \right)^2 \tr Y^2 
 + 8 \tr X \tr X^3 \left( \tr Y \right)^2 
 + 24 \left( \tr X \right)^2 \

In [39]:
#Fully orthogonalised basis for field content (3,3)
latexString(finalBasis,'O',['semi-standard tableau','partition','multiplicity'])

\ytableausetup{boxsize=7pt}
\begin{align*}
O_{
\fontsize{6pt}{0} \begin{ytableau}
1 & 1 & 1 & 2 & 2 & 2 
\end{ytableau} \fontsize{12pt}{0} ,
[1, 1, 1, 1, 1, 1] ,
1 } & =
 \frac{1}{360} \, \sqrt{5} \left[
 - 36 \tr X^3 Y^3 
 - 36 \tr X^2 Y^2 X Y 
 - 36 \tr X^2 Y X Y^2 
 - 12 \tr \left( X Y \right)^3 
 + 36 \tr X^3 Y^2 \tr Y 
 + 36 \tr X^2 Y X Y \tr Y 
 + 36 \tr X \tr X^2 Y^3 
 + 36 \tr X \tr X Y X Y^2 
 + 36 \tr X^2 Y^2 \tr X Y 
 + 18 \tr X Y \tr \left( X Y \right)^2 
 + 18 \tr X^3 Y \tr Y^2 
 + 18 \tr X^2 \tr X Y^3 
 + 36 \tr X^2 Y \tr X Y^2 
 + 4 \tr X^3 \tr Y^3 
 - 18 \tr X^3 Y \left( \tr Y \right)^2 
 - 36 \tr X \tr X^2 Y^2 \tr Y 
 - 18 \tr X \tr \left( X Y \right)^2 \tr Y 
 - 18 \left( \tr X \right)^2 \tr X Y^3 
 - 36 \tr X^2 Y \tr X Y \tr Y 
 - 6 \tr X^3 \tr Y \tr Y^2 
 - 18 \tr X^2 \tr X Y^2 \tr Y 
 - 36 \tr X \tr X Y \tr X Y^2 
 - 18 \tr X \tr X^2 Y \tr Y^2 
 - 6 \tr X \tr X^2 \tr Y^3 
 - 6 \left( \tr X Y \right)^3 
 - 9 \tr X^2 \tr X Y \tr Y^2 
 + 2 \tr X^3 \left( \tr Y \right)

In [36]:
#Checking [n-1,1] orthogonalisation for field content (4,2)
n=6
for lbl, op in finalBasis:
    if lbl[0] == [n-1,1]:
        p = lbl[1]
        norm = 1
        for i in range(len(p)):
            for j in range(p[i]):
                norm *= 1/(N-i+j)
        calculatedNorm = calculator.calculate(op,op,'G')
        matching = [Rational(norm(N=n+i)) == Rational(calculatedNorm(N=n+i)) for i in range(n+1)]
            
        print(all(matching),lbl,norm,calculatedNorm )

(True, [[5, 1], [2, 1, 1, 1, 1], 1], 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*N), 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*N))
(True, [[5, 1], [2, 2, 1, 1], 1], 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2), 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2))
(True, [[5, 1], [3, 1, 1, 1], 1], 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N), 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N))
(True, [[5, 1], [3, 2, 1], 1], 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*N^2), 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*N^2))
(True, [[5, 1], [3, 2, 1], 2], 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*N^2), 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*N^2))
(True, [[5, 1], [4, 1, 1], 1], 1/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*N), 1/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*N))
(True, [[5, 1], [4, 2], 1], 1/((N + 3)*(N + 2)*(N + 1)*(N - 1)*N^2), 1/((N + 3)*(N + 2)*(N + 1)*(N - 1)*N^2))
(True, [[5, 1], [5, 1], 1], 1/((N + 4)*(N + 3)*(N + 2)*(N + 1)*(N - 1)*N), 1/((N + 4)*(N + 3)*(N + 2)*(N + 1)*(N - 1)*N))


In [42]:
#Checking [n-1,1] orthogonalisation for field content (6,1)
n=7
for lbl, op in finalBasis:
    if lbl[0] == [n-1,1]:
        p = lbl[1]
        norm = 1
        for i in range(len(p)):
            for j in range(p[i]):
                norm *= 1/(N-i+j)
        calculatedNorm = calculator.calculate(op,op,'G')
        matching = [Rational(norm(N=n+i)) == Rational(calculatedNorm(N=n+i)) for i in range(n+1)]
            
        print(all(matching),lbl,norm,calculatedNorm )

(True, [[6, 1], [2, 1, 1, 1, 1, 1], 1], 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*(N - 5)*N), 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*(N - 5)*N))
(True, [[6, 1], [2, 2, 1, 1, 1], 1], 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*N^2), 1/((N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*N^2))
(True, [[6, 1], [3, 1, 1, 1, 1], 1], 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*N), 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*(N - 4)*N))
(True, [[6, 1], [2, 2, 2, 1], 1], 1/((N + 1)*(N - 1)^2*(N - 2)*(N - 3)*N^2), 1/((N + 1)*(N - 1)^2*(N - 2)*(N - 3)*N^2))
(True, [[6, 1], [3, 2, 1, 1], 1], 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2), 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2))
(True, [[6, 1], [3, 2, 1, 1], 2], 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2), 1/((N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2))
(True, [[6, 1], [4, 1, 1, 1], 1], 1/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N), 1/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N))
(True, [[6, 1], [3, 2, 2], 1], 1/((N + 2

In [ ]:
#Checking [n-1,1] orthogonalisation for field content (7,1)
n=8
for lbl, op in finalBasis:
    if lbl[0] == [n-1,1]:
        p = lbl[1]
        norm = 1
        for i in range(len(p)):
            for j in range(p[i]):
                norm *= 1/(N-i+j)
        calculatedNorm = calculator.calculate(op,op,'G')
        matching = [Rational(norm(N=n+i)) == Rational(calculatedNorm(N=n+i)) for i in range(n+1)]
            
        print(all(matching),lbl,norm,calculatedNorm )

In [204]:
calculator33.calculate(basis33, basis33, 'G')

KeyboardInterrupt: 

In [274]:
basis42

([6, 0], [6], 1):                (1/5*sqrt(5)*sqrt(2))(TrXXXXYY) + (1/5*sqrt(5)*sqrt(2))(TrXXXYXY) + (1/10*sqrt(5)*sqrt(2))(TrXXYXXY)
([6, 0], [5, 1], 1):             (-2/55*sqrt(11)*sqrt(5)*sqrt(3))(TrXXXXYY) + (-2/55*sqrt(11)*sqrt(5)*sqrt(3))(TrXXXYXY) + (-1/55*sqrt(11)*sqrt(5)*sqrt(3))(TrXXYXXY) + (1/33*sqrt(11)*sqrt(5)*sqrt(3))(TrXXXXY)(TrY) + (1/33*sqrt(11)*sqrt(5)*sqrt(3))(TrX)(TrXXXYY) + (1/33*sqrt(11)*sqrt(5)*sqrt(3))(TrX)(TrXXYXY)
([6, 0], [4, 2], 1):             (-1/35*sqrt(7)*sqrt(5)*sqrt(3)*sqrt(2))(TrXXXXYY) + (-1/35*sqrt(7)*sqrt(5)*sqrt(3)*sqrt(2))(TrXXXYXY) + (-1/70*sqrt(7)*sqrt(5)*sqrt(3)*sqrt(2))(TrXXYXXY) + (4/105*sqrt(7)*sqrt(5)*sqrt(3)*sqrt(2))(TrXXXY)(TrXY) + (1/210*sqrt(7)*sqrt(5)*sqrt(3)*sqrt(2))(TrXXXX)(TrYY) + (2/105*sqrt(7)*sqrt(5)*sqrt(3)*sqrt(2))(TrXX)(TrXXYY) + (1/105*sqrt(7)*sqrt(5)*sqrt(3)*sqrt(2))(TrXX)(TrXYXY)
([6, 0], [4, 1, 1], 1):          (2/5)(TrXXXXYY) + (2/5)(TrXXXYXY) + (1/5)(TrXXYXXY) + (-1/3)(TrXXXXY)(TrY) + (-1/3)(TrX)(TrXXXYY) + (-1/3)(TrX)(

In [651]:
print('The coefficients below are the expansion coefficients of the quarter-bps operators in terms of the GL(2) basis. In order, the GL(2) basis is labelled by:')
for lbl in basis42.labels:
    print(lbl)
print('')
for i in range(len(bpsvectors)):
    vec = bpsvectors[i]
    denom = vec[0].denominator()
    print('Operator ' + str(i+1) + ' has normalisation factor')
    print(denom)
    print('The relative coefficients are')
    for cf in vec*denom:
        print(cf)
    print('')

The coefficients below are the expansion coefficients of the quarter-bps operators in terms of the GL(2) basis. In order, the GL(2) basis is labelled by:
([4, 2], [5, 1], 1)
([4, 2], [4, 2], 1)
([4, 2], [4, 2], 2)
([4, 2], [4, 1, 1], 1)
([4, 2], [4, 1, 1], 2)
([4, 2], [3, 3], 1)
([4, 2], [3, 2, 1], 1)
([4, 2], [3, 2, 1], 2)
([4, 2], [3, 2, 1], 3)
([4, 2], [3, 1, 1, 1], 1)
([4, 2], [3, 1, 1, 1], 2)
([4, 2], [2, 2, 2], 1)
([4, 2], [2, 2, 1, 1], 1)
([4, 2], [2, 2, 1, 1], 2)
([4, 2], [2, 1, 1, 1, 1], 1)

Operator 1 has normalisation factor
2*sqrt(195*N^10 + 2298*N^9 + 9767*N^8 + 17008*N^7 + 21041*N^6 + 74974*N^5 + 135005*N^4 - 144704*N^3 - 399936*N^2 - 62976*N + 707328)
The relative coefficients are
(N - 1)*(N - 2)*(N - 3)*(N - 4)*N
0
-(N + 4)*(N - 1)*(N - 2)*(N - 3)*(N - 4)
-2/9*sqrt(5)*sqrt(3)*sqrt(2)*(N + 4)*(N - 1)*(N - 3)*(N - 4)*N
11/9*sqrt(3)*sqrt(2)*(N + 4)*(N - 1)*(N - 3)*(N - 4)*N
0
-2/3*(N + 4)*(N + 3)*(N - 1)*(N - 3)*(N - 4)
0
-8/3*sqrt(2)*(N + 4)*(N + 3)*(N - 1)*(N - 3)*(N - 4

In [536]:
for i in Fnorms:
    print(i.numerator())
    print('------------------------------------------------------------------------------------------------------------')
    print(i.denominator())
    print('')
    print('')

(195*N^5 + 1149*N^4 + 687*N^3 - 3927*N^2 - 1552*N + 4448)*(N + 4)*(N + 3)*(N + 2)*(N + 1)*(N - 1)^2*(N - 2)*(N - 3)*(N - 4)*N^2
------------------------------------------------------------------------------------------------------------
195*N^10 + 2298*N^9 + 9767*N^8 + 17008*N^7 + 21041*N^6 + 74974*N^5 + 135005*N^4 - 144704*N^3 - 399936*N^2 - 62976*N + 707328


(6135*N^8 + 69450*N^7 + 265508*N^6 + 271608*N^5 - 614501*N^4 - 1396058*N^3 + 54366*N^2 + 1782800*N + 1020192)*(195*N^5 + 1149*N^4 + 687*N^3 - 3927*N^2 - 1552*N + 4448)*(N + 3)*(N + 2)*(N + 1)^2*(N - 1)*(N - 2)*(N - 3)*N^2
------------------------------------------------------------------------------------------------------------
1196325*N^16 + 27085500*N^15 + 258799995*N^14 + 1318806504*N^13 + 3566832842*N^12 + 3188927936*N^11 - 8322235886*N^10 - 21588014040*N^9 + 10860905061*N^8 + 93884367956*N^7 + 85217854515*N^6 - 108073721232*N^5 - 238440710580*N^4 - 59355592848*N^3 + 182502383584*N^2 + 176417428224*N + 50393286144


(10035*

In [652]:
print('The coefficients below are the expansion coefficients of the symmetrised trace operators in terms of the GL(2) basis. In order, the GL(2) basis is labelled by:')
for lbl in basis42.labels:
    print(lbl)
print('')
for i in range(len(symtracevectors)):
    vec = symtracevectors[i]
    denom = vec[0].denominator()
    print('Operator ' + str(i+1) + ' has normalisation factor')
    print(denom)
    print('The relative coefficients are')
    for cf in vec*denom:
        print(cf)
    print('')

The coefficients below are the expansion coefficients of the symmetrised trace operators in terms of the GL(2) basis. In order, the GL(2) basis is labelled by:
([4, 2], [5, 1], 1)
([4, 2], [4, 2], 1)
([4, 2], [4, 2], 2)
([4, 2], [4, 1, 1], 1)
([4, 2], [4, 1, 1], 2)
([4, 2], [3, 3], 1)
([4, 2], [3, 2, 1], 1)
([4, 2], [3, 2, 1], 2)
([4, 2], [3, 2, 1], 3)
([4, 2], [3, 1, 1, 1], 1)
([4, 2], [3, 1, 1, 1], 2)
([4, 2], [2, 2, 2], 1)
([4, 2], [2, 2, 1, 1], 1)
([4, 2], [2, 2, 1, 1], 2)
([4, 2], [2, 1, 1, 1, 1], 1)

Operator 1 has normalisation factor
390
The relative coefficients are
sqrt(13)*sqrt(5)*sqrt(3)
0
-sqrt(13)*sqrt(5)*sqrt(3)
-10/3*sqrt(13)*sqrt(2)
11/3*sqrt(13)*sqrt(5)*sqrt(2)
0
-2/3*sqrt(13)*sqrt(5)*sqrt(3)
0
-8/3*sqrt(13)*sqrt(5)*sqrt(3)*sqrt(2)
50/3*sqrt(13)*sqrt(2)
29/3*sqrt(13)*sqrt(5)*sqrt(2)
10/3*sqrt(13)*sqrt(5)*sqrt(3)
2*sqrt(13)*sqrt(5)*sqrt(3)
13*sqrt(13)*sqrt(5)*sqrt(3)
65/3*sqrt(13)*sqrt(5)*sqrt(3)

Operator 2 has normalisation factor
3*sqrt(239265*N^10 + 2819646*N^9 + 1

In [767]:
for i in Gnorms:
    print(i.numerator())
    print('------------------------------------------------------------------------------------------------------------')
    print(i.denominator())
    print('')
    print('')

195*N^5 + 1149*N^4 + 687*N^3 - 3927*N^2 - 1552*N + 4448
------------------------------------------------------------------------------------------------------------
195*(N + 4)*(N + 3)*(N + 2)*(N + 1)*(N - 1)^2*(N - 2)*(N - 3)*(N - 4)*N^2


(6135*N^8 + 69450*N^7 + 265508*N^6 + 271608*N^5 - 614501*N^4 - 1396058*N^3 + 54366*N^2 + 1782800*N + 1020192)*(195*N^5 + 1149*N^4 + 687*N^3 - 3927*N^2 - 1552*N + 4448)
------------------------------------------------------------------------------------------------------------
5*(239265*N^10 + 2819646*N^9 + 10008075*N^8 + 379164*N^7 - 57964437*N^6 - 48042402*N^5 + 147342537*N^4 + 134046048*N^3 - 169443808*N^2 - 69184256*N + 127175168)*(N + 3)*(N + 2)*(N + 1)^2*(N - 1)*(N - 2)*(N - 3)*N^2


(10035*N^8 + 94914*N^7 + 264876*N^6 + 17268*N^5 - 819309*N^4 - 487830*N^3 + 780722*N^2 + 189568*N - 432744)*(6135*N^8 + 69450*N^7 + 265508*N^6 + 271608*N^5 - 614501*N^4 - 1396058*N^3 + 54366*N^2 + 1782800*N + 1020192)
-----------------------------------------------

In [281]:
coefficients1 = [0,0,0,0,1,-2,0,1,0,0,-1,2,-1,0,0,0,0,2,-2,0,0,0,0,-1,2,0,-1,0,-1,1,0,0,0,0]
coefficients2 = [0,0,0,0,0,0,0,0,1,-1,0,0,0,1,-1,1,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,-1,2,-1,-1]
coefficients3 = [0,1,-2,1,0,0,0,0,0,0,0,0,0,-1,-1,3,-2,0,0,-1,2,-2,1,0,0,0,0,0,0,0,4,0,-4,1]
op1 = Operator([4,2], [],[])
op2 = Operator([4,2],[],[])
op3 = Operator([4,2],[],[])
for i in range(len(coefficients1)):
    op1 += coefficients1[i] * tracelist[i]
    op2 += coefficients2[i] * tracelist[i]
    op3 += coefficients3[i] * tracelist[i]
op1 /= (3 * sqrt(5)*sqrt(2))
op2 /= (3 * sqrt(5))
op3 /= (6 * sqrt(5))
smallbasis = Basis([4,2], [op1,op2,op3])

In [679]:
calculator22 = correlatorCalculator([2,2])
calculator32 = correlatorCalculator([3,2])
calculator42 = correlatorCalculator([4,2])
calculator33 = correlatorCalculator([3,3])

In [ ]:
planarMatrix22 = calculator22.calculate(basis22, basis22, 'planar')
gMatrix22 = calculator22.calculate(basis22, basis22, 'G')

In [26]:
planarMatrix32 = calculator32.calculate(basis32, basis32, 'planar')
#gMatrix32 = calculator32.calculate(basis32, basis32, 'G') 

In [645]:
planarMatrix42 = calculator42.calculate(basis42, basis42, 'planar')
gMatrix42 = calculator42.calculate(basis42, basis42, 'G') 

In [996]:
calculator22.calculate(st22_2,st22_2,'planar').rational_simplify()

1

[-1/6*sqrt(3)*sqrt(2)*N/sqrt(3*N^2 + 2),
 1/6*sqrt(3)*sqrt(2)*N/sqrt(3*N^2 + 2),
 1/3*sqrt(3)*sqrt(2)*N/sqrt(3*N^2 + 2),
 1/3*sqrt(3)*sqrt(2)/sqrt(3*N^2 + 2),
 -1/6*sqrt(3)*sqrt(2)*N/sqrt(3*N^2 + 2),
 -1/6*sqrt(3)*sqrt(2)*N/sqrt(3*N^2 + 2),
 -1/3*sqrt(3)*sqrt(2)/sqrt(3*N^2 + 2)]

In [28]:
#Orthogonal basis for the Lambda = [3,2] sector in which the stringy exclusion principle is manifest
comm41 = 1/30 * ( 60/(sqrt(5)*sqrt(2)) * ( basis32[([3,2],[4,1],1)] - basis32[([3,2],[2,1,1,1],1)] ) 
              - 12/sqrt(2) * 5/2 * ( basis32[([3,2],[3,2],1)] - basis32[([3,2],[2,2,1],1)] ) 
              - 3 * 60/(sqrt(3)*sqrt(5)*sqrt(2)) * basis32[([3,2],[3,1,1],1)] )
comm41.canonicalizeRadicalsInCoefficients()
comm5 = 1/30 * ( 60/(sqrt(5)*sqrt(2)) * ( basis32[([3,2],[4,1],1)] + basis32[([3,2],[2,1,1,1],1)] ) 
              - 12/sqrt(2) * 5/2 * ( basis32[([3,2],[3,2],1)] + basis32[([3,2],[2,2,1],1)] ) 
              + 3 * 10/sqrt(5) * basis32[([3,2],[3,1,1],2)] )
stodd1 = sqrt(3) * (  ( basis32[([3,2],[4,1],1)] - basis32[([3,2],[2,1,1,1],1)] ) 
              + 2/(sqrt(3)) * basis32[([3,2],[3,1,1],1)] )
stodd2 = sqrt(3) * ( ( basis32[([3,2],[3,2],1)] - basis32[([3,2],[2,2,1],1)] ) 
              - sqrt(5)/(sqrt(3)) * basis32[([3,2],[3,1,1],1)] )
steven1 = 1 * ( 1 * ( basis32[([3,2],[4,1],1)] + basis32[([3,2],[2,1,1,1],1)] ) 
              - 2 * sqrt(2) * basis32[([3,2],[3,1,1],2)] )
steven2 = 1 * (  1 * ( basis32[([3,2],[3,2],1)] + basis32[([3,2],[2,2,1],1)] ) 
              + sqrt(5) * sqrt(2) * basis32[([3,2],[3,1,1],2)] )
print(comm41)
print(comm5)

(-2/5)(TrX)(TrX)(TrXY)(TrY) + (-1/5)(TrX)(TrXXYY) + (4/5)(TrXXY)(TrXY) + (-2/5)(TrXX)(TrXYY) + (1/5)(TrX)(TrXX)(TrY)(TrY) + (1/5)(TrX)(TrXYXY) + (-2/5)(TrXXX)(TrYY) + (1/5)(TrX)(TrX)(TrX)(TrYY)
(4/5)(TrXXYXY) + (-4/5)(TrXXXYY) + (1/5)(TrX)(TrXY)(TrXY) + (-2/5)(TrX)(TrXXY)(TrY) + (1/5)(TrX)(TrX)(TrXYY) + (-1/5)(TrX)(TrXX)(TrYY) + (1/5)(TrXXX)(TrY)(TrY)


In [664]:
matrix = [[sqrt(2/5), - 1/sqrt(2), - sqrt(6/5), 0, 1/sqrt(2), - sqrt(2/5)],
          [sqrt(2/5), - 1/sqrt(2), 0, 1/sqrt(5), - 1/sqrt(2), sqrt(2/5)],
          [sqrt(3), 0, 2, 0, 0, -sqrt(3)],
          [0, sqrt(3), -sqrt(5), 0, -sqrt(3), 0],
          [1, 0, 0, -2*sqrt(2), 0, 1],
          [0, 1, 0, sqrt(10), 1, 0]]
matrix= Matrix(matrix)
inv = matrix.inverse()
for i in range(6):
    for j in range(6):
        inv[i,j] = inv[i,j].canonicalize_radical()
print(inv)

[         1/15*sqrt(5)*sqrt(2)          1/10*sqrt(5)*sqrt(2)                 11/90*sqrt(3)          1/45*sqrt(5)*sqrt(3)                          3/10                  1/10*sqrt(5)]
[                 -1/6*sqrt(2)                  -1/4*sqrt(2)          1/45*sqrt(5)*sqrt(3)                   1/9*sqrt(3)                  1/10*sqrt(5)                           1/4]
[-1/15*sqrt(5)*sqrt(3)*sqrt(2)                             0                          2/15                 -1/15*sqrt(5)                             0                             0]
[                            0                  1/10*sqrt(5)                             0                             0                 -1/10*sqrt(2)          1/20*sqrt(5)*sqrt(2)]
[                  1/6*sqrt(2)                  -1/4*sqrt(2)         -1/45*sqrt(5)*sqrt(3)                  -1/9*sqrt(3)                  1/10*sqrt(5)                           1/4]
[        -1/15*sqrt(5)*sqrt(2)          1/10*sqrt(5)*sqrt(2)                -11/90*sqrt(3)

In [29]:
st32_1 = sqrt(3)/sqrt(2) * ( #- 1/15 * sqrt(5) * sqrt(2) * comm41 + 1/10 * sqrt(5) * sqrt(2) * comm5
        - 11/90 * sqrt(3) * stodd1 - sqrt(5)*sqrt(3)/45 * stodd2 
        + 3/10 * steven1 + sqrt(5)/10 * steven2)

st32_2 = ( #1/6*sqrt(2) * comm41 -1/4*sqrt(2) * comm5         
       - 1/45*sqrt(5)*sqrt(3) * stodd1 -1/9*sqrt(3) * stodd2
       + 1/10*sqrt(5) * steven1 + 1/4 * steven2 )
st32_2 -= calculator32.calculate(st32_1,st32_2)/calculator32.calculate(st32_1,st32_1) * st32_1
st32_2 *= 2 * (10*N^3 + 37*N^2 + 11*N - 36)/(sqrt(3)*sqrt(50*N^6 + 370*N^5 + 817*N^4 + 32*N^3 - 1629*N^2 - 276*N + 2088))
st32_2.factorCoefficients()

st32_3 = ( #sqrt(5)/10 * comm5 
          - sqrt(2)/10 * steven1 + sqrt(2)*sqrt(5)/20 * steven2)
st32_3 -= calculator32.calculate(st32_1,st32_3)/calculator32.calculate(st32_1,st32_1) * st32_1
st32_3 -= calculator32.calculate(st32_2,st32_3)/calculator32.calculate(st32_2,st32_2) * st32_2
st32_3 *= sqrt(5)*(5*N^3 + 11*N^2 - 7*N + 2)/(sqrt(6)*sqrt(15*N^6 + 66*N^5 + 33*N^4 - 82*N^3 + 51*N^2 - 12*N + 8))
st32_3.factorCoefficients()

st32_4 = ( #-1/6*sqrt(2) * comm41 -1/4*sqrt(2) * comm5
       + 1/45*sqrt(5)*sqrt(3) * stodd1 + 1/9*sqrt(3) * stodd2
       + 1/10*sqrt(5) * steven1 + 1/4 * steven2 )
st32_4 -= calculator32.calculate(st32_1,st32_4)/calculator32.calculate(st32_1,st32_1) * st32_1
st32_4 -= calculator32.calculate(st32_2,st32_4)/calculator32.calculate(st32_2,st32_2) * st32_2
st32_4 -= calculator32.calculate(st32_3,st32_4)/calculator32.calculate(st32_3,st32_3) * st32_3
st32_4 *= sqrt(3)*(3*N^3 + 5*N^2 - 5*N + 2)/(2*sqrt(3*N^6 + 10*N^5 - 22*N^3 + 34*N^2 - 24*N + 8))
st32_4.canonicalizeRadicalsInCoefficients()
st32_4.factorCoefficients()



In [928]:
print('1 with 1')
print(calculator32.calculate(st32_1,st32_1))
print(calculator32.calculate(st32_1,st32_1,'planar'))
print('1 with 2')
print(calculator32.calculate(st32_1,st32_2))
print(calculator32.calculate(st32_1,st32_2,'planar').rational_simplify())
print('2 with 2')
print(calculator32.calculate(st32_2,st32_2))
print(calculator32.calculate(st32_2,st32_2,'planar').rational_simplify().factor())
print('1 with 3')
print(calculator32.calculate(st32_1,st32_3))
print(calculator32.calculate(st32_1,st32_3,'planar').rational_simplify().factor())
print('2 with 3')
print(calculator32.calculate(st32_2,st32_3))
print(calculator32.calculate(st32_2,st32_3,'planar').rational_simplify().factor())
print('3 with 3')
print(calculator32.calculate(st32_3,st32_3))
print(calculator32.calculate(st32_3,st32_3,'planar').rational_simplify().factor())
print('1 with 4')
print(calculator32.calculate(st32_1,st32_4))
print(calculator32.calculate(st32_1,st32_4,'planar').rational_simplify().factor())
print('2 with 4')
print(calculator32.calculate(st32_2,st32_4))
print(calculator32.calculate(st32_2,st32_4,'planar').rational_simplify().factor())
print('3 with 4')
print(calculator32.calculate(st32_3,st32_4))
print(calculator32.calculate(st32_3,st32_4,'planar').rational_simplify().factor())
print('4 with 4')
print(calculator32.calculate(st32_4,st32_4))
print(calculator32.calculate(st32_4,st32_4,'planar').rational_simplify().factor())

1 with 1
1/10*(10*N^3 + 37*N^2 + 11*N - 36)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)
1
1 with 2
0
-1/2*sqrt(5)*(3*sqrt(2)*N^2 - sqrt(2)*N - 24*sqrt(2))/sqrt(50*N^6 + 370*N^5 + 817*N^4 + 32*N^3 - 1629*N^2 - 276*N + 2088)
2 with 2
(10*N^3 + 37*N^2 + 11*N - 36)*(5*N^3 + 11*N^2 - 7*N + 2)/((50*N^6 + 370*N^5 + 817*N^4 + 32*N^3 - 1629*N^2 - 276*N + 2088)*(N + 1)*(N - 1)*(N - 2)*N^2)
1
1 with 3
0
1/15*sqrt(6)*sqrt(5)*sqrt(3)*(N + 1)*(N - 3)/sqrt(15*N^6 + 66*N^5 + 33*N^4 - 82*N^3 + 51*N^2 - 12*N + 8)
2 with 3
0
-1/3*sqrt(6)*sqrt(3)*sqrt(2)*(5*N^4 + 15*N^3 - 18*N^2 - 34*N + 54)*(N + 1)/(sqrt(50*N^6 + 370*N^5 + 817*N^4 + 32*N^3 - 1629*N^2 - 276*N + 2088)*sqrt(15*N^6 + 66*N^5 + 33*N^4 - 82*N^3 + 51*N^2 - 12*N + 8))
3 with 3
(5*N^3 + 11*N^2 - 7*N + 2)*(3*N^3 + 5*N^2 - 5*N + 2)/((15*N^6 + 66*N^5 + 33*N^4 - 82*N^3 + 51*N^2 - 12*N + 8)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*N)
1
1 with 4
0
-1/10*sqrt(5)*sqrt(2)*(N - 2)*(N - 3)/sqrt(3*N^6 + 10*N^5 - 22*N^3 + 34*N^2 - 24*N + 8)
2 with 4
0
(5*N^4 

In [824]:
print(calculator32.calculate(( #1/6*sqrt(2) * comm41 -1/4*sqrt(2) * comm5         
       - 1/45*sqrt(5)*sqrt(3) * stodd1 -1/9*sqrt(3) * stodd2
       + 1/10*sqrt(5) * steven1 + 1/4 * steven2 ),st32_1).rational_simplify().factor())

1/60*sqrt(5)*sqrt(3)*sqrt(2)*(5*N^3 + 23*N^2 + 4*N - 54)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)


In [67]:
tracelist = []
for lbl, op in basis32:
    for cf, tr in op:
        if tr not in tracelist:
            tracelist.append(tr)
tracecoefficients = [ [0 for tr in tracelist] for lbl, op in basis32]
for i in range(len(basis32)):
    for cf, tr in basis32[i]:
        tracecoefficients[i][tracelist.index(tr)] = cf
matrix = Matrix(tracecoefficients).T

Gmultipliers = [0,0,0,0,0,0,0,0,0,0,0,0, 1/(N*(N-1)*(N+1)*(N+2)*(N+3)), 1/(N^2*(N-1)*(N+1)*(N+2)), 
                1/(N*(N-1)*(N+1)*(N+2)*(N-2)), 1/(N*(N-1)*(N+1)*(N+2)*(N-2)), 1/(N^2*(N-1)*(N+1)*(N-2)),
               1/(N*(N-1)*(N+1)*(N-2)*(N-3))]

In [69]:
coefficients = [0 for tr in tracelist]
for cf, tr in bps32_2:
    coefficients[tracelist.index(tr)] = cf
v = vector(coefficients)
v2 = matrix.solve_right(v)
#bps32_4 = Operator([3,2],[],[])
for i in range(len(v2)):
    v2[i] = v2[i].canonicalize_radical().factor()
    print(v2[i])
#    bps32_4 += Gmultipliers[i] * v2[i] * basis32[i]



0
0
0
0
0
0
0
0
0
0
0
0
1/3*sqrt(5)*sqrt(3)*(N - 1)*(N - 2)*N/sqrt(50*N^6 + 220*N^5 + 192*N^4 - 78*N^3 + 541*N^2 - 156*N + 78)
-5/6*sqrt(3)*(N + 3)*(N - 1)*(N - 2)/sqrt(50*N^6 + 220*N^5 + 192*N^4 - 78*N^3 + 541*N^2 - 156*N + 78)
2*sqrt(5)*(N + 3)*(N - 1)*N/sqrt(50*N^6 + 220*N^5 + 192*N^4 - 78*N^3 + 541*N^2 - 156*N + 78)
1/2*sqrt(5)*sqrt(3)*sqrt(2)*(N + 3)*(N - 1)*N/sqrt(50*N^6 + 220*N^5 + 192*N^4 - 78*N^3 + 541*N^2 - 156*N + 78)
1/6*sqrt(3)*(15*N^3 + 48*N^2 + 19*N + 6)/sqrt(50*N^6 + 220*N^5 + 192*N^4 - 78*N^3 + 541*N^2 - 156*N + 78)
-1/3*sqrt(5)*sqrt(3)*(3*N + 8)*N/sqrt(50*N^6 + 220*N^5 + 192*N^4 - 78*N^3 + 541*N^2 - 156*N + 78)


In [85]:
print(calculator32.calculate(bps32_1,bps32_2,'F'))
print(calculator32.calculate(bps32_1,bps32_3,'F'))
print(calculator32.calculate(bps32_1,bps32_4,'F'))
print(calculator32.calculate(bps32_2,bps32_3,'F'))
print(calculator32.calculate(bps32_2,bps32_4,'F'))
print(calculator32.calculate(bps32_3,bps32_4,'F'))

0
0
0
0
0
0


In [64]:
print(calculator32.calculate(bps32_1,bps32_1,'planar').canonicalize_radical().factor())
print(calculator32.calculate(bps32_1,bps32_1,'F').canonicalize_radical().factor())

1
(10*N^3 + 37*N^2 + 11*N - 36)*(N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2/(10*N^6 + 74*N^5 + 199*N^4 + 252*N^3 + 351*N^2 + 648*N + 702)


In [59]:
print(calculator32.calculate(bps32_2,bps32_2,'planar').canonicalize_radical().factor())
print(calculator32.calculate(bps32_2,bps32_2,'F').canonicalize_radical().factor())

1
(10*N^3 + 37*N^2 + 11*N - 36)*(5*N^3 + 11*N^2 - 7*N + 2)*(N + 1)*(N - 1)*(N - 2)*N^2/(50*N^6 + 220*N^5 + 192*N^4 - 78*N^3 + 541*N^2 - 156*N + 78)


In [57]:
print(calculator32.calculate(bps32_3,bps32_3,'planar').canonicalize_radical().factor())
print(calculator32.calculate(bps32_3,bps32_3,'F').canonicalize_radical().factor())

1
(5*N^3 + 11*N^2 - 7*N + 2)*(3*N^3 + 5*N^2 - 5*N + 2)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*N/(15*N^6 + 50*N^5 + 17*N^4 - 66*N^3 + 115*N^2 - 60*N + 20)


In [55]:
print(calculator32.calculate(bps32_4,bps32_4,'planar').canonicalize_radical().factor())
print(calculator32.calculate(bps32_4,bps32_4,'F').canonicalize_radical().factor())

1
(3*N^3 + 5*N^2 - 5*N + 2)*(N + 2)*(N + 1)*(N - 1)*N^3/(3*N^4 + 5*N^2 - 4*N + 2)


In [ ]:
st32_2alt = ( #sqrt(5)/10 * comm5 
          - sqrt(2)/10 * steven1 + sqrt(2)*sqrt(5)/20 * steven2)
st32_2alt -= calculator32.calculate(st32_1,st32_2alt)/calculator32.calculate(st32_1,st32_1) * st32_1
st32_2alt.factorCoefficients()
print(calculator32.calculate(st32_2alt,st32_alt,'planar'))

## Investigating [n-1,1] orthogonalisation

In [265]:
n=4
G = SymmetricGroup(n)
H = SymmetricGroup(n-1)
sigma = PermutationGroupElement([1,2,3,4],parent=G)
tau = PermutationGroupElement([(1,4,3,2)], parent=G)
print('sigma = ' + str(sigma))
print('tau = ' + str(tau))

print(sum([sum([sigma == beta.inverse() * rho * beta * tau * rho.inverse() for beta in H]) for rho in G]))

sigma = ()
tau = (1,4,3,2)
0


In [198]:
sum([True, True, False])

2

In [201]:
tau.inverse()

(1,2)

## Investigating lambda, p multiplicities

In [192]:
s = SymmetricFunctions(QQ).s()

In [197]:
n=10
for R in Partitions(n):
    c = [[0 for i in range(n)] for j in range(n)]
    for i in range(len(R)):
        a = R[:i].count(R[i])
        for j in range(R[i]):
            c[j][a] += 1
    q = [0 for i in range(n)]
    for i in range(n):
        q[i] = sum([c[j][i-j] for j in range(i+1)])
        
        
            
    print( R )
    print(c)
    print(q)
    print((s(R).itensor(s([n-1,1]))).scalar(s(R)), q[0] - 1)
    print((s(R).itensor(s([n-2,2]))).scalar(s(R)), q[1] + binomial(q[0],2) - q[0])
    print((s(R).itensor(s([n-3,3]))).scalar(s(R)), (q[2] + q[1]*(q[0] - 2) 
                                                   + q[0]*(q[0] - 1)*(q[0] - 5)/6) )
    print((s(R).itensor(s([n-4,4]))).scalar(s(R)), (q[3] + q[2]*(q[0] - 2) + q[1]*(q[0] - 1)*(q[0] - 4)/2
                                                   + binomial(q[1],2) + q[0]*(q[0] - 1)*(q[0]-2)*(q[0] - 7)/24) )
    print('')
    
    

[10]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(0, 0)
(0, 0)
(0, 0)
(0, 0)

[9, 1]
[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[2, 1, 1, 1, 1, 1, 1, 1, 1, 0]
(1, 1)
(1, 0)
(0, 0)
(0, 0)

[8, 2]
[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0

(1, 1)
(3, 2)
(3, 1)
(4, 2)

[4, 3, 3]
[[2, 1, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[2, 3, 3, 2, 0, 0, 0, 0, 0, 0]
(1, 1)
(2, 2)
(2, 2)
(2, 2)

[4, 3, 2, 1]
[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[4, 3, 2, 1, 0, 0, 0, 0, 0, 0]
(3, 3)
(8, 5)
(15, 6)
(15, 5)

[4, 3, 1, 1, 1]
[[3, 1, 1, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 

(0, 0)
(0, 0)



In [5]:
n=14
var('N')
for p in Partitions(n):
    #Calculating c_{i,j} for each partition
    c = [[0 for i in range(n)] for j in range(n)]
    for i in range(len(p)):
        a = p[:i].count(p[i])
        for j in range(p[i]):
            c[j][a] += 1
    #Summing the c_{i,j} to form q_i (called s_i in the latex document)
    q = [0 for i in range(n)]
    for i in range(n):
        q[i] = sum([c[j][i-j] for j in range(i+1)])
    #Calculating the multiplicities using the partition sum formula
    cfs = [0 for i in range(n)]
    cfs[0] = 1
    for exp in range(1,n):
        for partition in Partitions(exp):
            mults = [ list(partition).count(i+1) for i in range(n)]
            offset = 0
            prod = 1
            for component in partition:
                prod *= q[component-1] - offset
                offset += 1
            for mult in mults:
                prod /= factorial(mult)
            cfs[exp] += prod
    cfsdiff =[1] + [ cfs[i] - cfs[i-1] for i in range(1,n)]
        
    
    #Calculating the multiplicities directly from the character polynomial coefficients
    pmults = [ list(p).count(i+1) for i in range(n)]
    prod = 1
    for i in range(n):
        for j in range(i+1):
            prod *= (1-N^(pmults[i]+j+1))/(1-N^(j+1))
    prod *= (1-N)
    prod = prod.factor()
    prod = prod.expand()
    coefficients = [1] + [prod.coefficient(N^i) for i in range(1,n)]
    
    #Print the partition, all c_{i,j}, all q_i, then all lambda multiplicities twice, first calculated using the q_i and  
    # the partition sum formula, and the second time calculated by extracting the coefficients directly from the 
    # character as a polynomial
    print(p)
    print(c)
    print(q)
    print(cfsdiff[:floor(n/2+1)])
    print(coefficients[:floor(n/2+1)])
    print('')

[14]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]

[13, 1]
[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0

[9, 5]
[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0]

[9, 4, 1]
[[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 

[8, 3, 2, 1]
[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[4, 3, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 3, 5, 6, 5, 3, 1, 0]
[1, 3, 5, 6, 5, 3, 1, 0]

[8, 3, 1, 1, 1]
[[3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

[7, 4, 1, 1, 1]
[[3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[3, 3, 3, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 2, 3, 4, 4, 3, 2, 1]
[1, 2, 3, 4, 4, 3, 2, 1]

[7, 3, 3, 1]
[[3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

[6, 5, 2, 1]
[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[4, 3, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 3, 5, 6, 6, 6, 5, 2]
[1, 3, 5, 6, 6, 6, 5, 2]

[6, 5, 1, 1, 1]
[[3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

[5, 2, 2, 2, 1, 1, 1]
[[3, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[3, 4, 4, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 2, 4, 6, 7, 7, 5, 2]
[1, 2, 4, 6, 7, 7, 5, 2]

[5, 2, 2, 1, 1, 1, 1, 1]
[[3, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0,

[4, 4, 1, 1, 1, 1, 1, 1]
[[2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[2, 3, 3, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 2, 2, 3, 2, 2, 0]
[1, 1, 2, 2, 3, 2, 2, 0]

[4, 3, 3, 3, 1]
[[3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0,

[4, 2, 2, 1, 1, 1, 1, 1, 1]
[[3, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[3, 4, 3, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 2, 4, 5, 6, 5, 4, 1]
[1, 2, 4, 5, 6, 5, 4, 1]

[4, 2, 1, 1, 1, 1, 1, 1, 1, 1]
[[3, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0,

[3, 3, 1, 1, 1, 1, 1, 1, 1, 1]
[[2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[2, 3, 3, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 2, 2, 2, 1, 1, 0]
[1, 1, 2, 2, 2, 1, 1, 0]

[3, 2, 2, 2, 2, 2, 1]
[[3, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0,

[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 2, 1, 1, 0, 0, 0]
[1, 1, 2, 1, 1, 0, 0, 0]

[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
component

1

In [301]:
prod = 1
pmults = [1,0,0,0,1,0]
for i in range(n):
    for j in range(i):
        prod *= (1-N^(pmults[i]+j+1))/(1-N^(j+1))
        print(i,j,pmults[i],prod)

(1, 0, 0, 1)
(2, 0, 0, 1)
(2, 1, 0, 1)
(3, 0, 0, 1)
(3, 1, 0, 1)
(3, 2, 0, 1)
(4, 0, 1, (N^2 - 1)/(N - 1))
(4, 1, 1, (N^3 - 1)/(N - 1))
(4, 2, 1, (N^4 - 1)/(N - 1))
(4, 3, 1, (N^5 - 1)/(N - 1))
(5, 0, 0, (N^5 - 1)/(N - 1))
(5, 1, 0, (N^5 - 1)/(N - 1))
(5, 2, 0, (N^5 - 1)/(N - 1))
(5, 3, 0, (N^5 - 1)/(N - 1))
(5, 4, 0, (N^5 - 1)/(N - 1))


In [300]:
prod

(N^5 - 1)/(N - 1)

## Simplicial Homology

In [8]:
X = SimplicialComplex([[0,4],[0,5],[0,7],[1,4],[1,6],[1,8],[2,5],[2,6],[2,9],[3,7],[3,8],[3,9],
                      [4,10],[5,11],[6,12],[7,12],[8,11],[9,10]])
print(X.homology())
Y = SimplicialComplex([[0,4,13,14],[0,5,13,14],[0,7,13,14],[1,4,13,14],[1,6,13,14],[1,8,13,14],[2,5,13,14],
                       [2,6,13,14],[2,9,13,14],[3,7,13,14],[3,8,13,14],[3,9,13,14],
                      [4,10,13,14],[5,11,13,14],[6,12,13,14],[7,12,13,14],[8,11,13,14],[9,10,13,14]])
print(Y.homology())

{0: 0, 1: Z^6}
{0: 0, 1: 0, 2: 0, 3: 0}


In [10]:
complexes = [[0,6,17],[0,6,18],[0,6,19],[1,6,17],[1,6,18],[1,6,19],[2,6,17],[2,6,18],[2,6,19],
                      [0,7,17],[0,8,18],[0,9,19],[1,10,17],[1,11,18],[1,12,19],[2,13,17],[2,14,18],[2,15,19],
                      [0,7,20],[0,8,20],[0,9,20],[1,10,21],[1,11,21],[1,12,21],[2,13,22],[2,14,22],[2,15,22],
                      [3,7,17],[4,8,18],[5,9,19],[3,10,17],[4,11,18],[5,12,19],[3,13,17],[4,14,18],[5,15,19],
                      [3,7,20],[4,8,20],[5,9,20],[3,10,21],[4,11,21],[5,12,21],[3,13,22],[4,14,22],[5,15,22],
                      [3,16,20],[4,16,20],[5,16,20],[3,16,21],[4,16,21],[5,16,21],[3,16,22],[4,16,22],[5,16,22]]
X = SimplicialComplex(complexes)
print(X.homology())
newcomplexes = [c + [23,24] for c in complexes]
Y = SimplicialComplex(newcomplexes)
print(Y.homology())

{0: 0, 1: 0, 2: Z x Z x Z x Z}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0}


In [7]:
X = SimplicialComplex([[0,1],[0,2],[0,3],[1,4],[2,5],[3,6],[4,7],[5,7],[6,7]])
print(X.homology())
Y = SimplicialComplex([[0,1,8,9],[0,2,8,9],[0,3,8,9],[1,4,8,9],[2,5,8,9],[3,6,8,9],[4,7,8,9],[5,7,8,9],[6,7,8,9]])
print(Y.homology())

{0: 0, 1: Z x Z}
{0: 0, 1: 0, 2: 0, 3: 0}


In [12]:
X.homology()

{0: 0, 1: 0, 2: Z x Z x Z x Z}

In [7]:
X = SimplicialComplex([[3,6],[4,6],[5,6],[0,3],[1,4],[2,5],[0,7],[1,7],[2,7]])
X.homology()

{0: 0, 1: Z x Z}

In [8]:
X = SimplicialComplex([[0,3],[1,3],[0,4],[1,5],[2,4],[2,5]])
X.homology()

{0: 0, 1: Z}

In [2]:
class PseudoDendogram:
    
    def __init__(self,connections):
        self.connections = connections
        self.depth = len(connections)
        self.nodecounts = [len(l) for l in connections] + [max(connections[-1])+1]
        
    
    def contract(self, index):
        if index >= self.depth or index < 1:
            raise Exception('Index must be between 1 (inclusive) and the depth of the pseudo-dendogram (exclusive)')
        newconnections = [l[:] for l in self.connections]
        newconnections[index - 1] = [newconnections[index][i] for i in newconnections[index - 1]]
        newconnections.pop(index)
        return PseudoDendogram(newconnections)
    
    def __repr__(self):
        return str(self.connections)
                  
    def expand(self, layer):
        if self.nodecounts[layer] == self.nodecounts[layer-1]:
            raise Exception('This layer cannot be expanded')

        expandedconnectioncomponents= [[] for i in range(max(self.connections[layer])+1)]
        for k in range(max(self.connections[layer])+1):
            mult = self.connections[layer].count(k)
            for p in Partitions(mult):
                expandedconnectioncomponents[k] += twostageconnectionsfrompartition(list(p))


        indices = [0 for comp in expandedconnectioncomponents]
        maxindices = [len(comp)-1 for comp in expandedconnectioncomponents]
        expandedconnections = []
        while indices != maxindices:
            tempconnections = [[],[]]
            firstlayertotal = 0
            secondlayertotal = 0
            for i in range(len(indices)):
                comp = expandedconnectioncomponents[i][indices[i]]
                tempconnections[0] += [j+firstlayertotal for j in comp[0]]
                tempconnections[1] += [j+secondlayertotal for j in comp[1]]
                firstlayertotal += max(comp[0]) + 1
                secondlayertotal += max(comp[1]) + 1
            expandedconnections.append(tempconnections)
    
            i = len(indices) - 1
            while indices[i] == maxindices[i]:
                i -= 1
            indices[i] += 1
            for j in range(i+1, len(indices)):
                indices[j] = 0
    
        expandedconnections.pop(0)
        expandedconnections = [self.connections[:layer] + conn + self.connections[layer+1:] for conn in expandedconnections]
        return [PseudoDendogram(conn) for conn in expandedconnections]
    
    def __eq__(self,other):
        if isinstance(other,PseudoDendogram):
            return self.toset() == other.toset()
        else:
            return NotImplemented
        
    def __hash__(self):
        return hash(self.toset())

    
    def toset(self):
        inputs = [i+1 for i in range(len(self.connections[0]))]
        for j in range(self.depth):
            setlist = []
            for i in range(max(self.connections[j])+1):
                tempsetlist = []
                try:
                    index = -1
                    while True:
                        index = self.connections[j].index(i,index+1)
                        tempsetlist.append(index)
                except (ValueError):
                    pass
                tempsetlist = [inputs[k] for k in tempsetlist]
                setlist.append(frozenset(tempsetlist))
            inputs = setlist
        return frozenset(setlist)
    
def twostageconnectionsfrompartition(p):
    #Splitting up the partition by length of row. Finding the possible positions for each length.
    n=sum(p)
    pmults = [p.count(i+1) for i in range(n)]
    combspermult = [[] for i in range(n)]
    ntemp = n
    for i in range(n):
        combspermult[i] = list(Combinations(ntemp,(i+1)*pmults[i]))
        ntemp -= (i+1)*pmults[i]

    #For each length of row, finding the possible positions purely among themselves
    combinationslist = [[] for i in range(n)]
    for i in range(n):
        m = (i+1)*pmults[i]
        if m > 0:
            combs = list(Combinations(m,i+1))
            #setting up multiindex
            indices = [j for j in range(pmults[i])]
            maxindices = [len(combs) - pmults[i] +j for j in range(pmults[i])]
            while True:
                chosencombs = [ combs[j] for j in indices]
                if len(set([]).union(*chosencombs)) == m:
                    for combpermult in combspermult[i]:
                        combinationslist[i].append([ [combpermult[j] for j in comb] for comb in chosencombs])
                
                #iterator block for multiindex
                if indices == maxindices:
                    break
                j = len(indices) - 1
                while indices[j] == maxindices[j]:
                    j -= 1
                indices[j] += 1
                for k in range(j+1, len(indices)):
                    indices[k] = indices[j] + k - j
    
    #Combining the two sets of possible positions
    connectionslist = []
    indices = [ 0 for comb in combinationslist]
    maxindices = [max(len(comb)-1,0) for comb in combinationslist]
    while True:
        tempconnection = [[None for j in range(n)],[0 for j in  range(len(p))]]
        remainingpositions = [i for i in range(n)]
        target = 0
        for i in range(len(combinationslist)):
            if len(combinationslist[i]) > 0:
                comb = combinationslist[i][indices[i]]
            
                for subcomb in comb:
                    for j in subcomb:
                        tempconnection[0][remainingpositions[j]] = target
                    target += 1
                    
                positionstoremove = list(set([]).union(*comb))
                positionstoremove.sort()
                positionstoremove.reverse()
                for j in positionstoremove:
                    remainingpositions.pop(j)
                
                
        connectionslist.append(tempconnection)
        
        #iterator block
        if indices == maxindices:
            break
        i = len(indices) - 1
        while indices[i] == maxindices[i]:
            i -= 1
        indices[i] += 1
        for j in range(i+1, len(indices)):
            indices[j] = 0
    
    return connectionslist

def dendogramcomplex(onestagedend):
    n = onestagedend.nodecounts[0] - onestagedend.nodecounts[1]
    dendlist = [[onestagedend]] + [[] for i in range(n-1)]
    for i in range(n-1):
        for dend in dendlist[i]:
            for layer in range(dend.depth):
                if dend.nodecounts[layer] > dend.nodecounts[layer+1] + 1:
                    dendlist[i+1] += dend.expand(layer)
        dendlist[i+1] = list(set(dendlist[i+1]))

    facets = []
    for dend in dendlist[n-1]:
        facet = []
        for uncontractedlayer in range(n-1):
            contracteddend = dend
            for layer in range(n-2):
                if layer >= uncontractedlayer:
                    contracteddend = contracteddend.contract(2)
                else:
                    contracteddend = contracteddend.contract(1)
            facet.append(dendlist[1].index(contracteddend))
        facets.append(facet)
    return SimplicialComplex(facets)

def dendogramhomology(onestagedend):
    return onestagedend.dendogramcomplex().homology()
 
        

        

In [553]:
dendogramhomology(PseudoDendogram([[0,0,0,0,0,0]]))

[1, 201, 1865, 4245, 2700]
{0: 0, 1: 0, 2: 0, 3: Z^120}


In [12]:
X = dendogramcomplex(PseudoDendogram([[0,0,0,0,0,0,0]]))
a = X.cells()
[len(a[i]) for i in range(5)]

[875, 16674, 74165, 114345, 56700]

In [13]:
X.homology()

{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^720}

In [28]:
conn = [0,0,0,0,0,1,1,1]
n = len(conn) - max(conn) - 1
X = dendogramcomplex(PseudoDendogram([conn]))
a = X.cells()
print([len(a[i]) for i in range(n-1)])
print(X.homology())

[258, 3519, 13060, 17850, 8100]
{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^48}


In [20]:
print(X.homology())
          

{0: 0, 1: 0, 2: 0, 3: Z^120}


In [447]:
subcomb

[4, 5]

In [448]:
j

5

In [481]:
connections = [[0,0,0,0,1,1,1,2,2,3,3,4,5],[0,0,1,1,1,2],[0,0,1]]
connections2 = [[2,2,2,2,5,5,5,1,1,4,4,3,0],[0,1,2,1,1,2],[0,1,1]]
dend = PseudoDendogram(connections)
dend2 = PseudoDendogram(connections2)
print(dend.depth)
print(dend.nodecounts)
condend1 = dend.contract(1)
condend21 = dend2.contract(1)
print(condend1.connections)
condend2 = dend.contract(2)
condend22 = dend2.contract(2)
print(condend2.connections)

3
[13, 6, 3, 2]
[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2], [0, 0, 1]]
[[0, 0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 4, 5], [0, 0, 0, 0, 0, 1]]


NameError: global name 'expandedconnetions' is not defined

In [433]:
factorial(9)/(3*8*6)

2520

In [194]:
gArrayCorr(basis[1],basis[2])

(14.666666666666668*N^4 + 101.33333333333333*N^3 + 204.0*N^2 + 48.0*N - 144.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [195]:
gArrayCorr(basis[1],basis[3])

(16.666666666666668*N^4 + 96.0*N^3 + 162.0*N^2 + 96.0*N + 72.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [196]:
gArrayCorr(basis[1],basis[4])

(14.666666666666668*N^4 + 101.33333333333333*N^3 + 204.0*N^2 + 48.0*N - 144.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [197]:
gArrayCorr(basis[2],basis[2])

(50.666666666666664*N^4 + 106.66666666666666*N^3 - 168.0*N^2 + 288.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [198]:
gArrayCorr(basis[2],basis[3])

(14.666666666666668*N^4 + 101.33333333333333*N^3 + 204.0*N^2 + 48.0*N - 144.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [199]:
gArrayCorr(basis[2],basis[4])

(18.666666666666668*N^4 + 106.66666666666667*N^3 + 120.0*N^2 + 288.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [200]:
gArrayCorr(basis[3],basis[3])

(24.666666666666668*N^4 + 96.0*N^3 + 90.0*N^2 + 96.0*N + 72.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [201]:
gArrayCorr(basis[3],basis[4])

(14.666666666666668*N^4 + 101.33333333333333*N^3 + 204.0*N^2 + 48.0*N - 144.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [202]:
gArrayCorr(basis[4],basis[4])

(34.66666666666667*N^4 + 106.66666666666666*N^3 - 24.000000000000007*N^2 + 288.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2)

In [203]:
type((34.66666666666667*N^4 + 106.66666666666666*N^3 - 24.000000000000007*N^2 + 288.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2))

<type 'sage.symbolic.expression.Expression'>

In [150]:
((2*N^2 + 3*N + 3)*(N+3)*16/3).expand()

32/3*N^3 + 48*N^2 + 64*N + 48

In [130]:
matrix10 = np.zeros([9,9])
for i in range(9):
    for j in range(i,9):
        matrix10[i,j] = (N^4 * matrix[i,j])(N=10)
        matrix10[j,i] = matrix10[i,j]

In [66]:
Q,R = np.linalg.qr(matrix10)

In [75]:
R

array([[-40.113191  ,  69.95503563,  85.25709471,  69.95503563,
         77.18229338],
       [  0.        ,  -9.83204582,   9.12586206,  -2.62556516,
          4.87804346],
       [  0.        ,   0.        , -30.98258568,   2.12265526,
          1.33813509],
       [  0.        ,   0.        ,   0.        ,  -9.23416846,
          4.11449341],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         16.50104641]])

In [77]:
L = np.linalg.cholesky(matrix10)

In [98]:
basisChange = np.linalg.inv(L.T)

In [87]:
vec

array([0.27657098, 0.        , 0.        , 0.        , 0.        ])

In [88]:
vec2 = np.linalg.inv(L.T).dot(np.array([0,1,0,0,0]))

In [89]:
vec3 = np.linalg.inv(L.T).dot(np.array([0,0,1,0,0]))

In [95]:
vec3.dot(matrix10.dot(vec3))

1.0000000000000002

In [107]:
np.linalg.inv(L.T)

array([[ 0.27657098,  0.38059413,  0.17574233,  0.23213291,  0.17489193],
       [ 0.        ,  0.2663782 , -0.04030935, -0.11377425, -0.0332353 ],
       [ 0.        ,  0.        ,  0.15854832, -0.01573265, -0.0311687 ],
       [ 0.        ,  0.        ,  0.        ,  0.2924496 , -0.0332353 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.2145374 ]])

In [108]:
vec2

array([0.38059413, 0.2663782 , 0.        , 0.        , 0.        ])

In [70]:
perms

[(), (1,2), (1,2)(3,4), (1,2,3), (1,2,3,4)]

In [104]:
omegaCharacters = [sum([N ** len(perms[i].cycle_type()) * R(perms[i]) * sizes[i] for i in range(len(perms))]) for R in g.irreducible_characters()]

In [93]:
omegaCharacters

[N^4 - 6*N^3 + 11*N^2 - 6*N,
 3*N^4 - 6*N^3 - 3*N^2 + 6*N,
 2*N^4 - 2*N^2,
 3*N^4 + 6*N^3 - 3*N^2 - 6*N,
 N^4 + 6*N^3 + 11*N^2 + 6*N]

In [118]:
matrixReverse[0,0]

(2304.0*N^3 + 13824.0*N^2 + 25344.0*N + 13824.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N)

In [125]:
gArrayCorr(basisReverse[0],basisReverse[0])

(2304.0*N^3 + 13824.0*N^2 + 25344.0*N + 13824.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N)

In [126]:
gArrayCorr(basis[8],basis[8])

(2304.0*N^3 + 13824.0*N^2 + 25344.0*N + 13824.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N)

In [129]:
vpGenerate(2,2)

[[[2, 2]],
 [[2, 1], [0, 1]],
 [[2, 0], [0, 2]],
 [[2, 0], [0, 1], [0, 1]],
 [[1, 2], [1, 0]],
 [[1, 1], [1, 1]],
 [[1, 1], [1, 0], [0, 1]],
 [[1, 0], [1, 0], [0, 2]],
 [[1, 0], [1, 0], [0, 1], [0, 1]]]

In [132]:
sqrt(4)

2

In [135]:
type(sqrt(N-5))

<type 'sage.symbolic.expression.Expression'>

In [136]:
len(matrix)

9

In [148]:
copyMatrix=matrix.copy()

In [152]:
matrix[0,0]

(10.666666666666668*N^2 + 64.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3))

In [151]:
copyMatrix[0,0] 

(10.666666666666668*N^2 + 64.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)) + 1

In [154]:
type(np.identity(5))

<type 'numpy.ndarray'>

In [155]:
?np.outer

In [159]:
L = cholesky(matrix10)

In [165]:
max((L.dot(L.T) - matrix10).flatten())

2.2737367544323206e-13

In [174]:
L = symbolicCholesky(matrix)

In [188]:
L[1,0]

-1.0*(10.666666666666668*N^3 + 48.0*N^2 + 64.0*N + 48.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N*sqrt((10.666666666666668*N^2 + 64.0)/((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3))))

In [193]:
L[1,0].canonicalize_radical()

-sqrt(N^6 - 14*N^4 + 49*N^2 - 36)*(10.66666666666667*N^3 + 48.0*N^2 + 64.0*N + 48.0)/((N^7 - 14*N^5 + 49*N^3 - 36*N)*sqrt(10.66666666666667*N^2 + 64.0))

In [194]:
L[1,0].canonicalize_radical().factor()

-sqrt((N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3))*(10.66666666666667*N^3 + 48.0*N^2 + 64.0*N + 48.0)/(sqrt(10.66666666666667*N^2 + 64.0)*(N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N)

In [201]:
L[1,1]

1.0*sqrt((10.666666666666668*N^3 + 48.0*N^2 + 64.0*N + 48.0)^2/((10.666666666666668*N^2 + 64.0)*(N + 3)*(N + 2)*(N + 1)*(N - 1)*(N - 2)*(N - 3)*N^2))

In [202]:
L[1,1].canonicalize_radical()

1.0*sqrt(113.7777777777778*N^6 + 1024.0*N^5 + 3669.333333333333*N^4 + 7168.0*N^3 + 8704.0*N^2 + 6144.0*N + 2304.0)/(sqrt(10.66666666666667*N^8 - 85.33333333333334*N^6 - 373.3333333333333*N^4 + 2752.0*N^2 - 2304.0)*N)

In [203]:
L[1,1].canonicalize_radical().factor()

1.0*sqrt(113.7777777777778*N^6 + 1024.0*N^5 + 3669.333333333333*N^4 + 7168.0*N^3 + 8704.0*N^2 + 6144.0*N + 2304.0)/(sqrt(10.66666666666667*N^8 - 85.33333333333334*N^6 - 373.3333333333333*N^4 + 2752.0*N^2 - 2304.0)*N)

In [204]:
2/3

2/3

In [205]:
31/3

31/3

In [262]:
partition = [[2,2]]

In [274]:
print(symmTrPart(vpGenerate(2,2)[0]))

[[[0], [['X', 1], ['X', 2], ['Y', 3], ['Y', 0]]], [[0], [['X', 2], ['X', 3], ['Y', 1], ['Y', 0]]]]


In [286]:
a = vpGenerate(2,2)[0]

In [287]:
b = [[2,2]]

In [317]:
symmTrPart([[2,2]])

[[[2/3], [['X', 1], ['X', 2], ['Y', 3], ['Y', 0]]],
 [[1/3], [['X', 2], ['X', 3], ['Y', 1], ['Y', 0]]]]

In [288]:
a == b

True

In [292]:
type(b)

<type 'list'>

In [301]:
type(b[0][0])

<type 'sage.rings.integer.Integer'>

In [302]:
type(2)

<type 'sage.rings.integer.Integer'>

In [303]:
2/3

2/3

In [307]:
type(vpGenerate(2,2)[0][0][0])

<type 'int'>